In [1]:
# Import packages, libraries and modules

#!pip install ds
import ds
import warnings
warnings.simplefilter("ignore")
import json
import os
import pandas as pd
import numpy as np
import re
import datetime
from datetime import timedelta
from ds.utilities.io import ds_trino
import numpy as np

# Display settings
from IPython.display import HTML, display
display(
    HTML(
        data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 95%; }
    div#maintoolbar-container { width: 95%; }
</style>
"""
    )
)

# Set max rows and columns to display
pd.options.display.max_columns = None
pd.options.display.max_rows = 100
pd.options.mode.chained_assignment = None

In [2]:
# Create connection to SuperSet via ds_trino
conn = ds_trino.create_trino_connection()

Upload to test

In [3]:
# Import google modules
import pygsheets
from google.oauth2 import service_account

## TUNED

In [4]:
orders_query_2 = """
WITH

  cr_countries (facility_country_code, country, region) AS ( VALUES -- Filters which countries are pulled through, WILL CHANGE TO GOOGLE SHEET
    ('US', 'USA', 'US/CAN')
  )
  
  ,seller_org_types (seller_organization_id, org_type) AS ( VALUES
    ('508ca21a-7b27-49fc-abaa-b2babc31a141', '1PCR') -- CloudRetail Internal Brands for Owned Inventory
    ,('6d7e5e7b-8ed7-472e-8b30-7e297e098493', '2PCR') -- CloudRetail MarketPlace Internal Brands for Cross Selling
  )
  
  ,cr_orgs AS ( -- Pulls in all CloudRetail Organizations
    SELECT
      brand_organization_id AS prep_organization_id
      ,facility_id
      ,brand_organization_id IN ( -- CloudRetail Owned Inventory Organizations
          'bb2488c6-504c-4451-9658-74f7f432d8da'
      ) AS inventory_owned
      --,went_underway_date AS fac_went_underway_date 
      --,fully_trained_date AS fac_fully_trained_date -- could get rid off 
    FROM
      hudi_ingest.data_science.ds_stores_view
    /*
    LEFT JOIN
      google_spreadsheets.default.cr_facilities AS cr_facs
      USING(facility_id)
    */
    WHERE
      store_tenant_type IN ('VIRTUAL', 'CLOUD_RETAIL')
      AND is_css_facility
      AND facility_id NOT IN (
      '0a9925b9-9a1e-4f59-b10f-094a9e42274d'
      ,'983bc311-51e8-4e2e-ba27-6cc94bec511c'
      )
      AND brand_organization_id NOT IN ( --Virtual organizations that are NOT CloudRetail
        'a9a02b77-ee3f-48d0-a77e-e36ac770958b'
        ,'ac56d23b-a6a2-4c49-8412-a0a0949fb5ef'
        ,'36edf1da-d72f-4878-ae82-2d3b5644e752'
        ,'55b8f823-f905-470f-95f3-73bccbb5cbbd'
        ,'03701766-8f27-49f8-9e3d-05bbbe8fd725'
        ,'659d8941-5bd2-4203-a7a4-b16843551b94'
        ,'e8bc6b73-cbed-4d7f-9824-3be241e7a25c'
        ,'041ffc35-f8ad-4a08-98b2-d1c2ec32871a'
        ,'a6f34315-3385-435b-968f-13e131e0b7e4'
        ,'b5694cf7-63df-4791-8ad1-7ed62470a8f3' -- Homage Industrial
        ,'1249c286-4171-4b75-8ff2-de3b0253c9ae' -- US Excludes it
      )
    GROUP BY
      1,2,3
  )

  ,station_orders_raw_a AS (
    SELECT
      order_id
      ,AT_TIMEZONE(FROM_UNIXTIME(reference_time/1000, 'UTC'), facility_timezone) AS order_timestamp
      ,date_trunc('month', DATE(from_unixtime(reference_time/1000, facility_timezone))) as created_month
      ,date_trunc('week', DATE(from_unixtime(reference_time/1000, facility_timezone))) as created_week
      ,region
      ,country
      ,prep_organization_id
      ,prep_organization_name
      ,seller_organization_id
      ,seller_organization_name
      ,facility_id
      ,facility_name
      ,facility_timezone
      --,FLOOR((DATE_DIFF('day', DATE(fac_went_underway_date), DATE(AT_TIMEZONE(current_timestamp, facility_timezone))) - 1) / 7) + 1 AS underway_weeks_back_local
      --,FLOOR((DATE_DIFF('day', DATE(fac_fully_trained_date), DATE(AT_TIMEZONE(current_timestamp, facility_timezone))) - 1) / 7) + 1 AS trained_weeks_back_local
      ,inventory_owned
      ,ofo_slug
      ,CASE
        WHEN org_type = '1PCR' AND NOT inventory_owned THEN '2PCR'
        ELSE COALESCE(org_type, IF(seller_organization_id <> prep_organization_id,'2PCR','3PCR'))
      END AS distribution_type
      ,num_items AS num_items
      ,usd_subtotal AS usd_subtotal
    FROM
      hudi_ingest.api_orders.station_orders
    JOIN
      cr_orgs
      USING(prep_organization_id, facility_id)
    JOIN
      cr_countries
      USING(facility_country_code)
    LEFT JOIN
      seller_org_types
      USING(seller_organization_id)
    WHERE
      day_partition >= DATE_FORMAT(DATE_TRUNC('month', current_date) - interval '3' month, '%Y-%m-%d')
      AND NOT (customer_order_source = 'HOI' AND order_display_id IS NULL)
      AND ofo_slug <> 'Square POS'
  )
  
  ,customer_orders_raw AS (
    SELECT
      order_id
      ,is_cancelled
      ,order_issue_types
      ,cancellation_reason
      ,ofo_cancellation_reason
      ,cancellation_state
      ,ofo_status
      ,adjustment
      ,time_to_last_in_handoff_millis AS pick_time_millis
      ,confirmation_state
      ,in_handoff_at IS NOT NULL AS in_cubby
      ,handed_off_at IS NOT NULL AS is_handed_off
      ,time_to_last_trashed_millis AS cancel_time
      ,courier_arrived_at 
      ,discount
      ,order_rating
    FROM
      hudi_ingest.api_orders.customer_orders
    JOIN
      cr_countries
      USING(facility_country_code)
    WHERE
      day_partition >= DATE_FORMAT(DATE_TRUNC('month', current_date) - interval '3' month, '%Y-%m-%d')
      AND (is_cancelled OR order_issue_types IS NOT NULL)
  )
  
  ,order_scanouts AS (
    SELECT
      CAST(order_uuid AS VARCHAR) AS order_id
      ,TRUE AS order_scanned
    FROM
      hudi_ingest.inventory_restaurant_platform_cockroach_dbreplica.inventory_transaction
    WHERE
      created_at >= DATE_TRUNC('month', current_date) - interval '3' month
      AND type = 'TRANSACTION_ORDER_SCANOUT'
      AND cause = 'CAUSE_MANUAL'
      AND day_partition <> ''
    GROUP BY
      1,2
  )
  
  ,station_orders_raw AS (
    SELECT
      order_timestamp AS order_timestamp_local
      ,DATE(order_timestamp) AS order_date_local
      ,DATE(created_month) AS created_month
      ,DATE(created_week) AS created_week
      ,FLOOR((DATE_DIFF('day', DATE(order_timestamp), DATE(AT_TIMEZONE(current_timestamp, facility_timezone))) - 1) / 7) + 1 AS order_weeks_back_local
      ,order_id
      ,region
      ,country
      ,prep_organization_id
      ,prep_organization_name
      ,seller_organization_id
      ,seller_organization_name
      ,facility_id
      ,facility_name
      ,facility_timezone

      ,(
CASE
   WHEN
      facility_id IN 
      (
         '2019e0f4-7fd8-47fd-a858-9e5c4b409a39',  -- DEN -  AURORA
         'ab647486-a823-456e-8058-053d052106de',   -- DEN - GRAPE
         '1e55ab2f-64a3-498c-a692-9614b79ab834'   -- DEN - VALLEJO
      )
   THEN
      'Denver'
   WHEN
      facility_id IN 
      (
         'b0629713-f1bd-4d2b-a065-46ff82645987',  -- SF - CHARTER OAK
         '7d248b5d-c113-4269-8f4a-68d155a48adf'  -- SF - SoMa
      )
   THEN
      'San Francisco'
   WHEN
      facility_id IN 
      (
         '5d574e15-e1d4-42a3-837a-71eebe29728a',  -- OAK - ADELINE
         'bc9dcc91-8fec-4a62-8438-024a4f65e8e8'  -- OAK - JINGLETOWN
      )
   THEN
      'Oakland'
   WHEN
      facility_id IN 
      (
         '905cbe43-327f-4d3e-a897-48ee81a64ade'  -- SJO - Ruff (949 Ruff)
      )
   THEN
      'San Jose'
   WHEN
      facility_id IN 
      (
         '1305bca6-90ca-4292-865f-2b9da0a3811e'  -- NA-US-OR-PDX-Grand
      )
   THEN
      'Oregon'
   WHEN
      facility_id IN 
      (
         '557194c1-5796-469c-b3e0-d83736880340'  -- SF - San Mateo (21st Street)
      )
   THEN
      'San Mateo'
   WHEN
      facility_id IN 
      (
         '9c7b97b6-878f-4007-9032-558c4c6157e9'  -- LA - Anaheim (Lewis)
      )
   THEN
      'Anaheim'
   WHEN
      facility_id IN 
      (
         'f0029277-0e7e-410a-ad4e-5247b892a280',  -- LA - Hollywood (Western)
         '99573ac0-a3f2-4c70-946c-e9c06b425d5c',  -- LA - Pico Union (1842 Washington)
         '11bcb365-1910-40e7-a25a-4d3dd15f6eb2',  -- LA - South Bay (Hawthorne)
         '5315a66f-8337-4c68-af42-86089a86f8ba',  -- LA - Long Beach (Daisy)
         '55a786a5-1a9d-4611-95e2-81009b44ebdb',  -- NA-US-CA(S)-LA-Pico-SA
         '9732b09c-9d18-4f8e-a5d4-34173ddf1ea4',  -- LA - Culver City (Selmaraine)
         '5efa5206-dc53-4ab4-9f49-e5a5a4dc1c34'  -- LA - University Park (W 38th)
      )
   THEN
      'Los Angeles'
   WHEN
      facility_id IN 
      (
         '4fff0653-ae90-4e98-a67d-f67001e33c65',  -- SD - College Area (El Cajon)
         'c5c4ed02-1518-4970-a68f-432c75ecf6e2'  -- SD - Downtown (Boston)
      )
   THEN
      'San Diego'
   WHEN
      facility_id IN 
      (
         '2fe8235d-fe85-46e1-be54-023a65e34939'  -- LAS - North Strip (St Louis)
      )
   THEN
      'Las Vegas'
   WHEN
      facility_id IN 
      (
         '2d661ae4-d4bf-4d88-8af1-2df4f7d0fd75'  -- AZ - Tucson (E Delano ST)
      )
   THEN
      'Tucson'
   WHEN
      facility_id IN 
      (
         '937ddb7e-4c76-4cfd-bbb8-bd07b33ee7e9',  -- PHX - Midtown (W Highland)
         'a9eb0fc1-7699-43b4-bd4c-6bb5b8195a87'  -- PHX - Tempe (E 5th)
      )
   THEN
      'Phoenix'
   WHEN
      facility_id IN 
      (
         '54341152-da81-4b58-8ba3-8604bc1ff2dc',  -- CHI - Avondale (Spaulding)
         '0f9ac987-32ac-4b5a-91bf-78cbc7ea44bb',  -- CHI - Humboldt Park (Grand)
         '75187d2a-01d8-467f-9993-90e200a3a586',  -- CHI - River West (Superior)
         '6ab373fb-ee91-4dc6-b529-ef67f6326588',  -- CHI - North Center (Rockwell)
         '94e1cb0b-facf-4f72-b66d-2bd97d98e58f'  -- CHI - South Loop (Wabash)
      )
   THEN
      'Chicago'
    WHEN
      facility_id IN 
      (
         '0a25ff7d-1697-4e0c-a394-71347b0cdcc3',  -- Minneapolis - Blaine
         '99174fc4-d9f4-424f-96d8-e499a85e85b6'  -- Minneapolis - Currie Avenue
      )
   THEN
      'Minneapolis'
   WHEN
      facility_id IN 
      (
         '4f7010ef-ba20-4a6a-927b-d1a37903ef5e'  -- OH - Cleveland - Central (Carnegie Ave)
      )
   THEN
      'Cleveland'
   WHEN
      facility_id IN 
      (
         '6936d9f5-887a-4753-9c77-029c0164d956'  -- COL - Short North (Essex)
      )
   THEN
      'Columbus'
   WHEN
      facility_id IN 
      (
         'e1198539-7644-4c6d-9def-e64fa1cf4fb5',  -- MIA - South Beach (Alton)
         'd5f67e55-66be-4742-81c1-4720bb6906d8'  -- MIA - Wynwood (Miami Ct)
      )
   THEN
      'Miami'
   WHEN
      facility_id IN 
      (
         'd1814be1-e541-484d-9fff-9a4d800bfa26'  -- ORL - West Downtown (Dollins)
      )
   THEN
      'Orlando'
   WHEN
      facility_id IN 
      (
         'ba4cd06d-e256-4562-8ffc-372c7a2ead8c',  -- HOU - Midtown (Blodgett)
         '533a8386-c2a0-4e67-b6a3-ecea769ff764'  -- HOU - Galleria (Fairdale)
      )
   THEN
      'Houston'
   WHEN
      facility_id IN 
      (
         '68907756-0bb0-408f-a45c-3ab38083a57d',  -- DFW - Cullen (3004 Cullen)
         '2e30d25a-5c6b-46bf-b3f6-2030008dd5fd'  -- DFW - Towerwood (3230 Towerwood)
      )
   THEN
      'Fortworth'
   WHEN
      facility_id IN 
      (
         '4411e09a-c8b7-4a2e-ae6e-5d028e303791'  -- DAL - Trinity Groves (Commerce)
      )
   THEN
      'Dallas'
   WHEN
      facility_id IN 
      (
         'ee395d7f-589b-4208-a745-90a8a0aa3bfd'  -- San Antonio - Lombrano
      )
   THEN
      'San Antonio'
   WHEN
      facility_id IN 
      (
         'ce1e2ca1-6944-48d4-a93c-67bf011560f6'  -- ATX - North Loop (Hwy 35)
      )
   THEN
      'Austin'
   WHEN
      facility_id IN 
      (
         '6f24178a-58f3-4d0a-a2c3-107ad22dcd4e',  -- ATL - Midtown (Forrest)
         '288c069f-0d07-4d56-8d10-dc8afab2cb94'  -- ATL - West End (Oak)
      )
   THEN
      'Georgia'
   WHEN
      facility_id IN 
      (
         '305dca7a-ffcf-4ecf-9363-4238af3cba6b',  -- PHL - University City (Fairmount)
         'efa863fe-4cf4-4552-b73d-253470878e8f'  -- PHL - North City Center (1308 W Girard)
      )
   THEN
      'Pennsylvania'
   WHEN
      facility_id IN 
      (
         'e92e292a-4ca8-4cbd-8b7d-82d275637c1f'  -- New Jersey - Belleville
      )
   THEN
      'New Jersey'
   WHEN
      facility_id IN 
      (
         'e380140f-dfa9-4b17-84aa-61af20324195'  -- PROV - Smith Hill (Bath)
      )
   THEN
      'Providence'
   WHEN
      facility_id IN 
      (
         '6a896a7e-ad07-4f06-87e9-1af5ea87d6cb',  -- NYC - Long Island City
         '65820060-3a5f-4561-9f7a-9a7e34c581c1',  -- NYC - Brooklyn (Utica)
         '9854b8e1-8c33-4ee0-b6b5-4ccbc2012a38',  -- NYC - Astoria (24-26 47th)
         '9f5e4667-6a63-4af6-a805-476c979884e2',  -- NY - The Bronx (Bullard Ave)
         '9209f242-c27f-4174-b941-73e4f7b00e56',  -- NA-US-NY-NYC-Broadway 
         '05db32ee-158c-4baa-97c6-402bba3fcad3',  -- NA-US-NY-NYC-Church
         '628e10de-20b0-400b-a006-8e023e4f5c4d',  -- NA-US-NY-Brooklyn-Commerce
         '7d62cf8a-dfc8-4988-a02e-0b27303b19e2'  -- NA-US-NY-NYC-Devoe
      )
   THEN
      'New York City'
   WHEN
      facility_id IN 
      (
         '8365d2e5-dd91-4a87-beaa-cb54c374acdc'  -- NA-US-NY-BUF-Military
      )
   THEN
      'Buffalo'
    WHEN
      facility_id IN 
      (
         '54d4375f-4c79-4444-ad83-f54362c08198'  -- NA-US-DC-5th
      )
   THEN
      'DC'
END
) AS city
      ,ofo_slug
      --,underway_weeks_back_local
      --,trained_weeks_back_local
      ,inventory_owned
      ,distribution_type
      ,customer_orders_raw.cancellation_reason -- NAT
      ,customer_orders_raw.ofo_cancellation_reason
      ,customer_orders_raw.cancellation_state
      ,customer_orders_raw.ofo_status
      ,customer_orders_raw.is_handed_off
      ,customer_orders_raw.in_cubby
      ,customer_orders_raw.cancel_time BETWEEN 1 AND 1200000 AS order_20cancel
      ,customer_orders_raw.courier_arrived_at as courier_arrived_at
      ,is_cancelled
      ,CASE
        WHEN is_cancelled THEN 'cancel'
        WHEN order_issue_types IS NOT NULL THEN 'issue'
        ELSE 'success'
      END AS defect_type
      ,CASE
        WHEN is_cancelled AND cancellation_reason = 'UNAVAILABLE_ITEM' THEN 'item_unavailable'
        WHEN is_cancelled AND confirmation_state <> 'CONFIRMATION_CONFIRMED' THEN 'confirmation_failed'
        WHEN is_cancelled AND NOT is_handed_off THEN 'handoff_failed'
        WHEN is_cancelled AND pick_time_millis > 1200000 THEN 'pick_over_20' --Pick time was over 20 minutes long
        WHEN is_cancelled THEN 'other_cancel'
        WHEN array_join(order_issue_types, ', ') LIKE '%WRONG_ORDER%' THEN 'order_wrong'
        WHEN array_join(order_issue_types, ', ') LIKE '%MISSING_ITEM%' OR array_join(order_issue_types, ', ') LIKE '%WRONG_QUANTITY%' THEN 'item_missing'
        WHEN array_join(order_issue_types, ', ') LIKE '%WRONG_ITEM%' OR array_join(order_issue_types, ', ') LIKE '%WRONG_SIZE%' THEN 'item_wrong'
        WHEN array_join(order_issue_types, ', ') LIKE '%INCOMPLETE%' THEN 'order_incomplete'
        ELSE 'other_issue'
      END AS defect_reason
      ,COALESCE(IF(adjustment IS NOT NULL AND (is_cancelled OR order_issue_types IS NOT NULL), adjustment < 0), FALSE) AS is_refunds
      ,pick_time_millis BETWEEN 1 AND 1200000 AS order_20picked --TRUE if picked in under 20 minutes
      ,num_items
      ,usd_subtotal AS gmv
      ,customer_orders_raw.adjustment AS lost_gmv
      ,customer_orders_raw.discount
      ,order_rating
      ,order_scanned
    FROM
      station_orders_raw_a
    LEFT JOIN
      customer_orders_raw
      USING(order_id)
    LEFT JOIN
      order_scanouts
      USING(order_id)
  )
  

SELECT
--  AT_TIMEZONE(current_timestamp, 'America/Los_Angeles') AS pulled_date_pst
--  ,YEAR(order_timestamp_local) AS order_year_local
--  ,MONTH(order_timestamp_local) AS order_month_local
--  ,order_weeks_back_local
--  ,order_date_local
--  ,HOUR(order_timestamp_local) AS order_hour_local -- feed heat maps
--  ,region
--  ,country
 -- created_month
  created_week
  ,country
  ,prep_organization_id
  ,prep_organization_name
  ,seller_organization_id
  ,seller_organization_name
  ,facility_id
  ,facility_name
--  ,is_handed_off
--  ,in_cubby
--  ,cancellation_reason
--  ,ofo_cancellation_reason
--  ,cancellation_state
--  ,ofo_status
--  ,CASE
--    WHEN trained_weeks_back_local IS NULL AND underway_weeks_back_local IS NULL THEN 'untouched'
--    WHEN trained_weeks_back_local IS NOT NULL AND order_weeks_back_local <= trained_weeks_back_local THEN 'trained'
--    WHEN underway_weeks_back_local IS NOT NULL AND order_weeks_back_local <= underway_weeks_back_local THEN 'underway'
--    ELSE 'untouched'
--  END AS facility_training_status
  ,inventory_owned
  ,distribution_type
--  ,defect_type
--  ,IF(defect_type = 'success', 'success', defect_reason) AS defect_reason
--  ,CASE
--    WHEN defect_type = 'success' then 'not included'
--    WHEN defect_type = 'issue' AND is_refunds = TRUE then 'included'-- order issue, adjustment
--    WHEN defect_type = 'issue' AND is_refunds = FALSE then 'not included' -- order issue, no adjustment
--    WHEN defect_type = 'cancel' AND in_cubby = FALSE then 'included' -- 
--    WHEN defect_type = 'cancel' AND in_cubby = TRUE AND cancellation_reason IS NOT NULL THEN 'included' --if cancelation reason is null then OFO cancelled
--    WHEN defect_type = 'cancel' AND in_cubby = TRUE AND cancellation_reason IS NULL AND order_20picked = FALSE THEN 'included' -- yes 
--    WHEN defect_type = 'cancel' and courier_arrived_at IS NOT NULL AND cancellation_reason IS NULL AND order_20cancel = TRUE THEN 'included' -- OFO cancel < 20
--    WHEN defect_type = 'cancel' AND in_cubby = TRUE AND cancellation_reason IS NULL AND order_20picked = TRUE THEN 'not included' -- driver didnt show 
--    ELSE 'check logic'
--  END AS odr_logic
--  ,CASE 
--    WHEN defect_type = 'cancel' AND cancellation_reason IS NULL THEN 'OFO'
--    WHEN defect_type = 'cancel' AND cancellation_reason IS NOT NULL THEN 'Facility'
--    ELSE 'Not Cancelled'
--  END AS cancellation_party
--  ,is_refunds
  ,city
  ,COUNT(DISTINCT order_id) AS total_orders
  ,COUNT(DISTINCT CASE WHEN is_cancelled THEN order_id END) as cancelled_orders --
  ,COUNT (DISTINCT CASE WHEN defect_type = 'issue' THEN order_id END) as issue_orders --
  ,COUNT(DISTINCT CASE WHEN defect_type != 'success' THEN order_id END) as ODR_orders
  ,COUNT(DISTINCT order_id) FILTER (WHERE order_scanned) AS scanned_orders
  ,AVG(num_items) AS items
  ,COALESCE(ROUND(SUM(gmv), 2), 0.00) AS gmv
  ,COALESCE(ROUND(SUM(lost_gmv), 2), 0.00) AS lost_gmv
  ,SUM(CASE WHEN is_cancelled THEN coalesce(gmv, 0) ELSE 0 END) AS cancelled_gmv
  ,avg(order_rating) AS avg_order_rating
  ,COUNT(DISTINCT order_id) FILTER (WHERE order_20picked) AS orders_20picked
  ,COUNT (DISTINCT ofo_slug) as OFOs
  ,COUNT (DISTINCT city) as cities
  ,COUNT(DISTINCT facility_id) as facilities
FROM
  station_orders_raw
WHERE
  order_date_local <> AT_TIMEZONE(current_timestamp, facility_timezone)
GROUP BY
  1,2,3,4,5,6,7,8,9, 10,11
ORDER BY
  4 DESC, prep_organization_id

"""

print("Querying data...")
orders_data_2 = ds_trino.fetch_data(conn=conn, sql_str= orders_query_2, use_cache=False)
print("Data pulled!")
orders_data_2 = orders_data_2.replace(np.nan,'',regex=False)
print("replacing nan values in df")
orders_data_2.head()

Querying data...
Data pulled!
replacing nan values in df


,created_week,country,prep_organization_id,prep_organization_name,seller_organization_id,seller_organization_name,facility_id,facility_name,inventory_owned,distribution_type,city,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,items,gmv,lost_gmv,cancelled_gmv,avg_order_rating,orders_20picked,OFOs,cities,facilities
0,2023-08-07,USA,017b46ad-3a10-4d42-80f2-b3b84abb1019,​Tubby's Taste LLC,017b46ad-3a10-4d42-80f2-b3b84abb1019,​Tubby's Taste LLC,75187d2a-01d8-467f-9993-90e200a3a586,NA-US-IL-CHI-River West (Correct),False,3PCR,Chicago,1,0,0,0,1,3.000000,27.00,0.0,0.00,,0,1,1,1
1,2023-08-07,USA,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,4411e09a-c8b7-4a2e-ae6e-5d028e303791,NA-US-TX-DAL-Commerce,False,3PCR,Dallas,6,5,0,5,0,2.333333,127.66,0.0,106.68,,0,1,1,1
2,2023-07-31,USA,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,4411e09a-c8b7-4a2e-ae6e-5d028e303791,NA-US-TX-DAL-Commerce,False,3PCR,Dallas,2,2,0,2,0,2.000000,37.72,0.0,37.72,,0,1,1,1
3,2023-08-14,USA,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,4411e09a-c8b7-4a2e-ae6e-5d028e303791,NA-US-TX-DAL-Commerce,False,3PCR,Dallas,1,1,0,1,0,1.000000,3.99,0.0,3.99,,0,1,1,1
4,2023-07-31,USA,6440b884-ccd2-465d-b220-b93b8c7cda62,p.o.p. candy co.,6440b884-ccd2-465d-b220-b93b8c7cda62,p.o.p. candy co.,99573ac0-a3f2-4c70-946c-e9c06b425d5c,NA-US-CA(S)-LA-Koreatown,False,3PCR,Los Angeles,2,2,0,2,1,2.000000,64.00,0.0,64.00,,0,1,1,1


LATAM

In [5]:
client_query = """

SELECT organization_id,name 
FROM hudi_ingest.processed.store_config_organization_view

"""
print("Querying data...")
client_data = ds_trino.fetch_data(conn=conn, sql_str= client_query, use_cache=False)
print("Data pulled!")
print(len(client_data))

Querying data...
Data pulled!
341898


Latam Tuned

In [6]:
orders_query_2_latam = """

with ordenes as (



with scan_out as ( SELECT
      CAST(order_uuid AS VARCHAR) AS order_id
      ,TRUE AS order_scanned_out
      ,count ((order_uuid)) as scanned_items
    FROM
     
      hudi_ingest.inventory_restaurant_platform_cockroach_dbreplica.inventory_transaction 
    WHERE
      day_partition>= DATE_FORMAT(DATE_TRUNC('month', current_date) - interval '3' month, '%Y-%m-%d')
      
      AND type = 'TRANSACTION_ORDER_SCANOUT'
      AND cause = 'CAUSE_MANUAL'
    GROUP BY
      1,2)



SELECT 
DATE(from_unixtime(reference_time/1000, facility_timezone)) AS created_date,
--que empiece en domingo
date_trunc('month', DATE(from_unixtime(reference_time/1000, facility_timezone))) as created_month,
date_trunc('week', DATE(from_unixtime(reference_time/1000, facility_timezone))) as created_week,

--date_trunc('week', DATE(from_unixtime((reference_time)/1000, facility_timezone))+ interval '1' day)- interval '1' day as created_week,
FLOOR((DATE_DIFF('day', DATE(created_at_ts), DATE(AT_TIMEZONE(current_timestamp, facility_timezone))) - 3) / 7) + 1 AS order_weeks_back_local,
cast(local_created_at_ts as time) as local_created_time,
facility_timezone,
'LATAM' as region,
dc.country,



CASE 

when dc.country='Brazil' then 'Brazil'

else 'SSL'

end as subregion,


(
CASE
   WHEN
      facility_id IN 
      (
         'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7',  -- LIM - Surquillo
         '6d2d117b-3fa9-4a2c-85b5-9b86a758f849'   -- LIM - Magdalena
      )
   THEN
      'Lima'
   WHEN
      facility_id IN 
      (
         'd65136fe-862a-4da5-81ef-580c28f8c61a'  -- UIO - 10 de Agosto
      )
   THEN
      'Quito'
   WHEN
      facility_id IN 
      (
         '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf', -- BOG - Prado
         '199c5433-d863-4d5e-bb6e-a53d91e0605f', -- Modelia  BOGOTA
         '4d78d1e1-5bce-4307-8175-51c816120fce', -- BOG - Castellana
         'b2f57126-8e64-4b11-964f-c497a3b53571', -- BOG - Cocinas de la 73
         'a5852445-e223-46c9-be99-ba33f204bcd1', -- BOG - Puente Aranda
         '76a6ef30-ddb2-4178-8c37-86c0d10db4b3', -- BOG - Atabanza
         'dd16bf0e-8a89-4f21-bc02-1c9793227502', -- BOG - San Felipe
         '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82',  -- BOG - Cedritos
         '757182ff-7a54-4571-91d4-0027253c0117' --Salitre
      )
   THEN
      'Bogota'
  
  WHEN
      facility_id in 
      ('0016d9fc-4f27-4a05-88cb-90cd1cdf311f' --demoda
      )
  THEN 
      'Medellin'
      
      
      
   WHEN
      facility_id in 
      (
         '46fb5e19-3a79-4eda-b119-dee3a331fd81'  -- CLO - Chipichape
      )
   THEN
      'Cali' 
   WHEN
      facility_id in 
      (
         '58b818c6-615e-4316-b4a3-b9a6bb914c30'  --BAQ - Alto Prado
      )
   THEN
      'Barranquilla' 
   WHEN
      facility_id in 
      (
         '9eede066-b7e7-43c4-907c-21ece1688b14',  -- SJO - Guadalupe
         '967f0a2e-9c8e-4b35-929b-37420d9439c9'   -- SJO - Pavas
      )
   THEN
      'San Jose - CR' 
   WHEN
      facility_id in 
      (
         '47e2d054-8b82-420c-b1b0-18010f57489c', -- MEX - Insurgentes Sur
         '565273fe-0074-421d-b1e1-796d1e4390e1', -- MEX - Napoles
         '864e0a3c-ecba-4305-b3ff-f22edd58c902', -- MEX - Polanco
         'bcb04949-3cda-4ca0-be39-b9f579e356ca', -- MEX - Roma
         '9101d4f9-828f-45c5-b8b6-16986e76b374', -- MEX - Pedregal
         'b9545910-bca6-4b2a-a603-363721e7a23d', -- MEX - Santa Fe
         'da27f06f-01fd-4106-88d9-b3fd41446515'  -- MEX - Lindavista
      )
   then
      'Mexico City'
   when
      facility_id in 
      (
         '394c0103-1c5f-424b-b398-68455fced5bb', -- MTY - Cumbres
         '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f', -- MTY - Leones NK
         '988c22e5-20c2-42a5-bc30-989c68ff5eee', -- MTY - Tec
         '1b90b4ae-6d8f-4a73-bb9e-708c83387430', -- MTY - Santa Maria
         '88774bba-2e34-4771-8e64-8f059adf02a3'  -- MTY - Valle Oriente
      )
   then
      'Monterrey'
   when
      facility_id in 
      (
         'a9a6c4be-32b7-47cf-b783-7a5227a89ed4',  -- GDL - Ciudad del Sol
         'c66ce711-5486-433f-a9b7-1b36ae3a6dee' -- GDL - av mexico
      )
   then
      'Guadalajara'
   WHEN
      facility_id IN
      (
         '7b5c774f-7739-4d8e-a392-d49547d45fc3', -- BHZ - Contagem Industrial
         '554263c6-bde4-4d4c-92a7-21ed77e122cd', -- BHZ - Paracatu
         '522c745a-fa86-42de-83ca-227cbd3cb353', -- BHZ - Arthur Guimaraes
         '4518d99c-fa47-4da7-b5f5-c071c78b5ac2'  -- BHZ - Tancredo Neves'
      )
   then
      'Belo Horizonte' 
   WHEN
      facility_id IN
      (
         '81e6538d-c45b-4479-92e8-fc4e04373e12', -- RIO - Jose Bonifacio
         '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb', -- RIO - Barra
         '0ee5dccf-ed5b-465c-9858-b53351353e3b',  -- RIO - Camerino
         '976ae838-18e0-40bb-a8dc-360353c01b6d', -- RIO - Niteroi
         '3759ad21-4537-48ea-b1f1-48809e8a03ac', --rio botafogo
         'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6' -- RIO - Tijuca
      )
   then
      'Rio de Janeiro' 
   WHEN
      facility_id IN
      (
         '6c601837-403c-425f-a51c-31428ed1a67e', -- SAO - Rua do Acre
         '87a40094-cf48-4032-8166-e4cee61bce2f', -- SAO - Dino
         '4326199f-628a-4b00-a6e6-054fb5182c64', -- SAO - Guararapes (Brooklin)
         'dcc461ae-c80c-46cc-a5dc-4a5a72761510', -- SAO - Lapa Clelia
         '7aae3115-311b-449f-88f4-0f1612ecfcee', -- SAO - Borba
         '1840180b-9a3b-4fb6-b35c-ef4e6785b26e', -- SAO - Abelardo
         '88aaaf99-a444-4e0f-99f6-267b635faa53',  -- SAO - Vargas
         'a63559d1-93c6-4f60-b683-1e207ccda10a', -- SAO - Batista
          --'89cf86b9-9e74-4657-86df-2a851492af88', -- SAO - Guarulhos
          '89cf86b9-9e74-4657-86df-2a851492af88', --sao espanhola
          '14d9bb92-6376-4d69-adb6-64f97e4b3561', -- sao ferreira
          '812e6913-d78e-4dbc-877d-16993853aa7e' -- sao domingos
      )
   then
      'Sao Paulo'
      
  WHEN 
        facility_id IN 
        (
        'c559b06b-e193-495c-85cd-a5d1608ca27d'  -- BSB - Aguas Claras
        )
    THEN
        'Brasilia'
      
      
    WHEN
        facility_id IN
        (
            'db3a588c-59ac-458e-84de-9e1c8ef5e3e8' -- 'SCL - Recoleta'
        )
    THEN
        'Santiago'
END
) AS city,
(
CASE
   WHEN facility_id = 'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7' THEN 'LIM - Surquillo'
   WHEN facility_id = '6d2d117b-3fa9-4a2c-85b5-9b86a758f849' THEN 'LIM - Magdalena'
   WHEN facility_id = 'd65136fe-862a-4da5-81ef-580c28f8c61a' THEN 'UIO - 10 de Agosto'
   WHEN facility_id = '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf' THEN 'BOG - Prado'
   WHEN facility_id = '4d78d1e1-5bce-4307-8175-51c816120fce' THEN 'BOG - Castellana'
   WHEN facility_id = 'b2f57126-8e64-4b11-964f-c497a3b53571' THEN 'BOG - Cocinas de la 73'
   WHEN facility_id = 'a5852445-e223-46c9-be99-ba33f204bcd1' THEN 'BOG - Puente Aranda'
   WHEN facility_id = '76a6ef30-ddb2-4178-8c37-86c0d10db4b3' THEN 'BOG - Atabanza'
   WHEN facility_id = 'dd16bf0e-8a89-4f21-bc02-1c9793227502' THEN 'BOG - San Felipe'
   WHEN facility_id = '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82' THEN 'BOG - Cedritos'
   when facility_id = '757182ff-7a54-4571-91d4-0027253c0117' Then 'BOG - Salitre 2 EM'
   when facility_id = '0016d9fc-4f27-4a05-88cb-90cd1cdf311f' THEN 'MED - DModa'
 
   
   WHEN facility_id = '46fb5e19-3a79-4eda-b119-dee3a331fd81' THEN 'CLO - Chipichape' 
   WHEN facility_id = '58b818c6-615e-4316-b4a3-b9a6bb914c30' THEN 'BAQ - Alto Prado' 
   WHEN facility_id = '9eede066-b7e7-43c4-907c-21ece1688b14' THEN 'SJO - Guadalupe' 
   WHEN facility_id = '967f0a2e-9c8e-4b35-929b-37420d9439c9' THEN 'SJO - Pavas'
   
   WHEN facility_id = '47e2d054-8b82-420c-b1b0-18010f57489c' THEN 'MEX - Insurgentes Sur'
   WHEN facility_id = '565273fe-0074-421d-b1e1-796d1e4390e1' THEN 'MEX - Napoles'
   WHEN facility_id = '864e0a3c-ecba-4305-b3ff-f22edd58c902' THEN 'MEX - Polanco'
   WHEN facility_id = 'bcb04949-3cda-4ca0-be39-b9f579e356ca' THEN 'MEX - Roma'
   WHEN facility_id = '9101d4f9-828f-45c5-b8b6-16986e76b374' THEN 'MEX - Pedregal'
   WHEN facility_id = 'b9545910-bca6-4b2a-a603-363721e7a23d' THEN 'MEX - Santa Fe'
   WHEN facility_id = 'da27f06f-01fd-4106-88d9-b3fd41446515' THEN 'MEX - Lindavista'
   
   WHEN facility_id = '394c0103-1c5f-424b-b398-68455fced5bb' THEN 'MTY - Cumbres'
   WHEN facility_id = '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f' THEN 'MTY - Leones NK'
   WHEN facility_id = '988c22e5-20c2-42a5-bc30-989c68ff5eee' THEN 'MTY - Tec'
   WHEN facility_id = '1b90b4ae-6d8f-4a73-bb9e-708c83387430' THEN 'MTY - Santa Maria'
   WHEN facility_id = '88774bba-2e34-4771-8e64-8f059adf02a3' THEN 'MTY - Valle Oriente'
   
   WHEN facility_id = 'a9a6c4be-32b7-47cf-b783-7a5227a89ed4' THEN 'GDL - Ciudad del Sol'
   WHEN facility_id = 'c66ce711-5486-433f-a9b7-1b36ae3a6dee' THEN 'GDL - Av. Mexico'
    
   WHEN facility_id = '7b5c774f-7739-4d8e-a392-d49547d45fc3' THEN 'BHZ - Contagem Industrial'
   WHEN facility_id = '554263c6-bde4-4d4c-92a7-21ed77e122cd' THEN 'BHZ - Paracatu'
   WHEN facility_id = '522c745a-fa86-42de-83ca-227cbd3cb353' THEN 'BHZ - Arthur Guimaraes' 
   WHEN facility_id = '4518d99c-fa47-4da7-b5f5-c071c78b5ac2' THEN 'BHZ - Tancredo Neves'
   
   WHEN facility_id = '81e6538d-c45b-4479-92e8-fc4e04373e12' THEN 'RIO - Jose Bonifacio'
   WHEN facility_id = '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb' THEN 'RIO - Barra' 
   WHEN facility_id = '0ee5dccf-ed5b-465c-9858-b53351353e3b' THEN 'RIO - Camerino'
   WHEN facility_id = '976ae838-18e0-40bb-a8dc-360353c01b6d' THEN 'RIO - Niteroi'
   WHEN facility_id = 'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6' THEN 'RIO - Tijuca'
   WHEN facility_id = '3759ad21-4537-48ea-b1f1-48809e8a03ac' then 'RIO - Botafogo'
   
   WHEN facility_id = '6c601837-403c-425f-a51c-31428ed1a67e' THEN 'SAO - Rua do Acre'
   WHEN facility_id = '87a40094-cf48-4032-8166-e4cee61bce2f' THEN 'SAO - Dino'
   WHEN facility_id = '4326199f-628a-4b00-a6e6-054fb5182c64' THEN 'SAO - Guararapes (Brooklin)'
   WHEN facility_id = 'dcc461ae-c80c-46cc-a5dc-4a5a72761510' THEN 'SAO - Lapa Clelia'
   WHEN facility_id = '7aae3115-311b-449f-88f4-0f1612ecfcee' THEN 'SAO - Borba'
   WHEN facility_id = '88aaaf99-a444-4e0f-99f6-267b635faa53' THEN 'SAO - Vargas'
   WHEN facility_id = '1840180b-9a3b-4fb6-b35c-ef4e6785b26e' THEN 'SAO - Abelardo'
   WHEN facility_id = 'a63559d1-93c6-4f60-b683-1e207ccda10a' THEN 'SAO - Batista'
  -- WHEN facility_id = '89cf86b9-9e74-4657-86df-2a851492af88' THEN 'SAO - Guarulhos'
   when facility_id = '14d9bb92-6376-4d69-adb6-64f97e4b3561' THEN 'SAO - Ferreira'
    WHEN facility_id = '89cf86b9-9e74-4657-86df-2a851492af88' THEN 'SAO - Espanhola'
    WHEN facility_id = '812e6913-d78e-4dbc-877d-16993853aa7e' THEN 'SAO - Domingos'
  
   WHEN facility_id = 'c559b06b-e193-495c-85cd-a5d1608ca27d' THEN 'BSB - Aguas Claras'
   
   
  
 
   WHEN facility_id = 'db3a588c-59ac-458e-84de-9e1c8ef5e3e8' THEN 'SCL - Recoleta'

  
END
) AS facility_sf_name,
facility_name,
organization_id,
brand_id, 
co.order_id,
facility_id,
order_display_id,
ofo_slug, store_link_service_slug,
facility_country_code as country_code, 
is_cancelled,
oc_order_issue_count,
order_issue_details,
hoi_order_issue_details,
oc_order_issue_details,
order_issue_types,
hoi_order_issue_types,
oc_order_issue_types,

subtotal,
total,

commission, discount,	tip, tip_for_restaurant, tip_for_courier, delivery_fee, marketing_fee, service_fee

,ROW_NUMBER() OVER (PARTITION BY ofo_slug, external_order_id, order_items_quantity, subtotal ORDER BY co.order_id) as row_number

,cast(local_accepted_at_last_ts as TIME ) as accepted_local_time,
local_processed_at_ts,local_ordered_at_ts,local_accepted_at_ts, local_in_handoff_at_ts,
local_handed_off_at_ts,local_printed_at_ts,local_trashed_at_ts,local_pickup_picked_up_at_ts,
local_courier_checked_in_at_ts,local_courier_dropped_off_at_ts,external_order_id,fulfillment_state,fulfillment_mode,
pickup_state,scheduling_type,ofo_status,order_status
,cancellation_state,cancellation_reason as facility_cancellation_reason
,ofo_cancellation_reason,order_rating, order_review_full_text,location_state,

case when location_state='LOCATION_HANDED_OFF' then 1
  else 0
  end
as is_handed_off_num,

Case When in_handoff_at IS NOT NULL THEN 1
  else 0
  END 
as is_in_handoff_num,




CASE
        WHEN is_cancelled THEN 'cancel'
        WHEN order_issue_types IS NOT NULL THEN 'issue'
        ELSE 'success'
        END AS defect_type,

CASE
        WHEN is_cancelled AND cancellation_reason = 'UNAVAILABLE_ITEM' THEN 'item_unavailable'
        WHEN is_cancelled AND confirmation_state <> 'CONFIRMATION_CONFIRMED' THEN 'confirmation_failed'
        WHEN is_cancelled AND  location_state!='LOCATION_HANDED_OFF' THEN 'handoff_failed'
        WHEN is_cancelled AND time_to_last_in_handoff_millis > 1200000 THEN 'pick_over_20' --Pick time was over 20 minutes long
        WHEN is_cancelled THEN 'other_cancel'
        WHEN array_join(order_issue_types, ', ') LIKE '%WRONG_ORDER%' THEN 'order_wrong'
        WHEN array_join(order_issue_types, ', ') LIKE '%MISSING_ITEM%' OR array_join(order_issue_types, ', ') LIKE '%WRONG_QUANTITY%' THEN 'item_missing'
        WHEN array_join(order_issue_types, ', ') LIKE '%WRONG_ITEM%' OR array_join(order_issue_types, ', ') LIKE '%WRONG_SIZE%' THEN 'item_wrong'
        WHEN array_join(order_issue_types, ', ') LIKE '%INCOMPLETE%' THEN 'order_incomplete'
        ELSE 'other_issue'
      END AS defect_reason,

COALESCE(IF(adjustment IS NOT NULL AND (is_cancelled OR order_issue_types IS NOT NULL), adjustment < 0), FALSE) AS is_refunds
,time_to_last_in_handoff_millis BETWEEN 1 AND 1200000 AS order_20picked --TRUE if picked in under 20 minutes

,order_source, hoi_only_order_type,order_items_quantity,scan_out.order_scanned_out,


printed_at_ts, in_handoff_at_ts, handed_off_at_ts, courier_checked_in_at_ts, courier_dropped_off_at_ts, courier_arrived_at_ts

,scanned_items

,customer_note

,concat('https://tools.cloudkitchens.com/order-inspector/order-details/',co.order_id,'/overview') as link


FROM hudi_ingest.analytics_views.customer_orders_with_hoi_curr co
JOIN hudi_ingest.scratch.dim_countries dc 
   ON co.facility_country_code = dc.country_code AND region = 'LATAM'
   
left join scan_out
on co.order_id=scan_out.order_id






--aqui se filtra la fecha    
WHERE co.day_partition >= DATE_FORMAT(DATE_TRUNC('month', current_date) - interval '3' month, '%Y-%m-%d')

AND reference_time/1000 IS NOT NULL 
AND DATE(from_unixtime(reference_time/1000, facility_timezone)) >= DATE '2021-01-15'
AND tenant_type IN ('VIRTUAL', 'CLOUD_RETAIL')

--AND is_cancelled

AND facility_id IN ( -- 
-- ANDEAN
    --PERU
        --LIMA
           -- 'd0c82479-ad1c-4a7e-aaa5-d83ee49e9df7', -- LIM - Surquillo
            --'6d2d117b-3fa9-4a2c-85b5-9b86a758f849', -- LIM - Magdalena
    --ECUADOR
        -- QUITO
            --'d65136fe-862a-4da5-81ef-580c28f8c61a', -- UIO - 10 de Agosto
-- CCAC
    -- COLOMBIA
        -- BOGOTA
            '567933f5-72a7-4b7a-b6b0-9e1ff87f5bcf', -- BOG - Prado
          --'199c5433-d863-4d5e-bb6e-a53d91e0605f', -- Modelia  BOGOTA
            '4d78d1e1-5bce-4307-8175-51c816120fce', -- BOG - Castellana
            'b2f57126-8e64-4b11-964f-c497a3b53571', -- BOG - Cocinas de la 73
            'a5852445-e223-46c9-be99-ba33f204bcd1', -- BOG - Puente Aranda
            '76a6ef30-ddb2-4178-8c37-86c0d10db4b3', -- BOG - Atabanza
            'dd16bf0e-8a89-4f21-bc02-1c9793227502', -- BOG - San Felipe
            '3a6e7382-f2c0-4ebc-8f12-61f4d44dee82', -- BOG - Cedritos
            '757182ff-7a54-4571-91d4-0027253c0117', --BOG salitre
            --medellin
            '0016d9fc-4f27-4a05-88cb-90cd1cdf311f',--de moda
        -- CALI    
            '46fb5e19-3a79-4eda-b119-dee3a331fd81', -- CLO - Chipichape
        -- BARRANQUILLA
            '58b818c6-615e-4316-b4a3-b9a6bb914c30', -- BAQ - Alto Prado
    -- COSTA RICA
        -- SAN JOSE
            --'9eede066-b7e7-43c4-907c-21ece1688b14', -- SJO - Guadalupe
           -- '967f0a2e-9c8e-4b35-929b-37420d9439c9', -- SJO - Pavas
-- MEXICO
        -- MEXICO CITY
            '47e2d054-8b82-420c-b1b0-18010f57489c', -- MEX - Insurgentes Sur
            '565273fe-0074-421d-b1e1-796d1e4390e1', -- MEX - Napoles
            '864e0a3c-ecba-4305-b3ff-f22edd58c902', -- MEX - Polanco
            'bcb04949-3cda-4ca0-be39-b9f579e356ca', -- MEX - Roma
            '9101d4f9-828f-45c5-b8b6-16986e76b374', -- MEX - Pedregal
            'b9545910-bca6-4b2a-a603-363721e7a23d', -- MEX - Santa Fe
            'da27f06f-01fd-4106-88d9-b3fd41446515', -- MEX - Lindavista
        -- MONTERREY
            '394c0103-1c5f-424b-b398-68455fced5bb', -- MTY - Cumbres
            '6d2d6c66-47aa-4f6e-ad75-58f68cc36c5f', -- MTY - Leones NK
            '988c22e5-20c2-42a5-bc30-989c68ff5eee', -- MTY - Tec
            '1b90b4ae-6d8f-4a73-bb9e-708c83387430', -- MTY - Santa Maria
            '88774bba-2e34-4771-8e64-8f059adf02a3', -- MTY - Valle Oriente
        -- GUADALAJARA
            'a9a6c4be-32b7-47cf-b783-7a5227a89ed4', -- GDL - Ciudad del Sol
            'c66ce711-5486-433f-a9b7-1b36ae3a6dee', -- GDL - av mexico
            
-- CHILE
        -- SANTIAGO
            --'db3a588c-59ac-458e-84de-9e1c8ef5e3e8', -- SCL - Recoleta

        
-- BRAZIL
        -- BELO HORIZONTE
            '7b5c774f-7739-4d8e-a392-d49547d45fc3', -- BHZ - Contagem Industrial
            '554263c6-bde4-4d4c-92a7-21ed77e122cd', -- BHZ - Paracatu
            '522c745a-fa86-42de-83ca-227cbd3cb353', -- BHZ - Arthur Guimaraes
            '4518d99c-fa47-4da7-b5f5-c071c78b5ac2', -- BHZ - Tancredo Neves
        -- RIO DE JANEIRO
            '81e6538d-c45b-4479-92e8-fc4e04373e12', -- RIO - Jose Bonifacio
            '466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb', -- RIO - Barra
            '0ee5dccf-ed5b-465c-9858-b53351353e3b', -- RIO - Camerino
             '976ae838-18e0-40bb-a8dc-360353c01b6d', -- RIO - Niteroi
            'f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6', -- RIO - Tijuca
            '3759ad21-4537-48ea-b1f1-48809e8a03ac', --rio botafogo
        -- SAO PAULO
            '6c601837-403c-425f-a51c-31428ed1a67e', -- SAO - Rua do Acre
            '87a40094-cf48-4032-8166-e4cee61bce2f', -- SAO - Dino
            '4326199f-628a-4b00-a6e6-054fb5182c64', -- SAO - Guararapes (Brooklin)
            'dcc461ae-c80c-46cc-a5dc-4a5a72761510', -- SAO - Lapa Clelia
            '7aae3115-311b-449f-88f4-0f1612ecfcee', -- SAO - Borba
            '1840180b-9a3b-4fb6-b35c-ef4e6785b26e', -- SAO - Abelardo
            '88aaaf99-a444-4e0f-99f6-267b635faa53',  -- SAO - Vargas
             'a63559d1-93c6-4f60-b683-1e207ccda10a', -- SAO - Batista
            --'89cf86b9-9e74-4657-86df-2a851492af88', -- SAO - Guarulhos
            '89cf86b9-9e74-4657-86df-2a851492af88', --sao espanhola
            '14d9bb92-6376-4d69-adb6-64f97e4b3561',-- SAO - Ferreira
            '812e6913-d78e-4dbc-877d-16993853aa7e', --sao domingos
            
                
        -- BRASILIA
            'c559b06b-e193-495c-85cd-a5d1608ca27d'  -- BSB - Aguas Claras
                
                )


)


Select created_week,country, ordenes.organization_id, ordenes.facility_id, ordenes.facility_sf_name


,CASE
	WHEN ordenes.organization_id IN ('inventory_owned') THEN True 
	ELSE False
	END AS inventory_owned
,CASE 
	WHEN ordenes.organization_id IN ('xxxxxxxx') THEN '1PCR' 
	WHEN ordenes.organization_id IN ('xxxxxxxxxxx') THEN '2PCR'
	ELSE '3PCR' 
	END AS distribution_type
,city
,COUNT (DISTINCT (order_id)) as total_orders
,COUNT(DISTINCT CASE WHEN is_cancelled THEN order_id END) as cancelled_orders
,COUNT (DISTINCT CASE WHEN defect_type = 'issue' THEN order_id END) as issue_orders
,COUNT (DISTINCT CASE WHEN defect_type != 'success' THEN order_id END) as ODR_orders
,COUNT(DISTINCT  CASE WHEN order_scanned_out THEN order_id END) as scanned_orders
,SUM(subtotal) as total_gmv_local
,SUM(CASE WHEN is_cancelled THEN coalesce(subtotal, 0) ELSE 0 END) AS cancelled_gmv_local
,COUNT (DISTINCT ofo_slug) as OFOs
,COUNT(DISTINCT city) as cities
,COUNT(DISTINCT facility_sf_name) as facilities
,AVG (order_items_quantity) as Avg_Items_per_Order
,AVG(order_rating) as avg_rating
from ordenes

left join hudi_ingest.processed.store_config_organization_view as org

on ordenes.organization_id=org.organization_id

WHERE  row_number=1

and ordenes.organization_id not in (

'45ad476b-9874-4513-8a29-1b3b04180fbd', --casa de pastor
'2f3aefc9-2f28-45df-8de5-2b52530abef1', --bento vk
'cb671780-a757-4db5-8782-100106cad446', --delichurros vk
'842a7a9f-e25c-433d-bb4f-c1912db1bb04', --guisamor
'8d09b777-805e-49cf-a172-27fbf2840dd1',--donas americanas
'46f35c4f-1bf1-479d-92c9-e729a40ba816',-- FREE-TANGAS & GAR-NACHAS - VK
'66279914-31e6-4df9-9c57-36153545b28a',-- iRobot


--tests:

'beb89975-2feb-4d0b-a61e-ac2a3b87bd18', --Org Test PiknPak
'24622819-678c-45fb-a5be-acece3bf9c73', -- CR Brazil - Teste Superstore
'b2d4fbf2-5ddc-46ad-8207-36b7bf77c88a', -- Conta Teste - Catherine
'fae923c9-8a1d-468e-9740-95c36c0fd4fc', --Conta Teste Pick n pack - Comercial
'804c6336-abc2-4293-81ee-0cea88c15029', -- warehouse and ops analyst
'3046f0c0-ee65-4bb6-8cde-1dab0e459ee2' -- Cloud Distribuidora Pilots


)

and ordenes.brand_id not in (

'44d3d289-faff-4d79-b923-1fccefd84239' -- Unilever pilots
)

group by 1,2,3,4,5,6,7,8


  
"""

print("Querying data...")
orders_data_4 = ds_trino.fetch_data(conn=conn, sql_str= orders_query_2_latam, use_cache=False)
print("Data pulled!")
orders_data_4 = orders_data_4.replace(np.nan,'',regex=False)
print("replacing nan values in df")
orders_data_4.head()

Querying data...
Data pulled!
replacing nan values in df


,created_week,country,organization_id,facility_id,facility_sf_name,inventory_owned,distribution_type,city,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,total_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,Avg_Items_per_Order,avg_rating
0,2023-08-07,Brazil,893bb087-45ee-4436-9463-d8d3667b0d43,3759ad21-4537-48ea-b1f1-48809e8a03ac,RIO - Botafogo,False,3PCR,Rio de Janeiro,85,5,0,5,79,4576.19,242.90,2,1,1,2.152941,5.0
1,2023-07-17,Brazil,67f3637f-fff2-4ac7-9ad7-78d976319994,a63559d1-93c6-4f60-b683-1e207ccda10a,SAO - Batista,False,3PCR,Sao Paulo,32,0,0,0,30,698.40,0.00,2,1,1,1.062500,5.0
2,2023-06-26,Mexico,b556b267-8821-4162-a0cc-254fc70dafa0,9101d4f9-828f-45c5-b8b6-16986e76b374,MEX - Pedregal,False,3PCR,Mexico City,242,8,4,12,239,72497.79,3107.55,1,1,1,1.785124,4.904762
3,2023-07-10,Brazil,75a23483-f8f3-471b-85f9-6467f68ce8a0,f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6,RIO - Tijuca,False,3PCR,Rio de Janeiro,34,0,0,0,34,1396.20,0.00,1,1,1,3.235294,
4,2023-06-12,Brazil,a5b0cfd3-e887-4c62-b196-2507a8fb5cda,f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6,RIO - Tijuca,False,3PCR,Rio de Janeiro,60,0,0,0,60,2658.40,0.00,1,1,1,2.400000,4.2


In [7]:
# Merge to opp data
print(len(orders_data_4))
orders_data_4 = orders_data_4.merge(client_data, how = "left", on = "organization_id")
print(len(orders_data_4))

8183
8183


In [8]:
orders_data_4["seller_organization_id"] = orders_data_4["organization_id"]
orders_data_4["seller_organization_name"] = orders_data_4["name"]
orders_data_4["lost_gmv"] = 0


In [9]:
# Extract colnames / rearrange 
orders_data_4.columns

Index(['created_week', 'country', 'organization_id', 'facility_id',
       'facility_sf_name', 'inventory_owned', 'distribution_type', 'city',
       'total_orders', 'cancelled_orders', 'issue_orders', 'ODR_orders',
       'scanned_orders', 'total_gmv_local', 'cancelled_gmv_local', 'OFOs',
       'cities', 'facilities', 'Avg_Items_per_Order', 'avg_rating', 'name',
       'seller_organization_id', 'seller_organization_name', 'lost_gmv'],
      dtype='object')

In [10]:
orders_data_4 = orders_data_4[["created_week",
                               "country",
                               "organization_id",
                               "name",
                               "seller_organization_id",
                               "seller_organization_name",
                               "facility_id",
                               "facility_sf_name",
                               "inventory_owned",
                               "distribution_type",
                               "city",
                               "total_orders",
                               "cancelled_orders",
                               "issue_orders",
                               "ODR_orders",
                               "scanned_orders",
                               "Avg_Items_per_Order",
                               "total_gmv_local",
                               "lost_gmv",
                               "cancelled_gmv_local",
                               "OFOs",
                               "cities", 
                               "facilities"]]

In [11]:
# Connect to google sheets exchange rate file
import pygsheets
from google.oauth2 import service_account
from ds.utilities.secrets import secrets

In [12]:
s = secrets.Secrets()
SCOPES = ('https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/cloud-platform')

my_credentials = service_account.Credentials.from_service_account_info(
    s.get_secret("cloud_retail_sp_service_account"), scopes=SCOPES,
)

## Beam + SF integration

In [13]:
us_doors_query = """

WITH

  --ADD COUNTRY FILTER GOOGLE SHEET
  cr_countries (facility_country_code, country, region, currency_code) AS ( VALUES -- Filters which countries are pulled through, WILL CHANGE TO GOOGLE SHEET
    ('US', 'USA', 'US/CAN', 'USD')
    ,('CA', 'Canada', 'US/CAN', 'CAD')
   -- ,('CO', 'Colombia', 'SSL', 'COP')
    --,('MX', 'Mexico', 'SSL', 'MXN')
    --,('BR', 'Brazil', 'Brazil', 'BRL')
  )
  
  
  ,beam_proc_fees AS (
    SELECT
      licensee_id AS brand_organization_id
      ,is_latest_version 
      ,contract_signed_date AS beam_contract_signed_date
      ,billing_subscription_start_date_inclusive
      ,location_id 
      ,charge_spec_name
      ,static_fee_unit_price as processing_fee_percentage
      ,concat(licensee_id,location_id) as doors_id
      ,day_partition 
      FROM
        hudi_ingest.analytics_views_sensitive.licensee_billing_cr_lease_agreements 
      WHERE
        charge_spec_name = 'Order Processing Fees'
        and is_latest_version = True
        and day_partition IS NOT NULL
  )
  
  ,fx AS (
    SELECT
      region
      ,facility_country_code
      ,country
      ,ROUND(1 / rate, 5) AS fx_to_usd
    FROM
      hudi_ingest.analytics_views.monthly_usd_exchange_rates
    JOIN
      cr_countries
      USING(currency_code)
    WHERE
      day_partition = DATE_FORMAT(DATE_TRUNC('month', current_date - interval '1' month), '%Y-%m-%d')
  )
  
  ,ss_orgs AS (
    SELECT
      *
    FROM
      hudi_ingest.scratch.cr_starts
  )
  
  ,ss_facilities AS (
    SELECT
      facility_id
      ,MIN(org_fac_live_date) AS fac_live_date
    FROM
      ss_orgs
    GROUP BY
      1
  )
  
  ,sfdc_roles AS (
    SELECT
      id AS userroleid
      ,CASE
        WHEN name LIKE '%Account Executive%' THEN 'AE'
        WHEN name LIKE '%Sales Development Representative%' THEN 'SDR'
        ELSE 'OTHER'
      END AS role_name
    FROM
      hudi_ingest.salesforce_cloudkitchens.userrole
  )
  
  ,sfdc_users AS (
    SELECT
      id AS ownerid
      ,name AS user_name
      ,COALESCE(NULLIF(country, 'United States'), 'USA') AS owner_country
      ,CASE
        WHEN country = 'Brazil' THEN 'Brazil'
        WHEN region__c = 'LATAM' THEN 'SSL'
        ELSE region__c
      END AS owner_region
      ,role_name
    FROM
      hudi_ingest.salesforce_cloudkitchens.user
    JOIN
      sfdc_roles
      USING(userroleid)
  )
  
  ,sfdc_record_types AS (
    SELECT
      id AS recordtypeid
      ,name AS record_type_name
    FROM
      hudi_ingest.salesforce_cloudkitchens.recordtype
    GROUP BY
      1,2
  )
  
  ,sfdc_accounts AS (
    SELECT
      id AS account_id
      ,name AS account_name
      ,org_id_otter__c AS sfdc_ss_prep_organization_id
    FROM
      hudi_ingest.salesforce_cloudkitchens.account
  )
  
  ,sfdc_facilities AS (
    SELECT
      id AS sfdc_facility_id
      ,skusku_facility_id__c AS sfdc_ss_facility_id
    FROM
      hudi_ingest.salesforce_cloudkitchens.account
  )

  ,sfdc_leads AS (
    SELECT
      *
    FROM (
      SELECT
        lead_id
        ,lead_name
        ,lead_owner_id
        ,lead_owner_name
        ,lead_owner_role
        ,COALESCE(region, lead_region) AS lead_region
        ,country AS lead_country
        ,lead_type
        ,lead_assigned_date
        ,is_called
        ,IF(lead_status IN ('Disqualified', 'Qualified'), GREATEST(lead_processed_date, lead_assigned_date)) AS lead_processed_date
        ,lead_status
        ,IF(lead_processed_date IS NULL, GREATEST(days_since_last_activity, 0)) AS days_since_last_lead_activity
        ,account_id
      FROM (
        SELECT
          id AS lead_id
          ,name AS lead_name
          ,ownerid AS lead_owner_id
          ,user_name AS lead_owner_name
          ,role_name AS lead_owner_role
          ,owner_region AS lead_region
          ,COALESCE(NULLIF(COALESCE(kitchen_country__c, owner_country), 'United States'), 'USA') AS country
          ,CASE
            WHEN refreshed_lead__c THEN 'Refreshed'
            WHEN leadsource LIKE '%Inbound%' THEN 'Inbound'
            WHEN leadsource LIKE '%Scrape%' THEN 'List-Outbound'
            ELSE 'Self-Outbound'
          END AS lead_type
          ,DATE(COALESCE(refreshed_date__c, createddate)) AS lead_assigned_date
          ,COALESCE(first_call_date_time__c, converteddate) IS NOT NULL AS is_called
          ,DATE(COALESCE(converteddate, CASE
            WHEN NOT COALESCE(refreshed_lead__c, FALSE) THEN lead_unqualified_date__c
            ELSE IF(lead_unqualified_date__c >= refreshed_date__c, lead_unqualified_date__c)
          END)) AS lead_processed_date
          ,IF(status LIKE '%Not Converted%', 'Disqualified', status) AS lead_status
          ,days_since_last_activity__c AS days_since_last_activity
          ,isconverted
          ,convertedaccountid AS account_id
        FROM
          hudi_ingest.salesforce_cloudkitchens.lead
        JOIN
          sfdc_users
          USING(ownerid)
        JOIN
          sfdc_record_types
          USING(recordtypeid)
        WHERE
          record_type_name = 'CloudRetail' OR kitchen_type__c IN ('Virtual', 'CloudRetail')
      )
      LEFT JOIN
        cr_countries
        USING(country)
    )
    WHERE
      NOT COALESCE(lead_status = 'Disqualified' AND lead_processed_date < current_date - interval '4' month, FALSE)
  )
  
  --SELECT * FROM sfdc_leads

  ,sfdc_opps AS (
    SELECT
      account_id
      ,opportunity_id
      ,opp_name
      ,opp_owner_name
      ,opp_owner_role
      ,region
      ,country
      ,sfdc_facility_id
      ,CASE
        WHEN win_date_min IS NULL THEN 'New'
        WHEN deal_status = 'Deal Won' AND deal_processed_date = win_date_min THEN 'New'
        WHEN deal_status = 'Deal Lost' AND deal_processed_date < win_date_min THEN 'New'
        WHEN qual_lead_status = 'Qual Lead Lost' AND qual_lead_processed_date < win_date_min THEN 'New'
        ELSE 'Expansion'
      END AS deal_type
      ,deal_created_date
      ,is_meeting_scheduled
      ,qual_lead_processed_date
      ,qual_lead_status
      ,deal_processed_date
      ,deal_status
      ,deal_stage
      ,IF(deal_stage <> 'Won' AND deal_stage <> 'Lost', days_since_last_activity) AS days_since_last_deal_activity
      ,is_prepaid
      ,sfdc_start_date
      ,sfdc_billable_date
      ,sfdc_contract_months
      ,sfdc_churn_date
      ,sfdc_activation_fee
      ,sfdc_gross_mrr * COALESCE(1 - discount_months / sfdc_contract_months, 1) AS sfdc_gross_mrr
      ,sfdc_gross_mrr * COALESCE(sfdc_contract_months - discount_months, 1) + sfdc_activation_fee AS sfdc_tcv 
      ,sfdc_storage_cu_ft
    FROM (  
      SELECT  
        account_id
        ,opportunity_id
        ,opp_name
        ,opp_owner_name
        ,opp_owner_role
        ,region
        ,country
        ,sfdc_facility_id
        ,deal_created_date
        ,is_meeting_scheduled
        ,COALESCE(deal_opened_date, deal_closed_lost_date) AS qual_lead_processed_date
        ,CASE 
          WHEN is_opened THEN 'Deal Opened'
          WHEN deal_closed_lost_date IS NOT NULL THEN 'Qual Lead Lost'
          ELSE 'Qual Lead Open'
        END AS qual_lead_status
        ,IF(is_opened, COALESCE(deal_closed_won_date, deal_closed_lost_date)) AS deal_processed_date
        ,CASE
          WHEN is_won THEN 'Deal Won'
          WHEN is_opened AND deal_closed_lost_date IS NOT NULL THEN 'Deal Lost'
          WHEN is_opened AND deal_closed_lost_date IS NULL THEN 'Deal Open'
        END AS deal_status
        ,CASE
          WHEN is_won THEN 'Won'
          WHEN is_opened AND deal_closed_lost_date IS NOT NULL THEN 'Lost'
          ELSE stage_name
        END AS deal_stage
        ,days_since_last_activity
        ,MIN(IF(is_won, deal_closed_won_date)) OVER (PARTITION BY account_id) AS win_date_min
        ,is_prepaid
        ,sfdc_start_date
        ,sfdc_billable_date
        ,sfdc_contract_months
        ,sfdc_churn_date
        ,sfdc_activation_fee
        ,sfdc_gross_mrr
        ,sfdc_storage_cu_ft
        ,discount_months
      FROM (
        SELECT
          account_id
          ,opportunity_id
          ,opp_name
          ,opp_owner_name
          ,opp_owner_role
          ,region
          ,country
          ,sfdc_facility_id
          ,LEAST(deal_created_date, COALESCE(deal_opened_date, deal_closed_won_date, deal_closed_lost_date, deal_created_date)) AS deal_created_date
          ,is_meeting_scheduled
          ,deal_opened_date
          ,deal_opened_date IS NOT NULL AS is_opened
          ,deal_closed_won_date
          ,deal_closed_lost_date
          ,is_won
          ,is_prepaid
          ,sfdc_start_date
          ,sfdc_billable_date
          ,sfdc_contract_months
          ,sfdc_churn_date
          ,sfdc_activation_fee
          ,sfdc_gross_mrr
          ,sfdc_storage_cu_ft
          ,stage_name
          ,days_since_last_activity
          ,discount_months
        FROM (
          SELECT
            account_id
            ,opportunity_id
            ,opp_name
            ,opp_owner_name
            ,opp_owner_role
            ,region
            ,country
            ,sfdc_facility_id
            ,deal_created_date
            ,is_won OR deal_opened_date IS NOT NULL OR is_meeting_scheduled AS is_meeting_scheduled
            ,CASE
              WHEN is_won THEN LEAST(COALESCE(deal_opened_date, deal_closed_won_date), deal_closed_won_date)
              WHEN deal_closed_lost_date IS NOT NULL AND deal_opened_date IS NOT NULL THEN LEAST(deal_opened_date, deal_closed_lost_date)
              ELSE deal_opened_date
            END AS deal_opened_date
            ,deal_closed_won_date
            ,deal_closed_lost_date
            ,is_won
            ,COALESCE(IF(is_won, contract_type LIKE '%Pre-%'), FALSE) AS is_prepaid
            ,sfdc_start_date
            ,sfdc_billable_date
            ,sfdc_contract_months
            ,sfdc_churn_date
            ,sfdc_activation_fee
            ,sfdc_gross_mrr
            ,sfdc_storage_cu_ft
            ,stage_name
            ,days_since_last_activity
            ,discount_months
          FROM (
            SELECT
              account_id
              ,opportunity_id
              ,opp_name
              ,opp_owner_name
              ,opp_owner_role
              ,COALESCE(region, owner_region) AS region
              ,country
              ,sfdc_facility_id
              ,opp_closed_won_date AS deal_closed_won_date
              ,opp_closed_lost_date AS deal_closed_lost_date
              ,is_won
              ,MIN(opp_created_date) OVER (PARTITION BY account_id, opp_closed_won_date, opp_closed_lost_date) AS deal_created_date
              ,SUM(is_meeting_scheduled) OVER (PARTITION BY account_id, opp_closed_won_date, opp_closed_lost_date) > 0 AS is_meeting_scheduled
              ,MIN(opp_opened_date) OVER (PARTITION BY account_id, opp_closed_won_date, opp_closed_lost_date) AS deal_opened_date
              ,IF(sfdc_start_date <= current_date, sfdc_start_date) AS sfdc_start_date
              ,IF(sfdc_billable_date <= current_date, sfdc_billable_date) AS sfdc_billable_date
              ,IF(is_won, LEAST(GREATEST(COALESCE(sfdc_contract_months, 0), 3), 24)) AS sfdc_contract_months
              ,IF(sfdc_churn_date <= current_date, sfdc_churn_date) AS sfdc_churn_date
              ,IF(is_won, ROUND(COALESCE(sfdc_activation_fee, 0.00) * COALESCE(fx_to_usd, 1.00000), 2)) AS sfdc_activation_fee
              ,IF(is_won, ROUND(COALESCE(sfdc_gross_mrr, 0.00) * COALESCE(fx_to_usd, 1.00000), 2)) AS sfdc_gross_mrr
              ,IF(is_won, sfdc_storage_cu_ft) AS sfdc_storage_cu_ft
              ,stage_name
              ,days_since_last_activity
              ,discount_months
              ,contract_type
            FROM (
              SELECT
                accountid AS account_id
                ,id AS opportunity_id
                ,facility_name__c AS opp_name
                ,opportunity_owner_name__c AS opp_owner_name
                ,CASE
                  WHEN opp_owner_role__c LIKE '%Account Executive%' THEN 'AE'
                  WHEN opp_owner_role__c LIKE '%Sales Development Representative%' THEN 'SDR'
                  ELSE 'OTHER'
                END AS opp_owner_role
                ,CASE
                  WHEN COALESCE(kitchen_country__c, opp_owner_country__c) = 'Brazil' THEN 'Brazil'
                  WHEN opp_owner_region__c = 'LATAM' THEN 'SSL'
                  ELSE opp_owner_region__c
                END AS owner_region
                ,COALESCE(NULLIF(COALESCE(kitchen_country__c, opp_owner_country__c), 'United States'), 'USA') AS country
                ,facility__c AS sfdc_facility_id
                ,DATE(createddate) AS opp_created_date
                ,IF(COALESCE(scheduled_virtual_tour_date__c, scheduled_in_person_tour_date__c, sql_date__c, completed_virtual_tour_date__c, facility_tour_date__c) IS NOT NULL, 1, 0) AS is_meeting_scheduled
                ,DATE(COALESCE(sql_date__c, completed_virtual_tour_date__c, facility_tour_date__c)) AS opp_opened_date
                ,closed_won_date__c AS opp_closed_won_date
                ,IF(closed_won_date__c IS NOT NULL, NULL, closed_lost_date__c) AS opp_closed_lost_date
                ,iswon AS is_won
                ,IF(iswon, COALESCE(access_date__c, start_date__c, closed_won_date__c + interval '1' month)) AS sfdc_start_date
                ,IF(iswon, COALESCE(activation_date__c, closed_won_date__c + interval '1' month)) AS sfdc_billable_date
                ,IF(iswon, COALESCE(renewal_term_length_months__c, ROUND(DATE_DIFF('day', start_date__c, end_date__c) / (365 / 12), 0))) AS sfdc_contract_months
                ,IF(iswon, COALESCE(actual_churn_date__c, churn_date__c)) AS sfdc_churn_date
                ,COALESCE(onboarding_fee__c, 0.00) AS sfdc_activation_fee
                ,COALESCE(monthly_shelf_fee_dry_storage__c, 0.00) + COALESCE(monthly_shelf_fee_cold_storage__c, 0.00) + COALESCE(monthly_shelf_fee_frozen_storage__c, 0.00) AS sfdc_gross_mrr
                ,COALESCE(storage_amount_dry_cu_ft__c, 0) + COALESCE(storage_amount_cold_cu_ft__c, 0) + COALESCE(storage_amount_frozen_cu_ft__c, 0) AS sfdc_storage_cu_ft
                ,stagename AS stage_name
                ,COALESCE(initial_discount_length__c, 0) AS discount_months
                ,days_since_last_activity__c AS days_since_last_activity
                ,type AS contract_type
              FROM
                hudi_ingest.salesforce_cloudkitchens.opportunity
              JOIN
                sfdc_record_types
                USING(recordtypeid)
              WHERE
                record_type_name = 'CloudRetail' OR kitchen_type__c IN ('Virtual', 'CloudRetail')
            )
            LEFT JOIN
              fx
              USING(country)
          )
          JOIN (
            SELECT
              region
            FROM
              cr_countries
            GROUP BY
              1
          )  
            USING(region)
        )
      )
    )
  )
  
  --SELECT * FROM sfdc_opps
  
  ,beam AS (
    SELECT
      IF(COUNT(*) OVER (PARTITION BY opportunity_id) > 1, 'duplicate_' || opportunity_id, opportunity_id) AS opportunity_id
      ,beam_region
      ,beam_country
      ,prep_organization_id
      ,facility_id
      ,lease_agreement_id
      ,beam_closed_won_date
      ,beam_billable_date
      ,beam_churn_date
      ,beam_is_prepaid
      ,beam_ob_fee
      ,beam_mrr
      ,IF(beam_is_prepaid, beam_ob_fee + beam_tcv_raw) AS beam_tcv
      ,beam_cu_ft
    FROM (
      SELECT
        opportunity_id
        ,'US/CAN' AS beam_region --will need to update when we go multi-region
        ,'USA' AS beam_country
        ,prep_organization_id
        ,facility_id
        ,lease_agreement_id
        ,beam_closed_won_date
        ,beam_billable_date
        ,beam_churn_date
        
        ,COUNT(*) FILTER (WHERE is_prepaid) > 0 AS beam_is_prepaid
        ,COALESCE(SUM(amount) FILTER (WHERE NOT is_storage), 0) AS beam_ob_fee
        ,SUM(amount / mrr_calc_months) FILTER (WHERE is_storage AND amount > 0) AS beam_mrr
        ,SUM(IF(is_prepaid, amount)) FILTER (WHERE is_storage AND amount > 0) AS beam_tcv_raw
        ,SUM(beam_cu_ft) FILTER (WHERE is_storage AND amount > 0) AS beam_cu_ft
      FROM (
        SELECT
          licensee_id AS prep_organization_id
          ,location_id AS facility_id
          ,lease_agreement_id 
          ,charge_spec_cadence = 'ONE_TIME' AND charge_spec_category = 'Storage' AS is_prepaid
          ,charge_spec_name <> 'Onboarding Fees' AS is_storage
          ,CASE
            WHEN charge_spec_name LIKE '%6 Months Prepaid/6 Months Waived%' THEN 12
            WHEN charge_spec_name LIKE '%1st Year Prepaid/6 Months Waived%' THEN 18
            WHEN charge_spec_name LIKE '%1st Year Prepaid/2nd Year Waived%' THEN 24
            WHEN charge_spec_name LIKE '%1 Year Prepaid%' THEN 12
            WHEN charge_spec_name LIKE '%6 Months Prepaid%' THEN 6
            WHEN charge_spec_name LIKE '%3 Months Prepaid%' THEN 3
            ELSE 1
          END AS mrr_calc_months
          ,contract_signed_date AS beam_closed_won_date
          ,actual_license_fee_start_date_inclusive AS beam_billable_date
          ,termination_date AS beam_churn_date
          ,GREATEST(static_quantity * static_fee_unit_price + static_quantity * COALESCE(static_amount, 0.00), 0.00) AS amount
          ,static_quantity AS beam_cu_ft
          ,COALESCE(FIRST_VALUE(meta_value) IGNORE NULLS OVER (PARTITION BY lease_agreement_version_id), 'missing_' || licensee_id || '_' || location_id) AS opportunity_id
        FROM
          hudi_ingest.analytics_views_sensitive.licensee_billing_cr_lease_agreements
        WHERE
          day_partition IS NOT NULL
          and charge_spec_custom_effective_date_start_inclusive is null -- trying to fix mrr and not bring in the renewal mrr
          AND is_latest_version
          AND (charge_spec_name = 'Onboarding Fees' OR charge_spec_category = 'Storage')
          AND licensee_id <> '2d73f5b2-c99e-4f84-b309-3ff7e5b1f0ca' --Parker's test org, RIP Parker aka beam king aka the singapore savage
          AND licensee_id <> 'c9f8f23c-6526-4df7-bee9-22415b0d9727' --Parker's test org, RIP Parker aka beam king aka the singapore savage
          AND (termination_note IS NULL OR termination_note NOT LIKE '%Duplicate%')
      )
      GROUP BY
        1,2,3,4,5,6,7,8,9
    )
  )

  ,sfdc_beam_opps AS (
    SELECT
      pulled_date_pst
      ,lead_id
      ,account_name
      ,lead_owner_name
      ,lead_owner_role
      ,region
      ,country
      ,lead_type
      ,DATE(lead_assigned_date) AS lead_assigned_date
      ,is_called
      ,DATE(lead_processed_date) AS lead_processed_date
      ,lead_to_proc_days
      ,lead_status
      ,days_since_last_lead_activity
      ,account_id
      ,opportunity_id
      ,opp_name AS sfdc_fac_name
      ,opp_owner_name
      ,opp_owner_role
      ,deal_type
      ,deal_created_date
      ,is_meeting_scheduled
      ,qual_lead_processed_date
      ,qual_lead_to_proc_days
      ,qual_lead_status
      ,deal_processed_date
      ,open_deal_to_closed_days
      ,deal_status
      ,deal_stage
      ,days_since_last_deal_activity
      ,is_prepaid
      ,IF(deal_status = 'Deal Won' OR beam_sync_qa = 'SFDC Missing', activated_date IS NOT NULL) AS is_activated
      ,activated_date
      ,IF(deal_status = 'Deal Won' OR beam_sync_qa = 'SFDC Missing', billable_date IS NOT NULL) AS is_billable
      ,DATE(billable_date) AS billable_date
      ,sfdc_contract_months
      ,IF(deal_status = 'Deal Won' OR beam_sync_qa = 'SFDC Missing', churn_date IS NOT NULL) AS is_churned
      ,DATE(churn_date) AS churn_date
      ,activation_fee
      ,mrr
      ,tcv
      ,storage_cu_ft
      ,prep_organization_id
      ,facility_id
      ,beam_sync_qa
      ,org_id_qa
      ,fac_id_qa
      
      --,concat(prep_organization_id,facility_id) AS doors_id 
      
    FROM (
      SELECT
        pulled_date_pst
        ,lead_id
        ,account_name
        ,lead_owner_name
        ,lead_owner_role
        ,region
        ,country
        ,lead_type
        ,lead_assigned_date
        ,is_called
        ,lead_processed_date
        ,DATE_DIFF('day',lead_assigned_date, lead_processed_date) AS lead_to_proc_days
        ,lead_status
        ,days_since_last_lead_activity
        ,account_id
        ,opportunity_id
        ,opp_name
        ,opp_owner_name
        ,opp_owner_role
        ,deal_type
        ,DATE(deal_created_date) AS deal_created_date
        ,is_meeting_scheduled
        ,DATE(qual_lead_processed_date) AS qual_lead_processed_date
        ,DATE_DIFF('day', deal_created_date, qual_lead_processed_date) AS qual_lead_to_proc_days
        ,qual_lead_status
        ,DATE(deal_processed_date) AS deal_processed_date
        ,IF(deal_status = 'Deal Won', DATE_DIFF('day', qual_lead_processed_date, deal_processed_date)) AS open_deal_to_closed_days
        ,deal_status
        ,deal_stage
        ,days_since_last_deal_activity
        ,is_prepaid
        ,DATE(org_fac_live_date) AS activated_date
        ,IF(has_beam OR deal_status = 'Deal Won',
          CASE
            WHEN facility_id IS NULL THEN NULL
            WHEN deal_processed_date <= DATE('2021-10-01') AND prep_organization_id IS NULL THEN NULL
            WHEN deal_processed_date <= DATE('2021-10-01') THEN org_fac_live_date
            WHEN fac_live_date IS NULL THEN NULL
            ELSE GREATEST(billable_date, fac_live_date)
          END
        ) AS billable_date
        ,sfdc_contract_months
        ,churn_date
        ,activation_fee
        ,mrr
        ,tcv
        ,storage_cu_ft
        ,prep_organization_id
        ,facility_id
        ,CASE
          WHEN has_beam AND deal_status = 'Deal Won' THEN 'Synced'
          WHEN deal_status = 'Deal Won' THEN 'Beam Missing'
          WHEN has_beam AND account_id IS NULL THEN 'SFDC Missing'
        END AS beam_sync_qa
        ,IF(has_beam OR deal_status = 'Deal Won', prep_organization_id IS NOT NULL) AS org_id_qa
        ,IF(has_beam OR deal_status = 'Deal Won', facility_id IS NOT NULL) AS fac_id_qa
      FROM (
        SELECT
          AT_TIMEZONE(current_timestamp, 'America/Los_Angeles') AS pulled_date_pst
          ,COALESCE(lead_id, 'unknown_' || account_id) AS lead_id
          ,COALESCE(account_name, lead_name, 'unknown_' || account_id) AS account_name
          ,COALESCE(lead_owner_name, opp_owner_name) AS lead_owner_name
          ,COALESCE(lead_owner_role, opp_owner_role) AS lead_owner_role
          ,COALESCE(region, lead_region, beam_region) AS region
          ,COALESCE(country, lead_country, beam_country) AS country
          ,IF(lead_id IS NOT NULL OR account_id IS NOT NULL, COALESCE(lead_type, 'Self-Outbound')) AS lead_type
          ,COALESCE(lead_assigned_date, deal_created_date) AS lead_assigned_date
          ,COALESCE(is_called, IF(account_id IS NOT NULL, TRUE)) AS is_called
          ,COALESCE(lead_processed_date, deal_created_date) AS lead_processed_date
          ,IF(lead_id IS NOT NULL OR account_id IS NOT NULL, COALESCE(lead_status, 'Qualified')) AS lead_status
          ,days_since_last_lead_activity
          ,account_id
          ,opportunity_id
          ,opp_name
          ,opp_owner_name
          ,opp_owner_role
          ,deal_type
          ,deal_created_date
          ,is_meeting_scheduled
          ,qual_lead_processed_date
          ,qual_lead_status
          ,COALESCE(deal_processed_date, beam_closed_won_date) AS deal_processed_date
          ,deal_status
          ,deal_stage
          ,days_since_last_deal_activity
          ,COALESCE(beam_is_prepaid, is_prepaid) AS is_prepaid
          ,IF((deal_status = 'Deal Won' OR (account_id IS NULL AND lease_agreement_id IS NOT NULL)) AND COALESCE(beam_billable_date, sfdc_billable_date) <= current_date, COALESCE(beam_billable_date, sfdc_billable_date)) AS billable_date
          ,sfdc_contract_months
          ,IF((deal_status = 'Deal Won' OR (account_id IS NULL AND lease_agreement_id IS NOT NULL)) AND COALESCE(beam_churn_date, sfdc_churn_date) <= current_date, COALESCE(beam_churn_date, sfdc_churn_date)) AS churn_date
          ,COALESCE(beam_ob_fee, sfdc_activation_fee) AS activation_fee
          ,COALESCE(beam_mrr, sfdc_gross_mrr) AS mrr
          ,COALESCE(beam_tcv, beam_mrr * COALESCE(sfdc_contract_months, 3) + beam_ob_fee, sfdc_tcv) AS tcv
          ,COALESCE(beam_cu_ft, sfdc_storage_cu_ft) AS storage_cu_ft
          ,IF((deal_status = 'Deal Won' OR (account_id IS NULL AND lease_agreement_id IS NOT NULL)), COALESCE(prep_organization_id, sfdc_ss_prep_organization_id)) AS prep_organization_id
          ,IF((deal_status = 'Deal Won' OR (account_id IS NULL AND lease_agreement_id IS NOT NULL)), COALESCE(facility_id, sfdc_ss_facility_id)) AS facility_id
          ,lease_agreement_id IS NOT NULL AS has_beam
        FROM
          sfdc_leads
        FULL JOIN
          sfdc_opps
          USING(account_id)
        LEFT JOIN
          sfdc_accounts
          USING(account_id)
        LEFT JOIN
          sfdc_facilities
          USING(sfdc_facility_id)
        FULL JOIN
          beam
          USING(opportunity_id)
        ORDER BY
          COALESCE(lead_assigned_date, deal_created_date) DESC
      )
      JOIN
        cr_countries
        USING(country, region)
      LEFT JOIN
        ss_orgs
        USING(prep_organization_id, facility_id)
      LEFT JOIN
        ss_facilities
        USING(facility_id)
    )
  )
  
  ,doors AS (
    SELECT
      pulled_date_pst
      ,sfdc_account_name
      ,sfdc_account_id
      ,sfdc_facility_name
      ,prep_organization_id
      ,facility_id
      ,concat(prep_organization_id,facility_id) AS doors_id 
      ,region
      ,country
      ,fac_live_date
      ,is_fac_live
      ,is_expansion
      ,is_2p_l4w
      ,closed_won_date
      ,activated_date
      ,is_activated
      ,billable_date
      ,is_billable
      ,churn_date
      ,is_churned
      ,activation_fee
      ,mrr
      ,tcv
      ,storage_cu_ft
      ,beam_sync_qa
      ,org_id_qa
      ,fac_id_qa
      ,sgmv_l2w_1mo
      ,sgmv_2p_l2w_1mo
      ,ROUND(COALESCE((SUM(IF(is_billable AND NOT is_churned, sgmv_l2w_1mo, 0.00)) OVER (PARTITION BY prep_organization_id)) / (SUM(IF(is_billable AND NOT is_churned, mrr, 0.00)) OVER (PARTITION BY prep_organization_id)), 0.00), 2) AS gmv_mrr_acct
      ,processing_fee_percentage
    FROM (
      SELECT
        AT_TIMEZONE(current_timestamp, 'America/Los_Angeles') AS pulled_date_pst
        ,sfdc_account_name
        ,sfdc_account_id
        ,sfdc_facility_name
        ,prep_organization_id
        ,facility_id
        ,region
        ,country
        ,closed_won_date
        ,activated_date
        ,activated_date IS NOT NULL AS is_activated
        ,billable_date
        ,billable_date IS NOT NULL AS is_billable
        ,churn_date
        ,churn_date IS NOT NULL AS is_churned
        ,activation_fee
        ,mrr
        ,tcv
        ,storage_cu_ft
        ,beam_sync_qa
        ,NOT prep_organization_id LIKE '%missing%' AS org_id_qa
        ,NOT facility_id LIKE '%missing%' AS fac_id_qa
        
        ,MIN(activated_date) OVER (PARTITION BY facility_id) AS fac_live_date
        ,COUNT(activated_date) OVER (PARTITION BY facility_id) > 0 AS is_fac_live
        ,closed_won_date > MIN(closed_won_date) OVER (PARTITION BY prep_organization_id) AS is_expansion
        
        ,COALESCE(SUM(orders) FILTER (WHERE distribution_type = '2PCR' AND order_weeks_back_local BETWEEN 1 AND 4), 0) > 0 AS is_2p_l4w
        ,ROUND(COALESCE(SUM(gmv) FILTER (WHERE defect_type = 'success' AND order_weeks_back_local BETWEEN 1 AND 2), 0.00) * 2.17, 2)  AS sgmv_l2w_1mo
        ,ROUND(COALESCE(SUM(gmv) FILTER (WHERE distribution_type = '2PCR' AND defect_type = 'success' AND order_weeks_back_local BETWEEN 1 AND 2), 0.00) * 2.17, 2)  AS sgmv_2p_l2w_1mo
      FROM (
        SELECT
          COALESCE(account_name, 'unknown_' || prep_organization_id) AS sfdc_account_name
          ,COALESCE(account_id, 'missing_' || prep_organization_id) AS sfdc_account_id
          ,COALESCE(sfdc_fac_name, 'missing_' || facility_id) AS sfdc_facility_name
          ,COALESCE(prep_organization_id, 'missing_' || account_id) AS prep_organization_id
          ,COALESCE(facility_id, 'missing_' || sfdc_fac_name) AS facility_id
          ,region
          ,country
          
          ,MIN(deal_processed_date) AS closed_won_date
          ,MIN(activated_date) FILTER (WHERE is_activated) AS activated_date
          ,MIN(billable_date) FILTER (WHERE is_billable) AS billable_date
          ,IF(COUNT(*) = COUNT(*) FILTER (WHERE is_churned), MAX(churn_date)) AS churn_date
          ,SUM(activation_fee) AS activation_fee
          ,CASE
            WHEN COUNT(*) = COUNT(*) FILTER (WHERE is_churned) THEN SUM(mrr)
            ELSE SUM(mrr) FILTER (WHERE NOT is_churned)
          END AS mrr
          ,CASE
            WHEN COUNT(*) = COUNT(*) FILTER (WHERE is_churned) THEN SUM(tcv)
            ELSE SUM(tcv) FILTER (WHERE NOT is_churned)
          END AS tcv
          ,CASE
            WHEN COUNT(*) = COUNT(*) FILTER (WHERE is_churned) THEN SUM(storage_cu_ft)
            ELSE SUM(storage_cu_ft) FILTER (WHERE NOT is_churned)
          END AS storage_cu_ft
          ,COUNT(*) = COUNT(*) FILTER (WHERE beam_sync_qa = 'Synced') AS beam_sync_qa
        FROM
          sfdc_beam_opps as c
        WHERE
          (deal_status = 'Deal Won' AND region NOT IN (
            'US/CAN' --Beam enabled regions
          )) OR beam_sync_qa IN ('Synced', 'SFDC Missing')
        GROUP BY
          1,2,3,4,5,6,7
      ) as a
      LEFT JOIN
        hudi_ingest.scratch.cr_gmv  
        USING(prep_organization_id, facility_id, country, region)
      
      GROUP BY
        1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
    )left join beam_proc_fees as b on concat(prep_organization_id,facility_id) = b.doors_id
  )
    
SELECT
  *
FROM
  --sfdc_beam_opps --Use for D_SFDC
  doors --Use for D_Doors
  
"""

print("Querying data...")
doors_data = ds_trino.fetch_data(conn=conn, sql_str= us_doors_query, use_cache=False)
print("Data pulled!")
doors_data = doors_data.replace(np.nan,'',regex=False)
print("replacing nan values in df")
doors_data.head()

Querying data...
Data pulled!
replacing nan values in df


,pulled_date_pst,sfdc_account_name,sfdc_account_id,sfdc_facility_name,prep_organization_id,facility_id,doors_id,region,country,fac_live_date,is_fac_live,is_expansion,is_2p_l4w,closed_won_date,activated_date,is_activated,billable_date,is_billable,churn_date,is_churned,activation_fee,mrr,tcv,storage_cu_ft,beam_sync_qa,org_id_qa,fac_id_qa,sgmv_l2w_1mo,sgmv_2p_l2w_1mo,gmv_mrr_acct,processing_fee_percentage
0,2023-09-20 15:55:55.829 America/Los_Angeles,unknown_48c743af-1022-4e7c-a383-16100bd4da39,missing_48c743af-1022-4e7c-a383-16100bd4da39,missing_94e1cb0b-facf-4f72-b66d-2bd97d98e58f,48c743af-1022-4e7c-a383-16100bd4da39,94e1cb0b-facf-4f72-b66d-2bd97d98e58f,48c743af-1022-4e7c-a383-16100bd4da3994e1cb0b-f...,US/CAN,USA,2021-05-14,True,False,False,2022-07-08,,False,,False,2022-08-18,True,500.0,,,,False,True,True,0.00,0.0,NaN,0.15
1,2023-09-20 15:55:55.829 America/Los_Angeles,unknown_4d602cc1-f405-46e1-b13b-b1d861ede9f9,missing_4d602cc1-f405-46e1-b13b-b1d861ede9f9,missing_99573ac0-a3f2-4c70-946c-e9c06b425d5c,4d602cc1-f405-46e1-b13b-b1d861ede9f9,99573ac0-a3f2-4c70-946c-e9c06b425d5c,4d602cc1-f405-46e1-b13b-b1d861ede9f999573ac0-a...,US/CAN,USA,2021-01-11,True,False,False,2023-02-26,2023-04-11,True,,False,,False,500.0,,,,False,True,True,0.00,0.0,NaN,0.15
2,2023-09-20 15:55:55.829 America/Los_Angeles,unknown_a23123d4-990c-4d6b-bc76-e77f83c1607f,missing_a23123d4-990c-4d6b-bc76-e77f83c1607f,missing_305dca7a-ffcf-4ecf-9363-4238af3cba6b,a23123d4-990c-4d6b-bc76-e77f83c1607f,305dca7a-ffcf-4ecf-9363-4238af3cba6b,a23123d4-990c-4d6b-bc76-e77f83c1607f305dca7a-f...,US/CAN,USA,2021-04-02,True,False,False,2021-05-28,2021-07-21,True,2021-07-21,True,,False,250.0,,,,False,True,True,0.00,0.0,0.0,0.15
3,2023-09-20 15:55:55.829 America/Los_Angeles,unknown_b096bf2e-7afd-4940-8f70-39c1b1d58bbc,missing_b096bf2e-7afd-4940-8f70-39c1b1d58bbc,missing_75187d2a-01d8-467f-9993-90e200a3a586,b096bf2e-7afd-4940-8f70-39c1b1d58bbc,75187d2a-01d8-467f-9993-90e200a3a586,b096bf2e-7afd-4940-8f70-39c1b1d58bbc75187d2a-0...,US/CAN,USA,2021-03-19,True,False,False,2023-02-27,,False,,False,2023-04-11,True,500.0,,,,False,True,True,0.00,0.0,NaN,0.15
4,2023-09-20 15:55:55.829 America/Los_Angeles,unknown_c12024dd-2444-37dc-b7d5-b8746619eb55,missing_c12024dd-2444-37dc-b7d5-b8746619eb55,missing_75187d2a-01d8-467f-9993-90e200a3a586,c12024dd-2444-37dc-b7d5-b8746619eb55,75187d2a-01d8-467f-9993-90e200a3a586,c12024dd-2444-37dc-b7d5-b8746619eb5575187d2a-0...,US/CAN,USA,2021-03-19,True,,False,,2023-09-03,True,,False,2023-07-26,True,450.0,,,,False,True,True,305.97,0.0,NaN,0.15


Add additional SF fields

In [14]:
# Install Simplesalesforce
#!pip install simple-salesforce 
from simple_salesforce import Salesforce, SalesforceLogin, SFType
import json

In [15]:
# Import credentials
path = "/home/shared/joseph.jensen/Alan Data Pipelines/Credentials/sf_auth_JJ.json"

loginInfo = json.load(open(path))
session_id, instance = SalesforceLogin(username= loginInfo["username"],
                                                 password= loginInfo["password"],
                                                 security_token= loginInfo["security_token"],
                                                 domain= "login")

# Set SF connection
sf = Salesforce(instance= instance, session_id= session_id)

In [16]:
add_fields_3 = """
SELECT
Org_Id_Otter__c,
CR_Account_Categories__c,
CR_Account_Sub_category__c,
CR_Acc_Social_Media_Presence_Numeric__c,
Yelp_of_Reviews__c

FROM
Account
"""


print("Querying data...")

# Query users to dataframe
add_fields_3 = pd.DataFrame(sf.query_all(add_fields_3)['records'])
print("Data pulled!")

# Inspect 
add_fields_3.head()
print("Rows:", len(add_fields_3))

Querying data...
Data pulled!
Rows: 65570


In [17]:
social_media_2 = []

for s in add_fields_3["CR_Acc_Social_Media_Presence_Numeric__c"]:
    if s >= 100000:
        s = '>100k'
    elif s >= 50000:
        s = '50k-100k'
    elif s >= 50000:
        s = '20k-50k'
    elif s >= 10000:
        s = '10k-20k'
    elif s >= 7000:
        s = '7k-10k'
    elif s >= 5000:
        s = '5k-7k'
    elif s >= 3000:
        s = '3k-5k'
    elif s >= 1000:
        s = '1k-3k'
    else:
        s = '<1k'
    social_media_2.append(s)

# Create new col with ex SF name
add_fields_3["CR_Acc_Social_Media_Presence_Group"] = social_media_2
add_fields_3.head(2)

,attributes,Org_Id_Otter__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Acc_Social_Media_Presence_Numeric__c,Yelp_of_Reviews__c,CR_Acc_Social_Media_Presence_Group
0,"{'type': 'Account', 'url': '/services/data/v42...",None,None,None,NaN,NaN,<1k
1,"{'type': 'Account', 'url': '/services/data/v42...",None,None,None,NaN,NaN,<1k


In [18]:
print("len of doors data is {}".format(len(doors_data)))
doors_data.head(2)

len of doors data is 1752


,pulled_date_pst,sfdc_account_name,sfdc_account_id,sfdc_facility_name,prep_organization_id,facility_id,doors_id,region,country,fac_live_date,is_fac_live,is_expansion,is_2p_l4w,closed_won_date,activated_date,is_activated,billable_date,is_billable,churn_date,is_churned,activation_fee,mrr,tcv,storage_cu_ft,beam_sync_qa,org_id_qa,fac_id_qa,sgmv_l2w_1mo,sgmv_2p_l2w_1mo,gmv_mrr_acct,processing_fee_percentage
0,2023-09-20 15:55:55.829 America/Los_Angeles,unknown_48c743af-1022-4e7c-a383-16100bd4da39,missing_48c743af-1022-4e7c-a383-16100bd4da39,missing_94e1cb0b-facf-4f72-b66d-2bd97d98e58f,48c743af-1022-4e7c-a383-16100bd4da39,94e1cb0b-facf-4f72-b66d-2bd97d98e58f,48c743af-1022-4e7c-a383-16100bd4da3994e1cb0b-f...,US/CAN,USA,2021-05-14,True,False,False,2022-07-08,,False,,False,2022-08-18,True,500.0,,,,False,True,True,0.0,0.0,NaN,0.15
1,2023-09-20 15:55:55.829 America/Los_Angeles,unknown_4d602cc1-f405-46e1-b13b-b1d861ede9f9,missing_4d602cc1-f405-46e1-b13b-b1d861ede9f9,missing_99573ac0-a3f2-4c70-946c-e9c06b425d5c,4d602cc1-f405-46e1-b13b-b1d861ede9f9,99573ac0-a3f2-4c70-946c-e9c06b425d5c,4d602cc1-f405-46e1-b13b-b1d861ede9f999573ac0-a...,US/CAN,USA,2021-01-11,True,False,False,2023-02-26,2023-04-11,True,,False,,False,500.0,,,,False,True,True,0.0,0.0,NaN,0.15


In [19]:
doors_data_2 = doors_data.merge(add_fields_3, how = "left", left_on = ["prep_organization_id"], right_on = ["Org_Id_Otter__c"])
print("len of doors data 2 is {}".format(len(doors_data_2)))

len of doors data 2 is 1752


In [20]:
doors_data_2.drop(columns = ["attributes", "Org_Id_Otter__c"], inplace = True)

In [21]:
doors_data_2.head(2)

,pulled_date_pst,sfdc_account_name,sfdc_account_id,sfdc_facility_name,prep_organization_id,facility_id,doors_id,region,country,fac_live_date,is_fac_live,is_expansion,is_2p_l4w,closed_won_date,activated_date,is_activated,billable_date,is_billable,churn_date,is_churned,activation_fee,mrr,tcv,storage_cu_ft,beam_sync_qa,org_id_qa,fac_id_qa,sgmv_l2w_1mo,sgmv_2p_l2w_1mo,gmv_mrr_acct,processing_fee_percentage,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Acc_Social_Media_Presence_Numeric__c,Yelp_of_Reviews__c,CR_Acc_Social_Media_Presence_Group
0,2023-09-20 15:55:55.829 America/Los_Angeles,unknown_48c743af-1022-4e7c-a383-16100bd4da39,missing_48c743af-1022-4e7c-a383-16100bd4da39,missing_94e1cb0b-facf-4f72-b66d-2bd97d98e58f,48c743af-1022-4e7c-a383-16100bd4da39,94e1cb0b-facf-4f72-b66d-2bd97d98e58f,48c743af-1022-4e7c-a383-16100bd4da3994e1cb0b-f...,US/CAN,USA,2021-05-14,True,False,False,2022-07-08,,False,,False,2022-08-18,True,500.0,,,,False,True,True,0.0,0.0,NaN,0.15,NaN,NaN,NaN,NaN,NaN
1,2023-09-20 15:55:55.829 America/Los_Angeles,unknown_4d602cc1-f405-46e1-b13b-b1d861ede9f9,missing_4d602cc1-f405-46e1-b13b-b1d861ede9f9,missing_99573ac0-a3f2-4c70-946c-e9c06b425d5c,4d602cc1-f405-46e1-b13b-b1d861ede9f9,99573ac0-a3f2-4c70-946c-e9c06b425d5c,4d602cc1-f405-46e1-b13b-b1d861ede9f999573ac0-a...,US/CAN,USA,2021-01-11,True,False,False,2023-02-26,2023-04-11,True,,False,,False,500.0,,,,False,True,True,0.0,0.0,NaN,0.15,CR - Baked Goods,CR - Cookies,NaN,NaN,<1k


Upload to SS

In [22]:
# upload doots data to US MBR
print("Authenticating and opening spreadsheet...")
gc = pygsheets.authorize(custom_credentials=my_credentials)
print("Opening Spreadsheet..")
ss = gc.open_by_key("16xST2JWUrK2s-Kla6N82O8B9lHvIBUfJDcAMoxwnLNQ")

# Selecting location
ws = ss.worksheet("title","original_doors")

# Uploading data
print("Clearing sheet ...")
ws.clear(start="A", end="AT")
print("Sheet cleared")

print("Uploading data...")
ws.set_dataframe(doors_data_2, start= "A1",copy_head= True, extend= True, nan=  "")
print("Data uploaded!")

Authenticating and opening spreadsheet...
Opening Spreadsheet..
Clearing sheet ...
Sheet cleared
Uploading data...
Data uploaded!


## Appending both regions DF

**US appending**


In [23]:
orders_data_2.head(2)

,created_week,country,prep_organization_id,prep_organization_name,seller_organization_id,seller_organization_name,facility_id,facility_name,inventory_owned,distribution_type,city,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,items,gmv,lost_gmv,cancelled_gmv,avg_order_rating,orders_20picked,OFOs,cities,facilities
0,2023-08-07,USA,017b46ad-3a10-4d42-80f2-b3b84abb1019,​Tubby's Taste LLC,017b46ad-3a10-4d42-80f2-b3b84abb1019,​Tubby's Taste LLC,75187d2a-01d8-467f-9993-90e200a3a586,NA-US-IL-CHI-River West (Correct),False,3PCR,Chicago,1,0,0,0,1,3.000000,27.00,0.0,0.00,,0,1,1,1
1,2023-08-07,USA,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,4411e09a-c8b7-4a2e-ae6e-5d028e303791,NA-US-TX-DAL-Commerce,False,3PCR,Dallas,6,5,0,5,0,2.333333,127.66,0.0,106.68,,0,1,1,1


In [24]:
# Copy dataset
orders_data_2_copy = orders_data_2.copy()

In [25]:
# Create doors_id field in gmv_mrr table 
orders_data_2_copy["doors_id"] = orders_data_2_copy["prep_organization_id"]+orders_data_2_copy["facility_id"]
orders_data_2_copy.head(2)

,created_week,country,prep_organization_id,prep_organization_name,seller_organization_id,seller_organization_name,facility_id,facility_name,inventory_owned,distribution_type,city,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,items,gmv,lost_gmv,cancelled_gmv,avg_order_rating,orders_20picked,OFOs,cities,facilities,doors_id
0,2023-08-07,USA,017b46ad-3a10-4d42-80f2-b3b84abb1019,​Tubby's Taste LLC,017b46ad-3a10-4d42-80f2-b3b84abb1019,​Tubby's Taste LLC,75187d2a-01d8-467f-9993-90e200a3a586,NA-US-IL-CHI-River West (Correct),False,3PCR,Chicago,1,0,0,0,1,3.000000,27.00,0.0,0.00,,0,1,1,1,017b46ad-3a10-4d42-80f2-b3b84abb101975187d2a-0...
1,2023-08-07,USA,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,4411e09a-c8b7-4a2e-ae6e-5d028e303791,NA-US-TX-DAL-Commerce,False,3PCR,Dallas,6,5,0,5,0,2.333333,127.66,0.0,106.68,,0,1,1,1,658618ea-2e32-47ba-8a6b-fe89342ad1c54411e09a-c...


In [26]:
print("orig orders data shape is", orders_data_2_copy.shape)


orig orders data shape is (7792, 26)


In [27]:
# Matching to gmv_mrr_opp cols 

# No hay Cuisine Type, City, pf int

cols = ["doors_id",
        "sfdc_account_name",
        "sfdc_account_id",
        "is_churned", 
        "churn_date",
        "closed_won_date",
        "activated_date", 
        "billable_date", 
        "is_billable", 
        "storage_cu_ft", 
        "mrr",
        "activation_fee", 
        "processing_fee_percentage"]

In [28]:
# Doors test
doors_test = doors_data[cols]
doors_test["closed_won_date"] = pd.to_datetime(doors_test["closed_won_date"])
doors_test = doors_test.sort_values(by = "closed_won_date", ascending = False).reset_index(drop = True)

# Inspect 
print("Rows:", len(doors_test))

Rows: 1752


In [29]:
doors_clean = doors_test.drop_duplicates(subset = ["doors_id"], keep = "first").reset_index(drop = True)

In [30]:
print("clean doors data shape is", doors_clean.shape)


clean doors data shape is (1651, 13)


# merge gmv with doors

In [31]:
# Merge DF
gmv_mrr_us = orders_data_2_copy.merge(doors_clean, how = "left", left_on = ["doors_id"], right_on = ["doors_id"])

In [32]:
print("gmv_mrr_us data shape is", gmv_mrr_us.shape)


gmv_mrr_us data shape is (7792, 38)


In [33]:
gmv_mrr_us.head()

,created_week,country,prep_organization_id,prep_organization_name,seller_organization_id,seller_organization_name,facility_id,facility_name,inventory_owned,distribution_type,city,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,items,gmv,lost_gmv,cancelled_gmv,avg_order_rating,orders_20picked,OFOs,cities,facilities,doors_id,sfdc_account_name,sfdc_account_id,is_churned,churn_date,closed_won_date,activated_date,billable_date,is_billable,storage_cu_ft,mrr,activation_fee,processing_fee_percentage
0,2023-08-07,USA,017b46ad-3a10-4d42-80f2-b3b84abb1019,​Tubby's Taste LLC,017b46ad-3a10-4d42-80f2-b3b84abb1019,​Tubby's Taste LLC,75187d2a-01d8-467f-9993-90e200a3a586,NA-US-IL-CHI-River West (Correct),False,3PCR,Chicago,1,0,0,0,1,3.000000,27.00,0.0,0.00,,0,1,1,1,017b46ad-3a10-4d42-80f2-b3b84abb101975187d2a-0...,unknown_017b46ad-3a10-4d42-80f2-b3b84abb1019,missing_017b46ad-3a10-4d42-80f2-b3b84abb1019,False,,2023-07-20,2023-08-10,,False,,,120.0,
1,2023-08-07,USA,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,4411e09a-c8b7-4a2e-ae6e-5d028e303791,NA-US-TX-DAL-Commerce,False,3PCR,Dallas,6,5,0,5,0,2.333333,127.66,0.0,106.68,,0,1,1,1,658618ea-2e32-47ba-8a6b-fe89342ad1c54411e09a-c...,unknown_658618ea-2e32-47ba-8a6b-fe89342ad1c5,missing_658618ea-2e32-47ba-8a6b-fe89342ad1c5,False,,NaT,,,False,,,120.0,
2,2023-07-31,USA,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,4411e09a-c8b7-4a2e-ae6e-5d028e303791,NA-US-TX-DAL-Commerce,False,3PCR,Dallas,2,2,0,2,0,2.000000,37.72,0.0,37.72,,0,1,1,1,658618ea-2e32-47ba-8a6b-fe89342ad1c54411e09a-c...,unknown_658618ea-2e32-47ba-8a6b-fe89342ad1c5,missing_658618ea-2e32-47ba-8a6b-fe89342ad1c5,False,,NaT,,,False,,,120.0,
3,2023-08-14,USA,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,4411e09a-c8b7-4a2e-ae6e-5d028e303791,NA-US-TX-DAL-Commerce,False,3PCR,Dallas,1,1,0,1,0,1.000000,3.99,0.0,3.99,,0,1,1,1,658618ea-2e32-47ba-8a6b-fe89342ad1c54411e09a-c...,unknown_658618ea-2e32-47ba-8a6b-fe89342ad1c5,missing_658618ea-2e32-47ba-8a6b-fe89342ad1c5,False,,NaT,,,False,,,120.0,
4,2023-07-31,USA,6440b884-ccd2-465d-b220-b93b8c7cda62,p.o.p. candy co.,6440b884-ccd2-465d-b220-b93b8c7cda62,p.o.p. candy co.,99573ac0-a3f2-4c70-946c-e9c06b425d5c,NA-US-CA(S)-LA-Koreatown,False,3PCR,Los Angeles,2,2,0,2,1,2.000000,64.00,0.0,64.00,,0,1,1,1,6440b884-ccd2-465d-b220-b93b8c7cda6299573ac0-a...,unknown_6440b884-ccd2-465d-b220-b93b8c7cda62,missing_6440b884-ccd2-465d-b220-b93b8c7cda62,True,2023-05-16,2023-01-25,2023-02-10,,False,,,500.0,0.15


In [34]:
gmv_mrr_us["processing_fee_percentage"].unique()

array(['', 0.15, nan, 0.05, 0.3, 5.0, 0.1], dtype=object)

In [35]:
gmv_mrr_us["total_orders"] = pd.to_numeric(gmv_mrr_us["total_orders"])

In [36]:
# Convert string columns to numeric
gmv_mrr_us["cancelled_orders"] = pd.to_numeric(gmv_mrr_us["cancelled_orders"])
gmv_mrr_us["issue_orders"] = pd.to_numeric(gmv_mrr_us["issue_orders"])
gmv_mrr_us["ODR_orders"] = pd.to_numeric(gmv_mrr_us["ODR_orders"])
gmv_mrr_us["scanned_orders"] = pd.to_numeric(gmv_mrr_us["scanned_orders"])
gmv_mrr_us["mrr"] = pd.to_numeric(gmv_mrr_us["mrr"])
gmv_mrr_us["lost_gmv"] = pd.to_numeric(gmv_mrr_us["lost_gmv"])

In [37]:
# Calculations

# ODR metrics
gmv_mrr_us["pct_cancelled"] = gmv_mrr_us["cancelled_orders"] / gmv_mrr_us["total_orders"]
gmv_mrr_us["pct_issues"] = gmv_mrr_us["issue_orders"] / gmv_mrr_us["total_orders"]
gmv_mrr_us["pct_odr"] = gmv_mrr_us["ODR_orders"] /  gmv_mrr_us["total_orders"]
gmv_mrr_us["pct_scanned"] = gmv_mrr_us["scanned_orders"] / gmv_mrr_us["total_orders"]

In [38]:
# Calculating GMV, MRR and Health
gmv_mrr_us["succesful_orders"] = gmv_mrr_us["total_orders"] - gmv_mrr_us["cancelled_orders"]
gmv_mrr_us["succesful_gmv"] = gmv_mrr_us["gmv"] - gmv_mrr_us["cancelled_gmv"] + gmv_mrr_us["lost_gmv"] # Lost GMV is negative value

In [39]:
gmv_mrr_us["processing_fee_percentage"]=pd.to_numeric(gmv_mrr_us["processing_fee_percentage"])

In [40]:
gmv_mrr_us["est_processing_income"] = gmv_mrr_us["succesful_gmv"] * gmv_mrr_us["processing_fee_percentage"]

In [41]:
# Calculating processing income test with SKIMs adjustment
gmv_mrr_us["total_orders"] = gmv_mrr_us["total_orders"].fillna(0).astype(int)
gmv_mrr_us["ad_est_processing_income"] = gmv_mrr_us.apply(lambda x: x["succesful_orders"] * 5 if x["prep_organization_id"] == 'a10e3874-24c6-4c9e-a966-537f2c43369d' else x["succesful_gmv"] * x["processing_fee_percentage"], axis = 1)

In [42]:
gmv_mrr_us.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7792 entries, 0 to 7791
Data columns (total 46 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   created_week               7792 non-null   object        
 1   country                    7792 non-null   object        
 2   prep_organization_id       7792 non-null   object        
 3   prep_organization_name     7792 non-null   object        
 4   seller_organization_id     7792 non-null   object        
 5   seller_organization_name   7792 non-null   object        
 6   facility_id                7792 non-null   object        
 7   facility_name              7792 non-null   object        
 8   inventory_owned            7792 non-null   bool          
 9   distribution_type          7792 non-null   object        
 10  city                       7792 non-null   object        
 11  total_orders               7792 non-null   int64         
 12  cancel

In [43]:
# Calculating gmv/mrr ratio

gmv_mrr_us["store_gmv_mrr"] = gmv_mrr_us["succesful_gmv"] / ((gmv_mrr_us["mrr"]/30)*7)

# Create org_health categories
health_cats = []
for i in gmv_mrr_us['store_gmv_mrr']:
    if i>=15:
        j='15x or more'
    elif i>=10:
        j='10x to 14.9x'
    elif i>=5:
        j='5x to 9.9x'
    elif i>=4:
        j='4x'
    elif i>=3:
        j='3x'
    elif i>=2:
        j='2x'
    elif i>=1:
        j='1x'
    else:
        j='0X'
    health_cats.append(j)

# Append categories
gmv_mrr_us["gmv_mrr_cat"] = health_cats

In [44]:
gmv_mrr_us.head()

,created_week,country,prep_organization_id,prep_organization_name,seller_organization_id,seller_organization_name,facility_id,facility_name,inventory_owned,distribution_type,city,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,items,gmv,lost_gmv,cancelled_gmv,avg_order_rating,orders_20picked,OFOs,cities,facilities,doors_id,sfdc_account_name,sfdc_account_id,is_churned,churn_date,closed_won_date,activated_date,billable_date,is_billable,storage_cu_ft,mrr,activation_fee,processing_fee_percentage,pct_cancelled,pct_issues,pct_odr,pct_scanned,succesful_orders,succesful_gmv,est_processing_income,ad_est_processing_income,store_gmv_mrr,gmv_mrr_cat
0,2023-08-07,USA,017b46ad-3a10-4d42-80f2-b3b84abb1019,​Tubby's Taste LLC,017b46ad-3a10-4d42-80f2-b3b84abb1019,​Tubby's Taste LLC,75187d2a-01d8-467f-9993-90e200a3a586,NA-US-IL-CHI-River West (Correct),False,3PCR,Chicago,1,0,0,0,1,3.000000,27.00,0.0,0.00,,0,1,1,1,017b46ad-3a10-4d42-80f2-b3b84abb101975187d2a-0...,unknown_017b46ad-3a10-4d42-80f2-b3b84abb1019,missing_017b46ad-3a10-4d42-80f2-b3b84abb1019,False,,2023-07-20,2023-08-10,,False,,NaN,120.0,NaN,0.000000,0.0,0.000000,1.0,1,27.00,NaN,NaN,NaN,0X
1,2023-08-07,USA,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,4411e09a-c8b7-4a2e-ae6e-5d028e303791,NA-US-TX-DAL-Commerce,False,3PCR,Dallas,6,5,0,5,0,2.333333,127.66,0.0,106.68,,0,1,1,1,658618ea-2e32-47ba-8a6b-fe89342ad1c54411e09a-c...,unknown_658618ea-2e32-47ba-8a6b-fe89342ad1c5,missing_658618ea-2e32-47ba-8a6b-fe89342ad1c5,False,,NaT,,,False,,NaN,120.0,NaN,0.833333,0.0,0.833333,0.0,1,20.98,NaN,NaN,NaN,0X
2,2023-07-31,USA,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,4411e09a-c8b7-4a2e-ae6e-5d028e303791,NA-US-TX-DAL-Commerce,False,3PCR,Dallas,2,2,0,2,0,2.000000,37.72,0.0,37.72,,0,1,1,1,658618ea-2e32-47ba-8a6b-fe89342ad1c54411e09a-c...,unknown_658618ea-2e32-47ba-8a6b-fe89342ad1c5,missing_658618ea-2e32-47ba-8a6b-fe89342ad1c5,False,,NaT,,,False,,NaN,120.0,NaN,1.000000,0.0,1.000000,0.0,0,0.00,NaN,NaN,NaN,0X
3,2023-08-14,USA,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,4411e09a-c8b7-4a2e-ae6e-5d028e303791,NA-US-TX-DAL-Commerce,False,3PCR,Dallas,1,1,0,1,0,1.000000,3.99,0.0,3.99,,0,1,1,1,658618ea-2e32-47ba-8a6b-fe89342ad1c54411e09a-c...,unknown_658618ea-2e32-47ba-8a6b-fe89342ad1c5,missing_658618ea-2e32-47ba-8a6b-fe89342ad1c5,False,,NaT,,,False,,NaN,120.0,NaN,1.000000,0.0,1.000000,0.0,0,0.00,NaN,NaN,NaN,0X
4,2023-07-31,USA,6440b884-ccd2-465d-b220-b93b8c7cda62,p.o.p. candy co.,6440b884-ccd2-465d-b220-b93b8c7cda62,p.o.p. candy co.,99573ac0-a3f2-4c70-946c-e9c06b425d5c,NA-US-CA(S)-LA-Koreatown,False,3PCR,Los Angeles,2,2,0,2,1,2.000000,64.00,0.0,64.00,,0,1,1,1,6440b884-ccd2-465d-b220-b93b8c7cda6299573ac0-a...,unknown_6440b884-ccd2-465d-b220-b93b8c7cda62,missing_6440b884-ccd2-465d-b220-b93b8c7cda62,True,2023-05-16,2023-01-25,2023-02-10,,False,,NaN,500.0,0.15,1.000000,0.0,1.000000,0.5,0,0.00,0.0,0.0,NaN,0X


Import additional fields from SF

In [45]:
# Install Simplesalesforce
# !pip install simple-salesforce 
from simple_salesforce import Salesforce, SalesforceLogin, SFType
import json

In [46]:
# Import credentials
path = "/home/shared/joseph.jensen/Alan Data Pipelines/Credentials/sf_auth_JJ.json"

loginInfo = json.load(open(path))
session_id, instance = SalesforceLogin(username= loginInfo["username"],
                                                 password= loginInfo["password"],
                                                 security_token= loginInfo["security_token"],
                                                 domain= "login")

# Set SF connection
sf = Salesforce(instance= instance, session_id= session_id)

In [47]:
# Query category data from Salesforce
sf_query = """

SELECT
account.Account_ID_18__c, CR_Opp_OB_Status__c,
Type

FROM
Opportunity

WHERE
RecordType.Name IN('Virtual', 'CloudRetail','virtual;cloudretail', 'Cloud_Retail')

"""


print("Querying data...")

# Query users to dataframe
sf_data = pd.DataFrame(sf.query_all(sf_query)['records'])
print("Data pulled!")

# Inspect 
print("Rows:", len(sf_data))

Querying data...
Data pulled!
Rows: 35705


In [48]:
sf_data.head(3)

,attributes,Account,CR_Opp_OB_Status__c,Type
0,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",None,None
1,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",b - Documentation,Monthly Contract
2,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",None,None


In [49]:
# Extract account details from accounts list in accounts column
org_ids = [] # lista orgs id
for i in sf_data['Account']:
    try:
        j=(i.get('Account_ID_18__c'))
        #print(j)
        org_ids.append(j)
    except:
        org_ids.append(None)

In [50]:
# Append extracted lists to DF
sf_data['sfdc_org_id']= org_ids

In [51]:
sf_data.drop(columns = ["attributes", "Account"], inplace = True)

In [52]:
sf_data.head(3)

,CR_Opp_OB_Status__c,Type,sfdc_org_id
0,None,None,0015G00001jWLroQAG
1,b - Documentation,Monthly Contract,0016T000035CBhjQAG
2,None,None,0016T000036t34ZQAQ


In [53]:
print("before dupes:", len(sf_data))
sf_data = sf_data.drop_duplicates(subset = "sfdc_org_id")
print("after dropping dupes:", len(sf_data))

before dupes: 35705
after dropping dupes: 27383


In [54]:
print("gmv_mrr_us shape is {}".format(gmv_mrr_us.shape))

gmv_mrr_us shape is (7792, 48)


In [55]:
# Merge
gmv_mrr_us_2 = gmv_mrr_us.merge(sf_data, how = 'left', left_on = ['sfdc_account_id'], right_on = ['sfdc_org_id'])

In [56]:
print("gmv_mrr_us_2 shape is {}".format(gmv_mrr_us_2.shape))

gmv_mrr_us_2 shape is (7792, 51)


In [57]:
gmv_mrr_us_2 = gmv_mrr_us_2.drop(columns = 'sfdc_org_id')

In [58]:
gmv_mrr_us_2.head(2)

,created_week,country,prep_organization_id,prep_organization_name,seller_organization_id,seller_organization_name,facility_id,facility_name,inventory_owned,distribution_type,city,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,items,gmv,lost_gmv,cancelled_gmv,avg_order_rating,orders_20picked,OFOs,cities,facilities,doors_id,sfdc_account_name,sfdc_account_id,is_churned,churn_date,closed_won_date,activated_date,billable_date,is_billable,storage_cu_ft,mrr,activation_fee,processing_fee_percentage,pct_cancelled,pct_issues,pct_odr,pct_scanned,succesful_orders,succesful_gmv,est_processing_income,ad_est_processing_income,store_gmv_mrr,gmv_mrr_cat,CR_Opp_OB_Status__c,Type
0,2023-08-07,USA,017b46ad-3a10-4d42-80f2-b3b84abb1019,​Tubby's Taste LLC,017b46ad-3a10-4d42-80f2-b3b84abb1019,​Tubby's Taste LLC,75187d2a-01d8-467f-9993-90e200a3a586,NA-US-IL-CHI-River West (Correct),False,3PCR,Chicago,1,0,0,0,1,3.000000,27.00,0.0,0.00,,0,1,1,1,017b46ad-3a10-4d42-80f2-b3b84abb101975187d2a-0...,unknown_017b46ad-3a10-4d42-80f2-b3b84abb1019,missing_017b46ad-3a10-4d42-80f2-b3b84abb1019,False,,2023-07-20,2023-08-10,,False,,NaN,120.0,NaN,0.000000,0.0,0.000000,1.0,1,27.00,NaN,NaN,NaN,0X,NaN,NaN
1,2023-08-07,USA,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,4411e09a-c8b7-4a2e-ae6e-5d028e303791,NA-US-TX-DAL-Commerce,False,3PCR,Dallas,6,5,0,5,0,2.333333,127.66,0.0,106.68,,0,1,1,1,658618ea-2e32-47ba-8a6b-fe89342ad1c54411e09a-c...,unknown_658618ea-2e32-47ba-8a6b-fe89342ad1c5,missing_658618ea-2e32-47ba-8a6b-fe89342ad1c5,False,,NaT,,,False,,NaN,120.0,NaN,0.833333,0.0,0.833333,0.0,1,20.98,NaN,NaN,NaN,0X,NaN,NaN


Add cat, subcat and followers

In [59]:
# Import credentials
path = "/home/shared/joseph.jensen/Alan Data Pipelines/Credentials/sf_auth_JJ.json"

loginInfo = json.load(open(path))
session_id, instance = SalesforceLogin(username= loginInfo["username"],
                                                 password= loginInfo["password"],
                                                 security_token= loginInfo["security_token"],
                                                 domain= "login")

# Set SF connection
sf = Salesforce(instance= instance, session_id= session_id)

In [60]:
add_fields_2 = """
SELECT
Org_Id_Otter__c,
CR_Account_Categories__c,
CR_Account_Sub_category__c,
CR_Acc_Social_Media_Presence_Numeric__c,
Yelp_of_Reviews__c

FROM
Account
"""


print("Querying data...")

# Query users to dataframe
add_fields_2 = pd.DataFrame(sf.query_all(add_fields_2)['records'])
print("Data pulled!")

# Inspect 
add_fields_2.head()
print("Rows:", len(add_fields_2))

Querying data...
Data pulled!
Rows: 65570


In [61]:
print("gmv mrr us len is {}".format(len(gmv_mrr_us_2)))

gmv mrr us len is 7792


In [62]:
# Merge
gmv_mrr_us_3 = gmv_mrr_us_2.merge(add_fields_2, how = "left", left_on = ["prep_organization_id"], right_on = ["Org_Id_Otter__c"])
print("gmv mrr us new len is {}".format(len(gmv_mrr_us_3)))

gmv mrr us new len is 7792


In [63]:
# Create bucket for social media presence with same SF field name
social_media_num = []

for s in gmv_mrr_us_3["CR_Acc_Social_Media_Presence_Numeric__c"]:
    if s >= 100000:
        s = '>100k'
    elif s >= 50000:
        s = '50k-100k'
    elif s >= 50000:
        s = '20k-50k'
    elif s >= 10000:
        s = '10k-20k'
    elif s >= 7000:
        s = '7k-10k'
    elif s >= 5000:
        s = '5k-7k'
    elif s >= 3000:
        s = '3k-5k'
    elif s >= 1000:
        s = '1k-3k'
    else:
        s = '<1k'
    social_media_num.append(s)

# Create new col with ex SF name
gmv_mrr_us_3["CR_Acc_Social_Media_Presence_Group"] = social_media_num

In [64]:
gmv_mrr_us_3.head()

,created_week,country,prep_organization_id,prep_organization_name,seller_organization_id,seller_organization_name,facility_id,facility_name,inventory_owned,distribution_type,city,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,items,gmv,lost_gmv,cancelled_gmv,avg_order_rating,orders_20picked,OFOs,cities,facilities,doors_id,sfdc_account_name,sfdc_account_id,is_churned,churn_date,closed_won_date,activated_date,billable_date,is_billable,storage_cu_ft,mrr,activation_fee,processing_fee_percentage,pct_cancelled,pct_issues,pct_odr,pct_scanned,succesful_orders,succesful_gmv,est_processing_income,ad_est_processing_income,store_gmv_mrr,gmv_mrr_cat,CR_Opp_OB_Status__c,Type,attributes,Org_Id_Otter__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Acc_Social_Media_Presence_Numeric__c,Yelp_of_Reviews__c,CR_Acc_Social_Media_Presence_Group
0,2023-08-07,USA,017b46ad-3a10-4d42-80f2-b3b84abb1019,​Tubby's Taste LLC,017b46ad-3a10-4d42-80f2-b3b84abb1019,​Tubby's Taste LLC,75187d2a-01d8-467f-9993-90e200a3a586,NA-US-IL-CHI-River West (Correct),False,3PCR,Chicago,1,0,0,0,1,3.000000,27.00,0.0,0.00,,0,1,1,1,017b46ad-3a10-4d42-80f2-b3b84abb101975187d2a-0...,unknown_017b46ad-3a10-4d42-80f2-b3b84abb1019,missing_017b46ad-3a10-4d42-80f2-b3b84abb1019,False,,2023-07-20,2023-08-10,,False,,NaN,120.0,NaN,0.000000,0.0,0.000000,1.0,1,27.00,NaN,NaN,NaN,0X,NaN,NaN,"{'type': 'Account', 'url': '/services/data/v42...",017b46ad-3a10-4d42-80f2-b3b84abb1019,CR - Baked Goods,CR - Cookies,5851.0,5.0,5k-7k
1,2023-08-07,USA,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,4411e09a-c8b7-4a2e-ae6e-5d028e303791,NA-US-TX-DAL-Commerce,False,3PCR,Dallas,6,5,0,5,0,2.333333,127.66,0.0,106.68,,0,1,1,1,658618ea-2e32-47ba-8a6b-fe89342ad1c54411e09a-c...,unknown_658618ea-2e32-47ba-8a6b-fe89342ad1c5,missing_658618ea-2e32-47ba-8a6b-fe89342ad1c5,False,,NaT,,,False,,NaN,120.0,NaN,0.833333,0.0,0.833333,0.0,1,20.98,NaN,NaN,NaN,0X,NaN,NaN,"{'type': 'Account', 'url': '/services/data/v42...",658618ea-2e32-47ba-8a6b-fe89342ad1c5,CR - Baked Goods,CR - Bakery,350.0,4.5,<1k
2,2023-07-31,USA,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,4411e09a-c8b7-4a2e-ae6e-5d028e303791,NA-US-TX-DAL-Commerce,False,3PCR,Dallas,2,2,0,2,0,2.000000,37.72,0.0,37.72,,0,1,1,1,658618ea-2e32-47ba-8a6b-fe89342ad1c54411e09a-c...,unknown_658618ea-2e32-47ba-8a6b-fe89342ad1c5,missing_658618ea-2e32-47ba-8a6b-fe89342ad1c5,False,,NaT,,,False,,NaN,120.0,NaN,1.000000,0.0,1.000000,0.0,0,0.00,NaN,NaN,NaN,0X,NaN,NaN,"{'type': 'Account', 'url': '/services/data/v42...",658618ea-2e32-47ba-8a6b-fe89342ad1c5,CR - Baked Goods,CR - Bakery,350.0,4.5,<1k
3,2023-08-14,USA,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,4411e09a-c8b7-4a2e-ae6e-5d028e303791,NA-US-TX-DAL-Commerce,False,3PCR,Dallas,1,1,0,1,0,1.000000,3.99,0.0,3.99,,0,1,1,1,658618ea-2e32-47ba-8a6b-fe89342ad1c54411e09a-c...,unknown_658618ea-2e32-47ba-8a6b-fe89342ad1c5,missing_658618ea-2e32-47ba-8a6b-fe89342ad1c5,False,,NaT,,,False,,NaN,120.0,NaN,1.000000,0.0,1.000000,0.0,0,0.00,NaN,NaN,NaN,0X,NaN,NaN,"{'type': 'Account', 'url': '/services/data/v42...",658618ea-2e32-47ba-8a6b-fe89342ad1c5,CR - Baked Goods,CR - Bakery,350.0,4.5,<1k
4,2023-07-31,USA,6440b884-ccd2-465d-b220-b93b8c7cda62,p.o.p. candy co.,6440b884-ccd2-465d-b220-b93b8c7cda62,p.o.p. candy co.,99573ac0-a3f2-4c70-946c-e9c06b425d5c,NA-US-CA(S)-LA-Koreatown,False,3PCR,Los Angeles,2,2,0,2,1,2.000000,64.00,0.0,64.00,,0,1,1,1,6440b884-ccd2-465d-b220-b93b8c7cda6299573ac0-a...,unknown_6440b884-ccd2-465d-b220-b93b8c7cda62,missing_6440b884-ccd2-465d-b220-b93b8c7cda62,True,2023-05-16,2023-01-25,2023-02-10,,False,,NaN,500.0,0.15,1.000000,0.0,1.000000,0.5,0,0.00,0.0,0.0,NaN,0X,NaN,NaN,"{'type': 'Account', 'url': '/services/data/v42...",6440b884-ccd2-465d-b220-b93b8c7cda62,CR - Desserts,CR - Candy,2766.0,NaN,1k-3

In [65]:
# Drop cols
gmv_mrr_us_3.drop(columns = ["attributes","Org_Id_Otter__c"], inplace = True)
gmv_mrr_us_3.head()

,created_week,country,prep_organization_id,prep_organization_name,seller_organization_id,seller_organization_name,facility_id,facility_name,inventory_owned,distribution_type,city,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,items,gmv,lost_gmv,cancelled_gmv,avg_order_rating,orders_20picked,OFOs,cities,facilities,doors_id,sfdc_account_name,sfdc_account_id,is_churned,churn_date,closed_won_date,activated_date,billable_date,is_billable,storage_cu_ft,mrr,activation_fee,processing_fee_percentage,pct_cancelled,pct_issues,pct_odr,pct_scanned,succesful_orders,succesful_gmv,est_processing_income,ad_est_processing_income,store_gmv_mrr,gmv_mrr_cat,CR_Opp_OB_Status__c,Type,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Acc_Social_Media_Presence_Numeric__c,Yelp_of_Reviews__c,CR_Acc_Social_Media_Presence_Group
0,2023-08-07,USA,017b46ad-3a10-4d42-80f2-b3b84abb1019,​Tubby's Taste LLC,017b46ad-3a10-4d42-80f2-b3b84abb1019,​Tubby's Taste LLC,75187d2a-01d8-467f-9993-90e200a3a586,NA-US-IL-CHI-River West (Correct),False,3PCR,Chicago,1,0,0,0,1,3.000000,27.00,0.0,0.00,,0,1,1,1,017b46ad-3a10-4d42-80f2-b3b84abb101975187d2a-0...,unknown_017b46ad-3a10-4d42-80f2-b3b84abb1019,missing_017b46ad-3a10-4d42-80f2-b3b84abb1019,False,,2023-07-20,2023-08-10,,False,,NaN,120.0,NaN,0.000000,0.0,0.000000,1.0,1,27.00,NaN,NaN,NaN,0X,NaN,NaN,CR - Baked Goods,CR - Cookies,5851.0,5.0,5k-7k
1,2023-08-07,USA,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,4411e09a-c8b7-4a2e-ae6e-5d028e303791,NA-US-TX-DAL-Commerce,False,3PCR,Dallas,6,5,0,5,0,2.333333,127.66,0.0,106.68,,0,1,1,1,658618ea-2e32-47ba-8a6b-fe89342ad1c54411e09a-c...,unknown_658618ea-2e32-47ba-8a6b-fe89342ad1c5,missing_658618ea-2e32-47ba-8a6b-fe89342ad1c5,False,,NaT,,,False,,NaN,120.0,NaN,0.833333,0.0,0.833333,0.0,1,20.98,NaN,NaN,NaN,0X,NaN,NaN,CR - Baked Goods,CR - Bakery,350.0,4.5,<1k
2,2023-07-31,USA,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,4411e09a-c8b7-4a2e-ae6e-5d028e303791,NA-US-TX-DAL-Commerce,False,3PCR,Dallas,2,2,0,2,0,2.000000,37.72,0.0,37.72,,0,1,1,1,658618ea-2e32-47ba-8a6b-fe89342ad1c54411e09a-c...,unknown_658618ea-2e32-47ba-8a6b-fe89342ad1c5,missing_658618ea-2e32-47ba-8a6b-fe89342ad1c5,False,,NaT,,,False,,NaN,120.0,NaN,1.000000,0.0,1.000000,0.0,0,0.00,NaN,NaN,NaN,0X,NaN,NaN,CR - Baked Goods,CR - Bakery,350.0,4.5,<1k
3,2023-08-14,USA,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,4411e09a-c8b7-4a2e-ae6e-5d028e303791,NA-US-TX-DAL-Commerce,False,3PCR,Dallas,1,1,0,1,0,1.000000,3.99,0.0,3.99,,0,1,1,1,658618ea-2e32-47ba-8a6b-fe89342ad1c54411e09a-c...,unknown_658618ea-2e32-47ba-8a6b-fe89342ad1c5,missing_658618ea-2e32-47ba-8a6b-fe89342ad1c5,False,,NaT,,,False,,NaN,120.0,NaN,1.000000,0.0,1.000000,0.0,0,0.00,NaN,NaN,NaN,0X,NaN,NaN,CR - Baked Goods,CR - Bakery,350.0,4.5,<1k
4,2023-07-31,USA,6440b884-ccd2-465d-b220-b93b8c7cda62,p.o.p. candy co.,6440b884-ccd2-465d-b220-b93b8c7cda62,p.o.p. candy co.,99573ac0-a3f2-4c70-946c-e9c06b425d5c,NA-US-CA(S)-LA-Koreatown,False,3PCR,Los Angeles,2,2,0,2,1,2.000000,64.00,0.0,64.00,,0,1,1,1,6440b884-ccd2-465d-b220-b93b8c7cda6299573ac0-a...,unknown_6440b884-ccd2-465d-b220-b93b8c7cda62,missing_6440b884-ccd2-465d-b220-b93b8c7cda62,True,2023-05-16,2023-01-25,2023-02-10,,False,,NaN,500.0,0.15,1.000000,0.0,1.000000,0.5,0,0.00,0.0,0.0,NaN,0X,NaN,NaN,CR - Desserts,CR - Candy,2766.0,NaN,1k-3k


Add a last pulled timestamp

In [66]:
import pytz
import datetime
from datetime import timedelta

In [67]:
# Mex : gmv_mrr_us_2["data_pulled_ts_MEX"] = datetime.datetime.now(pytz.timezone('Mexico/General'))
gmv_mrr_us_3["data_pulled_ts_LAX"] = datetime.datetime.now(pytz.timezone('America/Los_Angeles'))

In [68]:
gmv_mrr_us_3.head()

,created_week,country,prep_organization_id,prep_organization_name,seller_organization_id,seller_organization_name,facility_id,facility_name,inventory_owned,distribution_type,city,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,items,gmv,lost_gmv,cancelled_gmv,avg_order_rating,orders_20picked,OFOs,cities,facilities,doors_id,sfdc_account_name,sfdc_account_id,is_churned,churn_date,closed_won_date,activated_date,billable_date,is_billable,storage_cu_ft,mrr,activation_fee,processing_fee_percentage,pct_cancelled,pct_issues,pct_odr,pct_scanned,succesful_orders,succesful_gmv,est_processing_income,ad_est_processing_income,store_gmv_mrr,gmv_mrr_cat,CR_Opp_OB_Status__c,Type,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Acc_Social_Media_Presence_Numeric__c,Yelp_of_Reviews__c,CR_Acc_Social_Media_Presence_Group,data_pulled_ts_LAX
0,2023-08-07,USA,017b46ad-3a10-4d42-80f2-b3b84abb1019,​Tubby's Taste LLC,017b46ad-3a10-4d42-80f2-b3b84abb1019,​Tubby's Taste LLC,75187d2a-01d8-467f-9993-90e200a3a586,NA-US-IL-CHI-River West (Correct),False,3PCR,Chicago,1,0,0,0,1,3.000000,27.00,0.0,0.00,,0,1,1,1,017b46ad-3a10-4d42-80f2-b3b84abb101975187d2a-0...,unknown_017b46ad-3a10-4d42-80f2-b3b84abb1019,missing_017b46ad-3a10-4d42-80f2-b3b84abb1019,False,,2023-07-20,2023-08-10,,False,,NaN,120.0,NaN,0.000000,0.0,0.000000,1.0,1,27.00,NaN,NaN,NaN,0X,NaN,NaN,CR - Baked Goods,CR - Cookies,5851.0,5.0,5k-7k,2023-09-20 15:56:41.170361-07:00
1,2023-08-07,USA,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,4411e09a-c8b7-4a2e-ae6e-5d028e303791,NA-US-TX-DAL-Commerce,False,3PCR,Dallas,6,5,0,5,0,2.333333,127.66,0.0,106.68,,0,1,1,1,658618ea-2e32-47ba-8a6b-fe89342ad1c54411e09a-c...,unknown_658618ea-2e32-47ba-8a6b-fe89342ad1c5,missing_658618ea-2e32-47ba-8a6b-fe89342ad1c5,False,,NaT,,,False,,NaN,120.0,NaN,0.833333,0.0,0.833333,0.0,1,20.98,NaN,NaN,NaN,0X,NaN,NaN,CR - Baked Goods,CR - Bakery,350.0,4.5,<1k,2023-09-20 15:56:41.170361-07:00
2,2023-07-31,USA,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,4411e09a-c8b7-4a2e-ae6e-5d028e303791,NA-US-TX-DAL-Commerce,False,3PCR,Dallas,2,2,0,2,0,2.000000,37.72,0.0,37.72,,0,1,1,1,658618ea-2e32-47ba-8a6b-fe89342ad1c54411e09a-c...,unknown_658618ea-2e32-47ba-8a6b-fe89342ad1c5,missing_658618ea-2e32-47ba-8a6b-fe89342ad1c5,False,,NaT,,,False,,NaN,120.0,NaN,1.000000,0.0,1.000000,0.0,0,0.00,NaN,NaN,NaN,0X,NaN,NaN,CR - Baked Goods,CR - Bakery,350.0,4.5,<1k,2023-09-20 15:56:41.170361-07:00
3,2023-08-14,USA,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,4411e09a-c8b7-4a2e-ae6e-5d028e303791,NA-US-TX-DAL-Commerce,False,3PCR,Dallas,1,1,0,1,0,1.000000,3.99,0.0,3.99,,0,1,1,1,658618ea-2e32-47ba-8a6b-fe89342ad1c54411e09a-c...,unknown_658618ea-2e32-47ba-8a6b-fe89342ad1c5,missing_658618ea-2e32-47ba-8a6b-fe89342ad1c5,False,,NaT,,,False,,NaN,120.0,NaN,1.000000,0.0,1.000000,0.0,0,0.00,NaN,NaN,NaN,0X,NaN,NaN,CR - Baked Goods,CR - Bakery,350.0,4.5,<1k,2023-09-20 15:56:41.170361-07:00
4,2023-07-31,USA,6440b884-ccd2-465d-b220-b93b8c7cda62,p.o.p. candy co.,6440b884-ccd2-465d-b220-b93b8c7cda62,p.o.p. candy co.,99573ac0-a3f2-4c70-946c-e9c06b425d5c,NA-US-CA(S)-LA-Koreatown,False,3PCR,Los Angeles,2,2,0,2,1,2.000000,64.00,0.0,64.00,,0,1,1,1,6440b884-ccd2-465d-b220-b93b8c7cda6299573ac0-a...,unknown_6440b884-ccd2-465d-b220-b93b8c7cda62,missing_6440b884-ccd2-465d-b220-b93b8c7cda62,True,2023-05-16,2023-01-25,2023-02-10,,False,,NaN,500.0,0.15,1.000000,0.0,1.000000,0.5,0,0.00,0.0,0.0,NaN,0X,NaN,NaN,CR - Desserts,CR - Candy,2766.0,NaN,1k-3k,2023-09-20 15:56:41.170361-07:00


**Upload to Sheets**

In [69]:
# upload doots data to MBR US
# Uploading to Google Sheets test
print("Authenticating and opening spreadsheet...")
#gc = pygsheets.authorize(custom_credentials=my_credentials)
print("Opening Spreadsheet..")
#ss = gc.open_by_key("16xST2JWUrK2s-Kla6N82O8B9lHvIBUfJDcAMoxwnLNQ")

# Selecting location
#ws = ss.worksheet("title","D_gmv_mrr_opp_uscan")

# Uploading data
print("Clearing sheet ...")
#ws.clear(start="A", end="BC")
print("Sheet cleared")

print("Uploading data...")
#ws.set_dataframe(gmv_mrr_us_3, start= "A1",copy_head= True, extend= True, nan=  "")
print("Data uploaded!")

Authenticating and opening spreadsheet...
Opening Spreadsheet..
Clearing sheet ...
Sheet cleared
Uploading data...
Data uploaded!


## Latam

In [70]:
# Tuned latam gmv_mrr
orders_data_4.head(2)

,created_week,country,organization_id,name,seller_organization_id,seller_organization_name,facility_id,facility_sf_name,inventory_owned,distribution_type,city,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,Avg_Items_per_Order,total_gmv_local,lost_gmv,cancelled_gmv_local,OFOs,cities,facilities
0,2023-08-07,Brazil,893bb087-45ee-4436-9463-d8d3667b0d43,Patrícia Croce Doces - Pik n' Pak,893bb087-45ee-4436-9463-d8d3667b0d43,Patrícia Croce Doces - Pik n' Pak,3759ad21-4537-48ea-b1f1-48809e8a03ac,RIO - Botafogo,False,3PCR,Rio de Janeiro,85,5,0,5,79,2.152941,4576.19,0,242.9,2,1,1
1,2023-07-17,Brazil,67f3637f-fff2-4ac7-9ad7-78d976319994,Açaí RUDE | Rude Food,67f3637f-fff2-4ac7-9ad7-78d976319994,Açaí RUDE | Rude Food,a63559d1-93c6-4f60-b683-1e207ccda10a,SAO - Batista,False,3PCR,Sao Paulo,32,0,0,0,30,1.062500,698.40,0,0.0,2,1,1


In [71]:
print("Finished running!")

Finished running!


**Latam SF**

In [72]:
# Install Simplesalesforce
# !pip install simple-salesforce 
from simple_salesforce import Salesforce, SalesforceLogin, SFType
import json

In [73]:
# Import credentials
path = "/home/shared/joseph.jensen/Alan Data Pipelines/Credentials/sf_auth_JJ.json"

loginInfo = json.load(open(path))
session_id, instance = SalesforceLogin(username= loginInfo["username"],
                                                 password= loginInfo["password"],
                                                 security_token= loginInfo["security_token"],
                                                 domain= "login")

# Set SF connection
sf = Salesforce(instance= instance, session_id= session_id)

In [74]:
# Query category data from Salesforce
sf_query = """
SELECT 
Account.Org_Id_Otter__c,
Account_Name_Jira__c, 
account.Current_Customer__c, 
Opportunity_ID_18__c, 
Name, Facility__r.Name,  
Cuisine_Type__c,  
Id, 
Kitchen_City__c, 
Kitchen_Country__c, 
Is_Churned__c,
Churn_Date__c,
Processing_Fee__c, 
Monthly_Shelf_Fee_Dry_Storage__c,
Monthly_Shelf_Fee_Frozen_Storage__c,
Monthly_Shelf_Fee_Cold_Storage__c,
Closed_Won_Date__c,
Move_In_Date__c, 
Start_Date__c,
Loss_Reason__c, 
Specific_Lost_Reason__c, 
Lost_Explanation__c,
CR_Opp_Opted_out_Affiliate__c,
CR_Opp_Restrictions_Affiliate__c,
CR_Opp_Type_of_Processing_Fee__c,
account.CR_Account_Subcategory__c, 
account.CR_Acc_Social_Media_Presence_Numeric__c, 
account.CR_Type_of_Customer__c,
Storage_Amount_Dry_CU_FT__c, 
Storage_Amount_Frozen_CU_FT__c, 
Storage_Amount_Cold_CU_FT__c,
CR_Opp_OB_Status__c


FROM
Opportunity  

WHERE
RecordType.Name in ('Virtual', 'CloudRetail','virtual;cloudretail') AND (
Opp_Owner_Region__c in ('LATAM') OR Kitchen_Country__c IN ('Mexico', 'Colombia', 'Brazil'))
AND StageName = 'Closed Won' ORDER BY Closed_Won_Date__c DESC

"""


print("Querying data...")

# Query users to dataframe
sf_data = pd.DataFrame(sf.query_all(sf_query)['records'])
print("Data pulled!")

# Inspect 
sf_data.head()
print("Rows:", len(sf_data))

Querying data...
Data pulled!
Rows: 2833


In [75]:
# Inspect
sf_data.head()
print(len(sf_data))

2833


In [76]:
# Extract account details from accounts list in accounts column
account_name = [] # lista orgs id
for i in sf_data['Facility__r']:
    try:
        j=(i.get('Name'))
        #print(j)
        account_name.append(j)
    except:
        account_name.append(None)

# Append extracted lists to DF
sf_data['Facility__r']= account_name

In [77]:
# Rename new facility name field to match old name
sf_data.rename(columns = {"Facility__r":"Facility_Name__c"}, inplace = True)

In [78]:
sf_data.head()
print(len(sf_data))

2833


In [79]:
acct_query = """
SELECT
Account.LAM_TierScrape__c, 
Account.CSM__c,
Org_Id_Otter__c,
CR_Account_Category__c,
CR_Account_Subcategory__c,
CR_Account_Cuisine_Type__c,
CR_Type_of_Customer__c,
CR_Acc_Social_Media_Presence_Numeric__c,
id

FROM
Account
"""


print("Querying data...")

# Query users to dataframe
acct_query = pd.DataFrame(sf.query_all(acct_query)['records'])
print("Data pulled!")

# Inspect 
acct_query.head()
print("Rows:", len(acct_query))

Querying data...
Data pulled!
Rows: 65570


In [80]:
acct_query = acct_query[["Org_Id_Otter__c", "CSM__c", "CR_Account_Category__c","CR_Account_Subcategory__c", "CR_Account_Cuisine_Type__c", "LAM_TierScrape__c","CR_Type_of_Customer__c","CR_Acc_Social_Media_Presence_Numeric__c"]]
acct_query.head()

,Org_Id_Otter__c,CSM__c,CR_Account_Category__c,CR_Account_Subcategory__c,CR_Account_Cuisine_Type__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c
0,None,None,None,None,None,None,None,NaN
1,None,None,None,None,None,None,None,NaN
2,ab4eb478-b0bb-4fd5-8de9-8c3274b76911,None,None,None,CR - Baked Goods,Tier 3,None,114000.0
3,None,None,None,None,None,None,None,NaN
4,None,None,None,None,None,None,None,NaN


In [81]:
acct_query = acct_query.drop_duplicates(subset = "Org_Id_Otter__c")

In [82]:
print(len(acct_query))

1849


In [83]:
# Extract account details from accounts list in accounts column
org_ids = [] # lista orgs id
actual_cust =[] #lista cliente actual
has_churned = [] # lista has churned
for i in sf_data['Account']:
    try:
        j=(i.get('Org_Id_Otter__c'))
        #print(j)
        org_ids.append(j)
        k=(i.get('Current_Customer__c'))
        actual_cust.append(k)
    except:
        org_ids.append(None)
        actual_cust.append(None)

In [84]:
# Append extracted lists to DF
sf_data['sfdc_org_id']= org_ids
sf_data['current_customer']= actual_cust

In [85]:
# Fill NAs with bfill 
sf_data['org_id']= sf_data[["sfdc_org_id"]].bfill(axis=1).iloc[:, 0]

In [86]:
# Merge data
sf_data = sf_data.merge(acct_query, how = "left", left_on = ["org_id"], right_on = ["Org_Id_Otter__c"])

In [87]:
# Order by closed_won date desc.
sf_data["Closed_Won_Date__c"] = pd.to_datetime(sf_data["Closed_Won_Date__c"])
sf_data = sf_data.sort_values(by = "Closed_Won_Date__c", ascending = False).reset_index(drop = True)

# Inspect 
sf_data.head()

,attributes,Account,Account_Name_Jira__c,Opportunity_ID_18__c,Name,Facility_Name__c,Cuisine_Type__c,Id,Kitchen_City__c,Kitchen_Country__c,Is_Churned__c,Churn_Date__c,Processing_Fee__c,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,Closed_Won_Date__c,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,CR_Opp_OB_Status__c,sfdc_org_id,current_customer,org_id,Org_Id_Otter__c,CSM__c,CR_Account_Category__c,CR_Account_Subcategory__c,CR_Account_Cuisine_Type__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c
0,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Pólen Artesanal,0066T00001D8vapQAB,Pólen Artesanal,RIO - Botafogo,None,0066T00001D8vapQAB,Rio de Janeiro,Brazil,False,None,12.5,0.0,0.0,0.0,2023-09-15,None,2023-10-23,None,None,None,True,False,Percentage / Order,0.0,0.0,0.0,b - Documentation,None,False,None,None,None,None,None,None,None,None,NaN
1,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Maddea Chocolateria,0066T00001D8nTuQAJ,Maddea Chocolateria - SAO - Ferreira,SAO - Ferreira,None,0066T00001D8nTuQAJ,Sao Paulo,Brazil,False,None,20.0,0.0,0.0,0.0,2023-09-15,None,2023-10-09,None,None,None,True,False,Percentage / Order,4.0,0.0,0.0,a - Pré Kickoff,None,False,None,None,None,None,None,None,None,None,NaN
2,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Cookie Cami's,0066T00001D8UViQAN,Cookie Cami's,BHZ - Tancredo Neves,None,0066T00001D8UViQAN,Belo Horizonte,Brazil,False,None,7.0,430.0,0.0,0.0,2023-09-13,None,2023-10-16,None,None,None,True,False,Percentage / Order,4.0,0.0,0.0,b - Documentation,None,False,None,None,None,None,None,None,None,None,NaN
3,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Gadonas,0066T00001C0UaDQAV,Gadonas - Bog - Castellana,BOG - Castellana,None,0066T00001C0UaDQAV,Bogota,Colombia,False,None,20.0,0.0,0.0,0.0,2023-09-12,None,2023-10-12,None,None,None,True,False,Percentage / Order,8.0,0.0,0.0,None,30603c89-f999-4b87-ba73-3a743adc42fd,False,30603c89-f999-4b87-ba73-3a743adc42fd,30603c89-f999-4b87-ba73-3a743adc42fd,None,None,None,None,None,Local,976.0
4,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Apicela Gourmet,0066T00001D9Xh5QAF,Apicela Gourmet - BHZ - Tancredo Neves,BHZ - Tancredo Neves,None,0066T00001D9Xh5QAF,Belo Horizonte,Brazil,False,None,7.0,0.0,0.0,860.0,2023-09-11,None,2023-10-16,None,None,None,True,False,Percentage / Order,0.0,0.0,8.0,a - Pré Kickoff,3b9af84f-92f8-4ab4-8a0f-b1a47e28ad51,True,3b9af84f-92f8-4ab4-8a0f-b1a47e28ad51,3b9af84f-92f8-4ab4-8a0f-b1a47e28ad51,005f40000054nEXAAY,CR - Baked Goods,CR - Cakes,CR - Baked Goods,Tier 2,Local,6701.0


In [88]:
# Dropping duplicates
print("Rows:", len(sf_data))
sf_data = sf_data.drop_duplicates(subset= ["org_id", "Facility_Name__c"], keep = "first").reset_index(drop=True)

Rows: 2833


In [89]:
# Merging SF to SS data
combined_data = orders_data_4.merge(sf_data,how='left', left_on=['organization_id',"facility_sf_name"], right_on=['org_id', "Facility_Name__c"])

In [90]:
# Inspect length
print("Rows:", len(combined_data))

Rows: 8183


In [91]:
# Creating MRR local column
combined_data["MRR_local"] = combined_data["Monthly_Shelf_Fee_Dry_Storage__c"] + combined_data["Monthly_Shelf_Fee_Frozen_Storage__c"] + combined_data["Monthly_Shelf_Fee_Cold_Storage__c"]

In [92]:
# Inspect length
print("Combined data rows:", len(combined_data))

Combined data rows: 8183


In [93]:
# Connect to google sheets exchange rate file
import pygsheets
from google.oauth2 import service_account
from ds.utilities.secrets import secrets

In [94]:
# Connecting and authenticating with credentials
gc = pygsheets.authorize(custom_credentials=my_credentials)

# Importing worksheet
ex = gc.open_by_key('1h_lgOO2wf89txM2jX5ZXS64DizqewyPUReI_0-zP2M0')
exws = ex.worksheet('title','latam_monthly')
latam_monthly = exws.get_as_df(has_header=True,start="A1", end=None,empty_value='')

In [95]:
# Inspecting
latam_monthly.head()

,created_month,country,currency_code,exchange_rate
0,2019-01-01,Mexico,MXN,18.8
1,2019-02-01,Mexico,MXN,19.59
2,2019-03-01,Mexico,MXN,22.26
3,2019-04-01,Mexico,MXN,24.2
4,2019-05-01,Mexico,MXN,23.47


In [96]:
# Replacing NAs with 0 to enable conversion to numeric
latam_monthly['exchange_rate']= latam_monthly['exchange_rate'].replace('#N/A','0')
latam_monthly['exchange_rate']= pd.to_numeric(latam_monthly['exchange_rate'], errors='coerce')

In [97]:
combined_data

,created_week,country,organization_id,name,seller_organization_id,seller_organization_name,facility_id,facility_sf_name,inventory_owned,distribution_type,city,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,Avg_Items_per_Order,total_gmv_local,lost_gmv,cancelled_gmv_local,OFOs,cities,facilities,attributes,Account,Account_Name_Jira__c,Opportunity_ID_18__c,Name,Facility_Name__c,Cuisine_Type__c,Id,Kitchen_City__c,Kitchen_Country__c,Is_Churned__c,Churn_Date__c,Processing_Fee__c,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,Closed_Won_Date__c,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,CR_Opp_OB_Status__c,sfdc_org_id,current_customer,org_id,Org_Id_Otter__c,CSM__c,CR_Account_Category__c,CR_Account_Subcategory__c,CR_Account_Cuisine_Type__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c,MRR_local
0,2023-08-07,Brazil,893bb087-45ee-4436-9463-d8d3667b0d43,Patrícia Croce Doces - Pik n' Pak,893bb087-45ee-4436-9463-d8d3667b0d43,Patrícia Croce Doces - Pik n' Pak,3759ad21-4537-48ea-b1f1-48809e8a03ac,RIO - Botafogo,False,3PCR,Rio de Janeiro,85,5,0,5,79,2.152941,4576.19,0,242.90,2,1,1,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Patrícia Croce Doces,0066T00001A3mPGQAZ,Patrícia Croce Doces - RIO - Botafogo,RIO - Botafogo,CR - Candy/Sweets/Snacks,0066T00001A3mPGQAZ,Rio de Janeiro,Brazil,False,None,10.00,0.0,0.00,740.00,2022-11-30,2023-01-06,2023-01-05,None,None,None,True,False,None,0.00,0.0,8.0,g - Launched,893bb087-45ee-4436-9463-d8d3667b0d43,False,893bb087-45ee-4436-9463-d8d3667b0d43,893bb087-45ee-4436-9463-d8d3667b0d43,0056T000009cfwEQAQ,CR - Desserts,CR - Pies,CR - Candy/Sweets/Snacks,Tier 2,Local,1341.0,740.00
1,2023-07-17,Brazil,67f3637f-fff2-4ac7-9ad7-78d976319994,Açaí RUDE | Rude Food,67f3637f-fff2-4ac7-9ad7-78d976319994,Açaí RUDE | Rude Food,a63559d1-93c6-4f60-b683-1e207ccda10a,SAO - Batista,False,3PCR,Sao Paulo,32,0,0,0,30,1.062500,698.40,0,0.00,2,1,1,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Rude food Açaí,0066T00001A3XIpQAN,Rude food Açaí - SAO - Batista,SAO - Batista,"CR - Ice Cream, Popsicle, Cookie Dough & Gelato",0066T00001A3XIpQAN,Sao Paulo,Brazil,False,None,10.00,0.0,400.00,0.00,2022-12-20,2023-02-17,2023-02-16,None,None,None,False,False,Percentage / Order,0.00,4.0,0.0,g - Launched,67f3637f-fff2-4ac7-9ad7-78d976319994,True,67f3637f-fff2-4ac7-9ad7-78d976319994,67f3637f-fff2-4ac7-9ad7-78d976319994,0056T000009M15JQAS,CR - Frozen Desserts,CR - Ice Cream,"CR - Ice Cream, Popsicle, Cookie Dough & Gelato",Tier 2,None,1640.0,400.00
2,2023-06-26,Mexico,b556b267-8821-4162-a0cc-254fc70dafa0,Tere Cazola,b556b267-8821-4162-a0cc-254fc70dafa0,Tere Cazola,9101d4f9-828f-45c5-b8b6-16986e76b374,MEX - Pedregal,False,3PCR,Mexico City,242,8,4,12,239,1.785124,72497.79,0,3107.55,1,1,1,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Tere Cazola,0065G00000j4xZXQAY,TERE CAZOLA PEDREGAL - KITCHEN,MEX - Pedregal,CR - Baked Goods,0065G00000j4xZXQAY,Mexico City,Mexico,False,None,3.51,15812.5,0.00,0.00,2021-11-19,2021-10-06,2021-10-06,None,None,None,False,False,None,79.06,0.0,0.0,Completed,b556b267-8821-4162-a0cc-254fc70dafa0,True,b556b267-8821-4162-a0cc-254fc70dafa0,b556b267-8821-4162-a0cc-254fc70dafa0,005f4000006BISzAAO,CR - Baked Goods,CR - Cakes,CR - Baked Goods,Tier 1,None,47800.0,15812.50
3,2023-07-10,Brazil,75a23483-f8f3-471b-85f9-6467f68ce8a0,Caramelle Lu,75a23483-f8f3-471b-85f9-6467f68ce8a0,Caramelle Lu,f7a65e9f-3342-48d2-a0fb-e9e4633c4bf6,RIO - Tijuca,False,3PCR,Rio de Janeiro,34,0,0,0,34,3.235294,1396.20,0,0.00,1,1,1,"{'type': 'Opportunity', 

In [98]:
combined_data['created_month']=pd.to_datetime(combined_data['created_week']).dt.to_period('M').dt.start_time

In [99]:
combined_data['created_month']

0      2023-08-01
1      2023-07-01
2      2023-06-01
3      2023-07-01
4      2023-06-01
          ...    
8178   2023-05-01
8179   2023-06-01
8180   2023-07-01
8181   2023-09-01
8182   2023-06-01
Name: created_month, Length: 8183, dtype: datetime64[ns]

In [100]:
latam_monthly['created_month']=pd.to_datetime(latam_monthly['created_month'])

In [101]:
# Merging FX rates to combined data
combined_data_fx = combined_data.merge(latam_monthly, how = "left", on = ['created_month','country'])

In [102]:
combined_data_fx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8183 entries, 0 to 8182
Data columns (total 67 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   created_week                             8183 non-null   object        
 1   country                                  8183 non-null   object        
 2   organization_id                          8183 non-null   object        
 3   name                                     8183 non-null   object        
 4   seller_organization_id                   8183 non-null   object        
 5   seller_organization_name                 8183 non-null   object        
 6   facility_id                              8183 non-null   object        
 7   facility_sf_name                         8183 non-null   object        
 8   inventory_owned                          8183 non-null   bool          
 9   distribution_type                        

In [103]:
combined_data_fx["total_orders"] = pd.to_numeric(combined_data_fx["total_orders"])
combined_data_fx["cancelled_orders"] = pd.to_numeric(combined_data_fx["cancelled_orders"])
combined_data_fx["issue_orders"] = pd.to_numeric(combined_data_fx["issue_orders"])
combined_data_fx["ODR_orders"] = pd.to_numeric(combined_data_fx["ODR_orders"])
combined_data_fx["scanned_orders"] = pd.to_numeric(combined_data_fx["scanned_orders"])
combined_data_fx["total_gmv_local"] = pd.to_numeric(combined_data_fx["total_gmv_local"])
combined_data_fx["OFOs"] = pd.to_numeric(combined_data_fx["OFOs"])
combined_data_fx["cities"] = pd.to_numeric(combined_data_fx["cities"])
combined_data_fx["facilities"] = pd.to_numeric(combined_data_fx["facilities"])

In [104]:
# Calculating GMV, MRR and Health in USD
combined_data_fx["mrr_usd"] = combined_data_fx["MRR_local"] / combined_data_fx["exchange_rate"]
combined_data_fx["total_gmv_usd"] = combined_data_fx["total_gmv_local"] / combined_data_fx["exchange_rate"]
combined_data_fx["cancelled_gmv_usd"] = combined_data_fx["cancelled_gmv_local"] / combined_data_fx["exchange_rate"]
combined_data_fx["succesful_gmv_usd"] = combined_data_fx["total_gmv_usd"] - combined_data_fx["cancelled_gmv_usd"]
combined_data_fx["succesful_orders"] = combined_data_fx["total_orders"] - combined_data_fx["cancelled_orders"]

In [105]:
# Calculate total CU FT per opps
combined_data_fx["total_cu_ft"] = combined_data_fx["Storage_Amount_Dry_CU_FT__c"] + combined_data_fx["Storage_Amount_Frozen_CU_FT__c"] + combined_data_fx["Storage_Amount_Cold_CU_FT__c"]

In [106]:
# Examining data types
combined_data_fx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8183 entries, 0 to 8182
Data columns (total 73 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   created_week                             8183 non-null   object        
 1   country                                  8183 non-null   object        
 2   organization_id                          8183 non-null   object        
 3   name                                     8183 non-null   object        
 4   seller_organization_id                   8183 non-null   object        
 5   seller_organization_name                 8183 non-null   object        
 6   facility_id                              8183 non-null   object        
 7   facility_sf_name                         8183 non-null   object        
 8   inventory_owned                          8183 non-null   bool          
 9   distribution_type                        

**Calculating operations handling income**

- average processing fee % * succesful gmv

In [107]:
# Calculating income in local currency and USD
combined_data_fx["processing_fee_pct"] = combined_data_fx["Processing_Fee__c"] / 100
combined_data_fx["succesful_gmv_local"] = combined_data_fx["total_gmv_local"] - combined_data_fx["cancelled_gmv_local"]
combined_data_fx["processing_income_local"] = combined_data_fx["succesful_gmv_local"] * combined_data_fx["processing_fee_pct"]
combined_data_fx["processing_income_usd"] = combined_data_fx["succesful_gmv_usd"] * combined_data_fx["processing_fee_pct"]

In [108]:
# Create a concatenated field for name + facililty
combined_data_fx["name_facility"] = combined_data_fx["name"] + "-" + combined_data_fx["facility_sf_name"]

In [109]:
combined_data_fx.head(2)

,created_week,country,organization_id,name,seller_organization_id,seller_organization_name,facility_id,facility_sf_name,inventory_owned,distribution_type,city,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,Avg_Items_per_Order,total_gmv_local,lost_gmv,cancelled_gmv_local,OFOs,cities,facilities,attributes,Account,Account_Name_Jira__c,Opportunity_ID_18__c,Name,Facility_Name__c,Cuisine_Type__c,Id,Kitchen_City__c,Kitchen_Country__c,Is_Churned__c,Churn_Date__c,Processing_Fee__c,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,Closed_Won_Date__c,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,CR_Opp_OB_Status__c,sfdc_org_id,current_customer,org_id,Org_Id_Otter__c,CSM__c,CR_Account_Category__c,CR_Account_Subcategory__c,CR_Account_Cuisine_Type__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c,MRR_local,created_month,currency_code,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,succesful_gmv_usd,succesful_orders,total_cu_ft,processing_fee_pct,succesful_gmv_local,processing_income_local,processing_income_usd,name_facility
0,2023-08-07,Brazil,893bb087-45ee-4436-9463-d8d3667b0d43,Patrícia Croce Doces - Pik n' Pak,893bb087-45ee-4436-9463-d8d3667b0d43,Patrícia Croce Doces - Pik n' Pak,3759ad21-4537-48ea-b1f1-48809e8a03ac,RIO - Botafogo,False,3PCR,Rio de Janeiro,85,5,0,5,79,2.152941,4576.19,0,242.9,2,1,1,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Patrícia Croce Doces,0066T00001A3mPGQAZ,Patrícia Croce Doces - RIO - Botafogo,RIO - Botafogo,CR - Candy/Sweets/Snacks,0066T00001A3mPGQAZ,Rio de Janeiro,Brazil,False,None,10.0,0.0,0.0,740.0,2022-11-30,2023-01-06,2023-01-05,None,None,None,True,False,None,0.0,0.0,8.0,g - Launched,893bb087-45ee-4436-9463-d8d3667b0d43,False,893bb087-45ee-4436-9463-d8d3667b0d43,893bb087-45ee-4436-9463-d8d3667b0d43,0056T000009cfwEQAQ,CR - Desserts,CR - Pies,CR - Candy/Sweets/Snacks,Tier 2,Local,1341.0,740.0,2023-08-01,BRL,4.9,151.020408,933.916327,49.571429,884.344898,80,8.0,0.1,4333.29,433.329,88.43449,Patrícia Croce Doces - Pik n' Pak-RIO - Botafogo
1,2023-07-17,Brazil,67f3637f-fff2-4ac7-9ad7-78d976319994,Açaí RUDE | Rude Food,67f3637f-fff2-4ac7-9ad7-78d976319994,Açaí RUDE | Rude Food,a63559d1-93c6-4f60-b683-1e207ccda10a,SAO - Batista,False,3PCR,Sao Paulo,32,0,0,0,30,1.062500,698.40,0,0.0,2,1,1,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Rude food Açaí,0066T00001A3XIpQAN,Rude food Açaí - SAO - Batista,SAO - Batista,"CR - Ice Cream, Popsicle, Cookie Dough & Gelato",0066T00001A3XIpQAN,Sao Paulo,Brazil,False,None,10.0,0.0,400.0,0.0,2022-12-20,2023-02-17,2023-02-16,None,None,None,False,False,Percentage / Order,0.0,4.0,0.0,g - Launched,67f3637f-fff2-4ac7-9ad7-78d976319994,True,67f3637f-fff2-4ac7-9ad7-78d976319994,67f3637f-fff2-4ac7-9ad7-78d976319994,0056T000009M15JQAS,CR - Frozen Desserts,CR - Ice Cream,"CR - Ice Cream, Popsicle, Cookie Dough & Gelato",Tier 2,None,1640.0,400.0,2023-07-01,BRL,4.8,83.333333,145.500000,0.000000,145.500000,32,4.0,0.1,698.40,69.840,14.55000,Açaí RUDE | Rude Food-SAO - Batista


In [110]:
# Calculate org_health = gmv / mrr with local currency 
combined_data_fx["store_gmv_mrr"] = combined_data_fx["succesful_gmv_local"] / ((combined_data_fx["MRR_local"]/30)*7)

# Create org_health categories
health_cats = []
for i in combined_data_fx['store_gmv_mrr']:
    if i>=15:
        j='15x or more'
    elif i>=10:
        j='10x to 14.9x'
    elif i>=5:
        j='5x to 9.9x'
    elif i>=4:
        j='4x'
    elif i>=3:
        j='3x'
    elif i>=2:
        j='2x'
    elif i>=1:
        j='1x'
    else:
        j='0X'
    health_cats.append(j)

# Append categories
combined_data_fx["gmv_mrr_cat"] = health_cats

# Inspect
combined_data_fx.sample(3)

,created_week,country,organization_id,name,seller_organization_id,seller_organization_name,facility_id,facility_sf_name,inventory_owned,distribution_type,city,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,Avg_Items_per_Order,total_gmv_local,lost_gmv,cancelled_gmv_local,OFOs,cities,facilities,attributes,Account,Account_Name_Jira__c,Opportunity_ID_18__c,Name,Facility_Name__c,Cuisine_Type__c,Id,Kitchen_City__c,Kitchen_Country__c,Is_Churned__c,Churn_Date__c,Processing_Fee__c,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,Closed_Won_Date__c,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,CR_Opp_OB_Status__c,sfdc_org_id,current_customer,org_id,Org_Id_Otter__c,CSM__c,CR_Account_Category__c,CR_Account_Subcategory__c,CR_Account_Cuisine_Type__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c,MRR_local,created_month,currency_code,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,succesful_gmv_usd,succesful_orders,total_cu_ft,processing_fee_pct,succesful_gmv_local,processing_income_local,processing_income_usd,name_facility,store_gmv_mrr,gmv_mrr_cat
2521,2023-08-07,Mexico,5610229c-c4ce-4d72-94b3-4dc903b873d4,Antonoff,5610229c-c4ce-4d72-94b3-4dc903b873d4,Antonoff,da27f06f-01fd-4106-88d9-b3fd41446515,MEX - Lindavista,False,3PCR,Mexico City,4,0,0,0,4,1.250000,1330.00,0,0.0,1,1,1,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Antonoff,0066T00001AtOkdQAF,Antonoff - MEX - Lindavista,MEX - Lindavista,None,0066T00001AtOkdQAF,Mexico City,Mexico,False,None,8.0,1500.0,0.0,2000.0,2023-03-02,2023-04-03,2023-04-03,None,None,None,False,False,Monthly Fixed Fee,4.0,0.0,12.0,Completed,5610229c-c4ce-4d72-94b3-4dc903b873d4,False,5610229c-c4ce-4d72-94b3-4dc903b873d4,5610229c-c4ce-4d72-94b3-4dc903b873d4,0055G000006QIsHQAW,CR - Baked Goods,CR - Cakes,CR - Baked Goods,Tier 1,Mid-Market,5682.0,3500.0,2023-08-01,MXN,16.98,206.124853,78.327444,0.0,78.327444,4,16.0,0.08,1330.00,106.4000,6.266196,Antonoff-MEX - Lindavista,1.628571,1x
3682,2023-06-05,Mexico,294ccb86-c300-4b29-827b-45bd77621bd3,La Michoacana - CR,294ccb86-c300-4b29-827b-45bd77621bd3,La Michoacana - CR,864e0a3c-ecba-4305-b3ff-f22edd58c902,MEX - Polanco,False,3PCR,Mexico City,20,0,1,1,20,5.800000,5545.00,0,0.0,2,1,1,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Grupo Anwe S.A. de C.V.,0066T00001BSd1MQAT,Grupo Anwe S.A. de C.V. - MEX - Polanco,MEX - Polanco,None,0066T00001BSd1MQAT,Mexico City,Mexico,False,None,8.0,0.0,4500.0,0.0,2023-05-09,2023-04-21,2023-06-10,None,None,None,True,False,Monthly Fixed Fee,0.0,12.0,0.0,Completed,294ccb86-c300-4b29-827b-45bd77621bd3,False,294ccb86-c300-4b29-827b-45bd77621bd3,294ccb86-c300-4b29-827b-45bd77621bd3,0055G0000080B5LQAU,CR - Frozen Desserts,CR - Popsicle,None,Tier 3,Local,953.0,4500.0,2023-06-01,MXN,17.26,260.718424,321.263036,0.0,321.263036,20,12.0,0.08,5545.00,443.6000,25.701043,La Michoacana - CR-MEX - Polanco,5.280952,5x to 9.9x
1865,2023-09-18,Brazil,37b177cd-20b9-44b8-a422-31fee16fbcc8,Alma Vinhos,37b177cd-20b9-44b8-a422-31fee16fbcc8,Alma Vinhos,4326199f-628a-4b00-a6e6-054fb5182c64,SAO - Guararapes (Brooklin),False,3PCR,Sao Paulo,3,0,0,0,2,1.666667,209.97,0,0.0,1,1,1,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Alma Vinhos,0065G00000oudllQAA,Alma Vinhos - SAO - Guararapes (Brooklin),SAO - Guararapes (Brooklin),CR - Alcoholic Beverages,0065G00000oudllQAA,Sao Paulo,Brazil,False,None,5.0,939.3,0.0,0.0,2022-06-07,2022-08-12,2022-08-15,None,None,None,False,False,None,8.0,0.0,0.0,g - Launched,37b177cd-20b9-44b8-a422-31fee16fbcc8,False,37b177cd-20b9-44b8-a422-31

In [111]:
combined_data_fx[combined_data_fx['name']=='Vape Mate'].head()

,created_week,country,organization_id,name,seller_organization_id,seller_organization_name,facility_id,facility_sf_name,inventory_owned,distribution_type,city,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,Avg_Items_per_Order,total_gmv_local,lost_gmv,cancelled_gmv_local,OFOs,cities,facilities,attributes,Account,Account_Name_Jira__c,Opportunity_ID_18__c,Name,Facility_Name__c,Cuisine_Type__c,Id,Kitchen_City__c,Kitchen_Country__c,Is_Churned__c,Churn_Date__c,Processing_Fee__c,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,Closed_Won_Date__c,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,CR_Opp_OB_Status__c,sfdc_org_id,current_customer,org_id,Org_Id_Otter__c,CSM__c,CR_Account_Category__c,CR_Account_Subcategory__c,CR_Account_Cuisine_Type__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c,MRR_local,created_month,currency_code,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,succesful_gmv_usd,succesful_orders,total_cu_ft,processing_fee_pct,succesful_gmv_local,processing_income_local,processing_income_usd,name_facility,store_gmv_mrr,gmv_mrr_cat


In [112]:
# Renaming columns
combined_data_fx = combined_data_fx.rename(columns = {"facility_sf_name":"facility_name",
                                                      "Account_Name_Jira__c":"acct_name_jira",
                                                      "Kitchen_City__c":"kitchen_city",
                                                     "Kitchen_Country__c":"kitchen_country",
                                                     "Is_Churned__c":"is_churned",
                                                     "Churn_Date__c":"churn_date",
                                                     "Processing_Fee__c":"processing_fee_int",
                                                     "Closed_Won_Date__c":"closed_won_date"})

#Inspect
combined_data_fx.head()

,created_week,country,organization_id,name,seller_organization_id,seller_organization_name,facility_id,facility_name,inventory_owned,distribution_type,city,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,Avg_Items_per_Order,total_gmv_local,lost_gmv,cancelled_gmv_local,OFOs,cities,facilities,attributes,Account,acct_name_jira,Opportunity_ID_18__c,Name,Facility_Name__c,Cuisine_Type__c,Id,kitchen_city,kitchen_country,is_churned,churn_date,processing_fee_int,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,closed_won_date,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,CR_Opp_OB_Status__c,sfdc_org_id,current_customer,org_id,Org_Id_Otter__c,CSM__c,CR_Account_Category__c,CR_Account_Subcategory__c,CR_Account_Cuisine_Type__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c,MRR_local,created_month,currency_code,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,succesful_gmv_usd,succesful_orders,total_cu_ft,processing_fee_pct,succesful_gmv_local,processing_income_local,processing_income_usd,name_facility,store_gmv_mrr,gmv_mrr_cat
0,2023-08-07,Brazil,893bb087-45ee-4436-9463-d8d3667b0d43,Patrícia Croce Doces - Pik n' Pak,893bb087-45ee-4436-9463-d8d3667b0d43,Patrícia Croce Doces - Pik n' Pak,3759ad21-4537-48ea-b1f1-48809e8a03ac,RIO - Botafogo,False,3PCR,Rio de Janeiro,85,5,0,5,79,2.152941,4576.19,0,242.90,2,1,1,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Patrícia Croce Doces,0066T00001A3mPGQAZ,Patrícia Croce Doces - RIO - Botafogo,RIO - Botafogo,CR - Candy/Sweets/Snacks,0066T00001A3mPGQAZ,Rio de Janeiro,Brazil,False,None,10.00,0.0,0.0,740.0,2022-11-30,2023-01-06,2023-01-05,None,None,None,True,False,None,0.00,0.0,8.0,g - Launched,893bb087-45ee-4436-9463-d8d3667b0d43,False,893bb087-45ee-4436-9463-d8d3667b0d43,893bb087-45ee-4436-9463-d8d3667b0d43,0056T000009cfwEQAQ,CR - Desserts,CR - Pies,CR - Candy/Sweets/Snacks,Tier 2,Local,1341.0,740.0,2023-08-01,BRL,4.90,151.020408,933.916327,49.571429,884.344898,80,8.00,0.1000,4333.29,433.329000,88.434490,Patrícia Croce Doces - Pik n' Pak-RIO - Botafogo,25.096274,15x or more
1,2023-07-17,Brazil,67f3637f-fff2-4ac7-9ad7-78d976319994,Açaí RUDE | Rude Food,67f3637f-fff2-4ac7-9ad7-78d976319994,Açaí RUDE | Rude Food,a63559d1-93c6-4f60-b683-1e207ccda10a,SAO - Batista,False,3PCR,Sao Paulo,32,0,0,0,30,1.062500,698.40,0,0.00,2,1,1,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Rude food Açaí,0066T00001A3XIpQAN,Rude food Açaí - SAO - Batista,SAO - Batista,"CR - Ice Cream, Popsicle, Cookie Dough & Gelato",0066T00001A3XIpQAN,Sao Paulo,Brazil,False,None,10.00,0.0,400.0,0.0,2022-12-20,2023-02-17,2023-02-16,None,None,None,False,False,Percentage / Order,0.00,4.0,0.0,g - Launched,67f3637f-fff2-4ac7-9ad7-78d976319994,True,67f3637f-fff2-4ac7-9ad7-78d976319994,67f3637f-fff2-4ac7-9ad7-78d976319994,0056T000009M15JQAS,CR - Frozen Desserts,CR - Ice Cream,"CR - Ice Cream, Popsicle, Cookie Dough & Gelato",Tier 2,None,1640.0,400.0,2023-07-01,BRL,4.80,83.333333,145.500000,0.000000,145.500000,32,4.00,0.1000,698.40,69.840000,14.550000,Açaí RUDE | Rude Food-SAO - Batista,7.482857,5x to 9.9x
2,2023-06-26,Mexico,b556b267-8821-4162-a0cc-254fc70dafa0,Tere Cazola,b556b267-8821-4162-a0cc-254fc70dafa0,Tere Cazola,9101d4f9-828f-45c5-b8b6-16986e76b374,MEX - Pedregal,False,3PCR,Mexico City,242,8,4,12,239,1.785124,72497.79,0,3107.55,1,1,1,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Tere Cazola,0065G00000j4xZXQAY,TERE CAZOLA PEDREGAL - KITCHEN,MEX - Pedregal,CR - Baked Goods,0065G00000j4xZXQAY,Mexico City,Mexico,False,None,3.51,15812.5,0.0,0.0,2

In [113]:
# Filling NaNs with 0 on numeric columns
na_cols_gmv = ["MRR_local", "mrr_usd", "succesful_gmv_usd", "store_gmv_mrr"]

# Replacing NaNs
combined_data_fx[na_cols_gmv] = combined_data_fx[na_cols_gmv].fillna(0)

In [114]:
# Replacing inf values
combined_data_fx.replace([np.inf, -np.inf], 0, inplace=True)

In [115]:
combined_data_fx.head()

,created_week,country,organization_id,name,seller_organization_id,seller_organization_name,facility_id,facility_name,inventory_owned,distribution_type,city,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,Avg_Items_per_Order,total_gmv_local,lost_gmv,cancelled_gmv_local,OFOs,cities,facilities,attributes,Account,acct_name_jira,Opportunity_ID_18__c,Name,Facility_Name__c,Cuisine_Type__c,Id,kitchen_city,kitchen_country,is_churned,churn_date,processing_fee_int,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,closed_won_date,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,CR_Opp_OB_Status__c,sfdc_org_id,current_customer,org_id,Org_Id_Otter__c,CSM__c,CR_Account_Category__c,CR_Account_Subcategory__c,CR_Account_Cuisine_Type__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c,MRR_local,created_month,currency_code,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,succesful_gmv_usd,succesful_orders,total_cu_ft,processing_fee_pct,succesful_gmv_local,processing_income_local,processing_income_usd,name_facility,store_gmv_mrr,gmv_mrr_cat
0,2023-08-07,Brazil,893bb087-45ee-4436-9463-d8d3667b0d43,Patrícia Croce Doces - Pik n' Pak,893bb087-45ee-4436-9463-d8d3667b0d43,Patrícia Croce Doces - Pik n' Pak,3759ad21-4537-48ea-b1f1-48809e8a03ac,RIO - Botafogo,False,3PCR,Rio de Janeiro,85,5,0,5,79,2.152941,4576.19,0,242.90,2,1,1,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Patrícia Croce Doces,0066T00001A3mPGQAZ,Patrícia Croce Doces - RIO - Botafogo,RIO - Botafogo,CR - Candy/Sweets/Snacks,0066T00001A3mPGQAZ,Rio de Janeiro,Brazil,False,None,10.00,0.0,0.0,740.0,2022-11-30,2023-01-06,2023-01-05,NaN,None,None,True,False,None,0.00,0.0,8.0,g - Launched,893bb087-45ee-4436-9463-d8d3667b0d43,False,893bb087-45ee-4436-9463-d8d3667b0d43,893bb087-45ee-4436-9463-d8d3667b0d43,0056T000009cfwEQAQ,CR - Desserts,CR - Pies,CR - Candy/Sweets/Snacks,Tier 2,Local,1341.0,740.0,2023-08-01,BRL,4.90,151.020408,933.916327,49.571429,884.344898,80,8.00,0.1000,4333.29,433.329000,88.434490,Patrícia Croce Doces - Pik n' Pak-RIO - Botafogo,25.096274,15x or more
1,2023-07-17,Brazil,67f3637f-fff2-4ac7-9ad7-78d976319994,Açaí RUDE | Rude Food,67f3637f-fff2-4ac7-9ad7-78d976319994,Açaí RUDE | Rude Food,a63559d1-93c6-4f60-b683-1e207ccda10a,SAO - Batista,False,3PCR,Sao Paulo,32,0,0,0,30,1.062500,698.40,0,0.00,2,1,1,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Rude food Açaí,0066T00001A3XIpQAN,Rude food Açaí - SAO - Batista,SAO - Batista,"CR - Ice Cream, Popsicle, Cookie Dough & Gelato",0066T00001A3XIpQAN,Sao Paulo,Brazil,False,None,10.00,0.0,400.0,0.0,2022-12-20,2023-02-17,2023-02-16,NaN,None,None,False,False,Percentage / Order,0.00,4.0,0.0,g - Launched,67f3637f-fff2-4ac7-9ad7-78d976319994,True,67f3637f-fff2-4ac7-9ad7-78d976319994,67f3637f-fff2-4ac7-9ad7-78d976319994,0056T000009M15JQAS,CR - Frozen Desserts,CR - Ice Cream,"CR - Ice Cream, Popsicle, Cookie Dough & Gelato",Tier 2,None,1640.0,400.0,2023-07-01,BRL,4.80,83.333333,145.500000,0.000000,145.500000,32,4.00,0.1000,698.40,69.840000,14.550000,Açaí RUDE | Rude Food-SAO - Batista,7.482857,5x to 9.9x
2,2023-06-26,Mexico,b556b267-8821-4162-a0cc-254fc70dafa0,Tere Cazola,b556b267-8821-4162-a0cc-254fc70dafa0,Tere Cazola,9101d4f9-828f-45c5-b8b6-16986e76b374,MEX - Pedregal,False,3PCR,Mexico City,242,8,4,12,239,1.785124,72497.79,0,3107.55,1,1,1,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Tere Cazola,0065G00000j4xZXQAY,TERE CAZOLA PEDREGAL - KITCHEN,MEX - Pedregal,CR - Baked Goods,0065G00000j4xZXQAY,Mexico City,Mexico,False,None,3.51,15812.5,0.0,0.0,202

In [116]:
# Drop unused columns
combined_data_fx = combined_data_fx.drop(columns = ["Name"])

# Inspect
combined_data_fx.head()

,created_week,country,organization_id,name,seller_organization_id,seller_organization_name,facility_id,facility_name,inventory_owned,distribution_type,city,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,Avg_Items_per_Order,total_gmv_local,lost_gmv,cancelled_gmv_local,OFOs,cities,facilities,attributes,Account,acct_name_jira,Opportunity_ID_18__c,Facility_Name__c,Cuisine_Type__c,Id,kitchen_city,kitchen_country,is_churned,churn_date,processing_fee_int,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,closed_won_date,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,CR_Opp_OB_Status__c,sfdc_org_id,current_customer,org_id,Org_Id_Otter__c,CSM__c,CR_Account_Category__c,CR_Account_Subcategory__c,CR_Account_Cuisine_Type__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c,MRR_local,created_month,currency_code,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,succesful_gmv_usd,succesful_orders,total_cu_ft,processing_fee_pct,succesful_gmv_local,processing_income_local,processing_income_usd,name_facility,store_gmv_mrr,gmv_mrr_cat
0,2023-08-07,Brazil,893bb087-45ee-4436-9463-d8d3667b0d43,Patrícia Croce Doces - Pik n' Pak,893bb087-45ee-4436-9463-d8d3667b0d43,Patrícia Croce Doces - Pik n' Pak,3759ad21-4537-48ea-b1f1-48809e8a03ac,RIO - Botafogo,False,3PCR,Rio de Janeiro,85,5,0,5,79,2.152941,4576.19,0,242.90,2,1,1,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Patrícia Croce Doces,0066T00001A3mPGQAZ,RIO - Botafogo,CR - Candy/Sweets/Snacks,0066T00001A3mPGQAZ,Rio de Janeiro,Brazil,False,None,10.00,0.0,0.0,740.0,2022-11-30,2023-01-06,2023-01-05,NaN,None,None,True,False,None,0.00,0.0,8.0,g - Launched,893bb087-45ee-4436-9463-d8d3667b0d43,False,893bb087-45ee-4436-9463-d8d3667b0d43,893bb087-45ee-4436-9463-d8d3667b0d43,0056T000009cfwEQAQ,CR - Desserts,CR - Pies,CR - Candy/Sweets/Snacks,Tier 2,Local,1341.0,740.0,2023-08-01,BRL,4.90,151.020408,933.916327,49.571429,884.344898,80,8.00,0.1000,4333.29,433.329000,88.434490,Patrícia Croce Doces - Pik n' Pak-RIO - Botafogo,25.096274,15x or more
1,2023-07-17,Brazil,67f3637f-fff2-4ac7-9ad7-78d976319994,Açaí RUDE | Rude Food,67f3637f-fff2-4ac7-9ad7-78d976319994,Açaí RUDE | Rude Food,a63559d1-93c6-4f60-b683-1e207ccda10a,SAO - Batista,False,3PCR,Sao Paulo,32,0,0,0,30,1.062500,698.40,0,0.00,2,1,1,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Rude food Açaí,0066T00001A3XIpQAN,SAO - Batista,"CR - Ice Cream, Popsicle, Cookie Dough & Gelato",0066T00001A3XIpQAN,Sao Paulo,Brazil,False,None,10.00,0.0,400.0,0.0,2022-12-20,2023-02-17,2023-02-16,NaN,None,None,False,False,Percentage / Order,0.00,4.0,0.0,g - Launched,67f3637f-fff2-4ac7-9ad7-78d976319994,True,67f3637f-fff2-4ac7-9ad7-78d976319994,67f3637f-fff2-4ac7-9ad7-78d976319994,0056T000009M15JQAS,CR - Frozen Desserts,CR - Ice Cream,"CR - Ice Cream, Popsicle, Cookie Dough & Gelato",Tier 2,None,1640.0,400.0,2023-07-01,BRL,4.80,83.333333,145.500000,0.000000,145.500000,32,4.00,0.1000,698.40,69.840000,14.550000,Açaí RUDE | Rude Food-SAO - Batista,7.482857,5x to 9.9x
2,2023-06-26,Mexico,b556b267-8821-4162-a0cc-254fc70dafa0,Tere Cazola,b556b267-8821-4162-a0cc-254fc70dafa0,Tere Cazola,9101d4f9-828f-45c5-b8b6-16986e76b374,MEX - Pedregal,False,3PCR,Mexico City,242,8,4,12,239,1.785124,72497.79,0,3107.55,1,1,1,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Tere Cazola,0065G00000j4xZXQAY,MEX - Pedregal,CR - Baked Goods,0065G00000j4xZXQAY,Mexico City,Mexico,False,None,3.51,15812.5,0.0,0.0,2021-11-19,2021-10-06,2021-10-06,NaN,None,None,False,False,None,79.06,0.0,0.0,Completed,b556b267-8821-4162-a

In [117]:
combined_data_fx["total_gmv_usd"].sum()

2850431.0758653907

In [118]:
# Converting corresponding columns to numeric and datetime
combined_data_fx["created_week"] = pd.to_datetime(combined_data_fx["created_week"])
combined_data_fx["total_orders"] = pd.to_numeric(combined_data_fx["total_orders"])
combined_data_fx["cancelled_orders"] = pd.to_numeric(combined_data_fx["cancelled_orders"])
combined_data_fx["issue_orders"] = pd.to_numeric(combined_data_fx["issue_orders"])
combined_data_fx["ODR_orders"] = pd.to_numeric(combined_data_fx["ODR_orders"])
combined_data_fx["scanned_orders"] = pd.to_numeric(combined_data_fx["scanned_orders"])
combined_data_fx["churn_date"] = pd.to_datetime(combined_data_fx["churn_date"])

In [119]:
combined_data_fx["pct_cancelled"] = combined_data_fx["cancelled_orders"] / combined_data_fx["total_orders"]
combined_data_fx["pct_issues"] = combined_data_fx["issue_orders"] / combined_data_fx["total_orders"]
combined_data_fx["pct_odr"] = combined_data_fx["ODR_orders"] /  combined_data_fx["total_orders"]
combined_data_fx["pct_scanned"] = combined_data_fx["scanned_orders"] / combined_data_fx["total_orders"]
combined_data_fx["lost_gmv_usd"] = 0

In [120]:
combined_data_fx=combined_data_fx.drop(columns=['created_month'])

In [121]:
combined_data_fx

,created_week,country,organization_id,name,seller_organization_id,seller_organization_name,facility_id,facility_name,inventory_owned,distribution_type,city,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,Avg_Items_per_Order,total_gmv_local,lost_gmv,cancelled_gmv_local,OFOs,cities,facilities,attributes,Account,acct_name_jira,Opportunity_ID_18__c,Facility_Name__c,Cuisine_Type__c,Id,kitchen_city,kitchen_country,is_churned,churn_date,processing_fee_int,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,closed_won_date,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,CR_Opp_OB_Status__c,sfdc_org_id,current_customer,org_id,Org_Id_Otter__c,CSM__c,CR_Account_Category__c,CR_Account_Subcategory__c,CR_Account_Cuisine_Type__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c,MRR_local,currency_code,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,succesful_gmv_usd,succesful_orders,total_cu_ft,processing_fee_pct,succesful_gmv_local,processing_income_local,processing_income_usd,name_facility,store_gmv_mrr,gmv_mrr_cat,pct_cancelled,pct_issues,pct_odr,pct_scanned,lost_gmv_usd
0,2023-08-07,Brazil,893bb087-45ee-4436-9463-d8d3667b0d43,Patrícia Croce Doces - Pik n' Pak,893bb087-45ee-4436-9463-d8d3667b0d43,Patrícia Croce Doces - Pik n' Pak,3759ad21-4537-48ea-b1f1-48809e8a03ac,RIO - Botafogo,False,3PCR,Rio de Janeiro,85,5,0,5,79,2.152941,4576.19,0,242.90,2,1,1,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Patrícia Croce Doces,0066T00001A3mPGQAZ,RIO - Botafogo,CR - Candy/Sweets/Snacks,0066T00001A3mPGQAZ,Rio de Janeiro,Brazil,False,NaT,10.00,0.0,0.00,740.00,2022-11-30,2023-01-06,2023-01-05,NaN,None,None,True,False,None,0.00,0.0,8.0,g - Launched,893bb087-45ee-4436-9463-d8d3667b0d43,False,893bb087-45ee-4436-9463-d8d3667b0d43,893bb087-45ee-4436-9463-d8d3667b0d43,0056T000009cfwEQAQ,CR - Desserts,CR - Pies,CR - Candy/Sweets/Snacks,Tier 2,Local,1341.0,740.00,BRL,4.90,151.020408,933.916327,49.571429,884.344898,80,8.00,0.1000,4333.29,433.329000,88.434490,Patrícia Croce Doces - Pik n' Pak-RIO - Botafogo,25.096274,15x or more,0.058824,0.000000,0.058824,0.929412,0
1,2023-07-17,Brazil,67f3637f-fff2-4ac7-9ad7-78d976319994,Açaí RUDE | Rude Food,67f3637f-fff2-4ac7-9ad7-78d976319994,Açaí RUDE | Rude Food,a63559d1-93c6-4f60-b683-1e207ccda10a,SAO - Batista,False,3PCR,Sao Paulo,32,0,0,0,30,1.062500,698.40,0,0.00,2,1,1,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Rude food Açaí,0066T00001A3XIpQAN,SAO - Batista,"CR - Ice Cream, Popsicle, Cookie Dough & Gelato",0066T00001A3XIpQAN,Sao Paulo,Brazil,False,NaT,10.00,0.0,400.00,0.00,2022-12-20,2023-02-17,2023-02-16,NaN,None,None,False,False,Percentage / Order,0.00,4.0,0.0,g - Launched,67f3637f-fff2-4ac7-9ad7-78d976319994,True,67f3637f-fff2-4ac7-9ad7-78d976319994,67f3637f-fff2-4ac7-9ad7-78d976319994,0056T000009M15JQAS,CR - Frozen Desserts,CR - Ice Cream,"CR - Ice Cream, Popsicle, Cookie Dough & Gelato",Tier 2,None,1640.0,400.00,BRL,4.80,83.333333,145.500000,0.000000,145.500000,32,4.00,0.1000,698.40,69.840000,14.550000,Açaí RUDE | Rude Food-SAO - Batista,7.482857,5x to 9.9x,0.000000,0.000000,0.000000,0.937500,0
2,2023-06-26,Mexico,b556b267-8821-4162-a0cc-254fc70dafa0,Tere Cazola,b556b267-8821-4162-a0cc-254fc70dafa0,Tere Cazola,9101d4f9-828f-45c5-b8b6-16986e76b374,MEX - Pedregal,False,3PCR,Mexico City,242,8,4,12,239,1.785124,72497.79,0,3107.55,1,1,1,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Tere Cazola,0065G00000j4xZXQAY,MEX - Pedregal,CR - Baked Goods,0065G00000j4xZXQAY,Mexico City,Mexico,False,NaT,3.51,15812.5,0.00,0.00,2021-

# Appending both regions

In [122]:
# inspecting us table
gmv_mrr_us.head(2)

,created_week,country,prep_organization_id,prep_organization_name,seller_organization_id,seller_organization_name,facility_id,facility_name,inventory_owned,distribution_type,city,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,items,gmv,lost_gmv,cancelled_gmv,avg_order_rating,orders_20picked,OFOs,cities,facilities,doors_id,sfdc_account_name,sfdc_account_id,is_churned,churn_date,closed_won_date,activated_date,billable_date,is_billable,storage_cu_ft,mrr,activation_fee,processing_fee_percentage,pct_cancelled,pct_issues,pct_odr,pct_scanned,succesful_orders,succesful_gmv,est_processing_income,ad_est_processing_income,store_gmv_mrr,gmv_mrr_cat
0,2023-08-07,USA,017b46ad-3a10-4d42-80f2-b3b84abb1019,​Tubby's Taste LLC,017b46ad-3a10-4d42-80f2-b3b84abb1019,​Tubby's Taste LLC,75187d2a-01d8-467f-9993-90e200a3a586,NA-US-IL-CHI-River West (Correct),False,3PCR,Chicago,1,0,0,0,1,3.000000,27.00,0.0,0.00,,0,1,1,1,017b46ad-3a10-4d42-80f2-b3b84abb101975187d2a-0...,unknown_017b46ad-3a10-4d42-80f2-b3b84abb1019,missing_017b46ad-3a10-4d42-80f2-b3b84abb1019,False,,2023-07-20,2023-08-10,,False,,NaN,120.0,NaN,0.000000,0.0,0.000000,1.0,1,27.00,NaN,NaN,NaN,0X
1,2023-08-07,USA,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,4411e09a-c8b7-4a2e-ae6e-5d028e303791,NA-US-TX-DAL-Commerce,False,3PCR,Dallas,6,5,0,5,0,2.333333,127.66,0.0,106.68,,0,1,1,1,658618ea-2e32-47ba-8a6b-fe89342ad1c54411e09a-c...,unknown_658618ea-2e32-47ba-8a6b-fe89342ad1c5,missing_658618ea-2e32-47ba-8a6b-fe89342ad1c5,False,,NaT,,,False,,NaN,120.0,NaN,0.833333,0.0,0.833333,0.0,1,20.98,NaN,NaN,NaN,0X


In [123]:
gmv_mrr_us["exchange_rate"] = 1
gmv_mrr_us["exchange_rate"].unique()

array([1])

In [124]:
gmv_mrr_us["mrr_usd"] = gmv_mrr_us["mrr"] / gmv_mrr_us["exchange_rate"]
gmv_mrr_us["total_gmv_usd"] = gmv_mrr_us["gmv"] / gmv_mrr_us["exchange_rate"]
gmv_mrr_us["cancelled_gmv_usd"] = gmv_mrr_us["cancelled_gmv"] / gmv_mrr_us["exchange_rate"]
gmv_mrr_us["succesful_gmv_usd"] = gmv_mrr_us["succesful_gmv"] / gmv_mrr_us["exchange_rate"]
gmv_mrr_us["processing_income_usd"] = gmv_mrr_us["ad_est_processing_income"] / gmv_mrr_us["exchange_rate"]
gmv_mrr_us["lost_gmv_usd"] = gmv_mrr_us["lost_gmv"] / gmv_mrr_us["exchange_rate"]

In [125]:
# Selecting US columns
gmv_mrr_us_2 = gmv_mrr_us[['created_week',
                                     'country',
                                     'city',
                                     'prep_organization_id',
                                     'prep_organization_name',
                                     'seller_organization_id',
                                     'seller_organization_name',
                                     'facility_id',
                                     'facility_name',
                                     'inventory_owned',
                                     'distribution_type',
                                     'total_orders',
                                     'cancelled_orders',
                                     'issue_orders',
                                     'ODR_orders',
                                     'scanned_orders',
                                     'items',
                                     'gmv',
                                     'lost_gmv',
                                     'cancelled_gmv',
                                     'OFOs',
                                     'cities',
                                     'facilities',
                                     'is_churned',
                                     'churn_date',
                                     'closed_won_date',
                                     'activated_date',
                                     'billable_date',
                                     'storage_cu_ft',
                                     'mrr',
                                     'processing_fee_percentage',
                                     'pct_cancelled',
                                     'pct_issues',
                                     'pct_odr',
                                     'pct_scanned',
                                     'succesful_orders',
                                     'succesful_gmv',
                                     'ad_est_processing_income',
                                     'store_gmv_mrr',
                                     'gmv_mrr_cat',
                                     'exchange_rate',
                                     'mrr_usd',
                                     'total_gmv_usd',
                                     'cancelled_gmv_usd',
                                     'lost_gmv_usd',
                                     'succesful_gmv_usd',
                                     'processing_income_usd']]
                            

In [126]:
gmv_mrr_us_2.head(2)

,created_week,country,city,prep_organization_id,prep_organization_name,seller_organization_id,seller_organization_name,facility_id,facility_name,inventory_owned,distribution_type,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,items,gmv,lost_gmv,cancelled_gmv,OFOs,cities,facilities,is_churned,churn_date,closed_won_date,activated_date,billable_date,storage_cu_ft,mrr,processing_fee_percentage,pct_cancelled,pct_issues,pct_odr,pct_scanned,succesful_orders,succesful_gmv,ad_est_processing_income,store_gmv_mrr,gmv_mrr_cat,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,lost_gmv_usd,succesful_gmv_usd,processing_income_usd
0,2023-08-07,USA,Chicago,017b46ad-3a10-4d42-80f2-b3b84abb1019,​Tubby's Taste LLC,017b46ad-3a10-4d42-80f2-b3b84abb1019,​Tubby's Taste LLC,75187d2a-01d8-467f-9993-90e200a3a586,NA-US-IL-CHI-River West (Correct),False,3PCR,1,0,0,0,1,3.000000,27.00,0.0,0.00,1,1,1,False,,2023-07-20,2023-08-10,,,NaN,NaN,0.000000,0.0,0.000000,1.0,1,27.00,NaN,NaN,0X,1,NaN,27.00,0.00,0.0,27.00,NaN
1,2023-08-07,USA,Dallas,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,4411e09a-c8b7-4a2e-ae6e-5d028e303791,NA-US-TX-DAL-Commerce,False,3PCR,6,5,0,5,0,2.333333,127.66,0.0,106.68,1,1,1,False,,NaT,,,,NaN,NaN,0.833333,0.0,0.833333,0.0,1,20.98,NaN,NaN,0X,1,NaN,127.66,106.68,0.0,20.98,NaN


In [127]:
# Inspecting latam table
combined_data_fx.head(2)

,created_week,country,organization_id,name,seller_organization_id,seller_organization_name,facility_id,facility_name,inventory_owned,distribution_type,city,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,Avg_Items_per_Order,total_gmv_local,lost_gmv,cancelled_gmv_local,OFOs,cities,facilities,attributes,Account,acct_name_jira,Opportunity_ID_18__c,Facility_Name__c,Cuisine_Type__c,Id,kitchen_city,kitchen_country,is_churned,churn_date,processing_fee_int,Monthly_Shelf_Fee_Dry_Storage__c,Monthly_Shelf_Fee_Frozen_Storage__c,Monthly_Shelf_Fee_Cold_Storage__c,closed_won_date,Move_In_Date__c,Start_Date__c,Loss_Reason__c,Specific_Lost_Reason__c,Lost_Explanation__c,CR_Opp_Opted_out_Affiliate__c,CR_Opp_Restrictions_Affiliate__c,CR_Opp_Type_of_Processing_Fee__c,Storage_Amount_Dry_CU_FT__c,Storage_Amount_Frozen_CU_FT__c,Storage_Amount_Cold_CU_FT__c,CR_Opp_OB_Status__c,sfdc_org_id,current_customer,org_id,Org_Id_Otter__c,CSM__c,CR_Account_Category__c,CR_Account_Subcategory__c,CR_Account_Cuisine_Type__c,LAM_TierScrape__c,CR_Type_of_Customer__c,CR_Acc_Social_Media_Presence_Numeric__c,MRR_local,currency_code,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,succesful_gmv_usd,succesful_orders,total_cu_ft,processing_fee_pct,succesful_gmv_local,processing_income_local,processing_income_usd,name_facility,store_gmv_mrr,gmv_mrr_cat,pct_cancelled,pct_issues,pct_odr,pct_scanned,lost_gmv_usd
0,2023-08-07,Brazil,893bb087-45ee-4436-9463-d8d3667b0d43,Patrícia Croce Doces - Pik n' Pak,893bb087-45ee-4436-9463-d8d3667b0d43,Patrícia Croce Doces - Pik n' Pak,3759ad21-4537-48ea-b1f1-48809e8a03ac,RIO - Botafogo,False,3PCR,Rio de Janeiro,85,5,0,5,79,2.152941,4576.19,0,242.9,2,1,1,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Patrícia Croce Doces,0066T00001A3mPGQAZ,RIO - Botafogo,CR - Candy/Sweets/Snacks,0066T00001A3mPGQAZ,Rio de Janeiro,Brazil,False,NaT,10.0,0.0,0.0,740.0,2022-11-30,2023-01-06,2023-01-05,NaN,None,None,True,False,None,0.0,0.0,8.0,g - Launched,893bb087-45ee-4436-9463-d8d3667b0d43,False,893bb087-45ee-4436-9463-d8d3667b0d43,893bb087-45ee-4436-9463-d8d3667b0d43,0056T000009cfwEQAQ,CR - Desserts,CR - Pies,CR - Candy/Sweets/Snacks,Tier 2,Local,1341.0,740.0,BRL,4.9,151.020408,933.916327,49.571429,884.344898,80,8.0,0.1,4333.29,433.329,88.43449,Patrícia Croce Doces - Pik n' Pak-RIO - Botafogo,25.096274,15x or more,0.058824,0.0,0.058824,0.929412,0
1,2023-07-17,Brazil,67f3637f-fff2-4ac7-9ad7-78d976319994,Açaí RUDE | Rude Food,67f3637f-fff2-4ac7-9ad7-78d976319994,Açaí RUDE | Rude Food,a63559d1-93c6-4f60-b683-1e207ccda10a,SAO - Batista,False,3PCR,Sao Paulo,32,0,0,0,30,1.062500,698.40,0,0.0,2,1,1,"{'type': 'Opportunity', 'url': '/services/data...","{'attributes': {'type': 'Account', 'url': '/se...",Rude food Açaí,0066T00001A3XIpQAN,SAO - Batista,"CR - Ice Cream, Popsicle, Cookie Dough & Gelato",0066T00001A3XIpQAN,Sao Paulo,Brazil,False,NaT,10.0,0.0,400.0,0.0,2022-12-20,2023-02-17,2023-02-16,NaN,None,None,False,False,Percentage / Order,0.0,4.0,0.0,g - Launched,67f3637f-fff2-4ac7-9ad7-78d976319994,True,67f3637f-fff2-4ac7-9ad7-78d976319994,67f3637f-fff2-4ac7-9ad7-78d976319994,0056T000009M15JQAS,CR - Frozen Desserts,CR - Ice Cream,"CR - Ice Cream, Popsicle, Cookie Dough & Gelato",Tier 2,None,1640.0,400.0,BRL,4.8,83.333333,145.500000,0.000000,145.500000,32,4.0,0.1,698.40,69.840,14.55000,Açaí RUDE | Rude Food-SAO - Batista,7.482857,5x to 9.9x,0.000000,0.0,0.000000,0.937500,0


In [128]:
# Selecting columns for latam
gmv_mrr_latam = combined_data_fx[['created_week',
                                     'country',
                                     'city',
                                     'organization_id',
                                     'name',
                                     'seller_organization_id',
                                     'seller_organization_name',
                                     'facility_id',
                                     'facility_name',
                                     'inventory_owned',
                                     'distribution_type',
                                     'total_orders',
                                     'cancelled_orders',
                                     'issue_orders',
                                     'ODR_orders',
                                     'scanned_orders',
                                     'Avg_Items_per_Order',
                                     'total_gmv_local',
                                     'lost_gmv',
                                     'cancelled_gmv_local',
                                     'OFOs',
                                     'cities',
                                     'facilities',
                                     'is_churned',
                                     'churn_date',
                                     'closed_won_date',
                                     'Start_Date__c',
                                     'Move_In_Date__c',
                                    'CR_Opp_OB_Status__c',
                                     'total_cu_ft',
                                     'MRR_local',
                                     'processing_fee_pct',
                                     'pct_cancelled',
                                     'pct_issues',
                                     'pct_odr',
                                     'pct_scanned',
                                     'succesful_orders',
                                     'succesful_gmv_local',
                                     'processing_income_local',
                                     'store_gmv_mrr',
                                     'gmv_mrr_cat',
                                     'exchange_rate',
                                     'mrr_usd',
                                     'total_gmv_usd',
                                     'cancelled_gmv_usd',
                                     'lost_gmv_usd',
                                     'succesful_gmv_usd',
                                     'processing_income_usd']]

In [129]:
# Inspect
gmv_mrr_latam.head(2)

,created_week,country,city,organization_id,name,seller_organization_id,seller_organization_name,facility_id,facility_name,inventory_owned,distribution_type,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,Avg_Items_per_Order,total_gmv_local,lost_gmv,cancelled_gmv_local,OFOs,cities,facilities,is_churned,churn_date,closed_won_date,Start_Date__c,Move_In_Date__c,CR_Opp_OB_Status__c,total_cu_ft,MRR_local,processing_fee_pct,pct_cancelled,pct_issues,pct_odr,pct_scanned,succesful_orders,succesful_gmv_local,processing_income_local,store_gmv_mrr,gmv_mrr_cat,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,lost_gmv_usd,succesful_gmv_usd,processing_income_usd
0,2023-08-07,Brazil,Rio de Janeiro,893bb087-45ee-4436-9463-d8d3667b0d43,Patrícia Croce Doces - Pik n' Pak,893bb087-45ee-4436-9463-d8d3667b0d43,Patrícia Croce Doces - Pik n' Pak,3759ad21-4537-48ea-b1f1-48809e8a03ac,RIO - Botafogo,False,3PCR,85,5,0,5,79,2.152941,4576.19,0,242.9,2,1,1,False,NaT,2022-11-30,2023-01-05,2023-01-06,g - Launched,8.0,740.0,0.1,0.058824,0.0,0.058824,0.929412,80,4333.29,433.329,25.096274,15x or more,4.9,151.020408,933.916327,49.571429,0,884.344898,88.43449
1,2023-07-17,Brazil,Sao Paulo,67f3637f-fff2-4ac7-9ad7-78d976319994,Açaí RUDE | Rude Food,67f3637f-fff2-4ac7-9ad7-78d976319994,Açaí RUDE | Rude Food,a63559d1-93c6-4f60-b683-1e207ccda10a,SAO - Batista,False,3PCR,32,0,0,0,30,1.062500,698.40,0,0.0,2,1,1,False,NaT,2022-12-20,2023-02-16,2023-02-17,g - Launched,4.0,400.0,0.1,0.000000,0.0,0.000000,0.937500,32,698.40,69.840,7.482857,5x to 9.9x,4.8,83.333333,145.500000,0.000000,0,145.500000,14.55000


In [130]:
# Renaming to new convention
gmv_mrr_us_3 = gmv_mrr_us_2.rename(columns={
                                     'gmv': 'total_gmv_local',
                                     'lost_gmv': 'lost_gmv_local',
                                     'items':'Avg_Items_per_Order',
                                     'cancelled_gmv':'cancelled_gmv_local',
                                     'storage_cu_ft': 'total_cu_ft',
                                     'mrr':'MRR_local',
                                     'processing_fee_percentage':'processing_fee_pct',
                                     'succesful_gmv': 'succesful_gmv_local' ,
                                     'ad_est_processing_income': 'processing_income_local'})

In [131]:
# Renaming to new convention
gmv_mrr_latam_2= gmv_mrr_latam.rename(columns={
                                     'organization_id': 'prep_organization_id',
                                     'name': 'prep_organization_name',
                                     'lost_gmv':'lost_gmv_local',
                                     'cancelled_gmv':'cancelled_gmv_local',
                                     'storage_cu_ft': 'total_cu_ft',
                                     'mrr':'MRR_local',
                                     'processing_fee_percentage':'processing_fee_pct',
                                     'succesful_gmv': 'succesful_gmv_local' ,
                                     'ad_est_processing_income': 'processing_income_local'})

In [132]:
# Inspect
gmv_mrr_us_3.head(2)

,created_week,country,city,prep_organization_id,prep_organization_name,seller_organization_id,seller_organization_name,facility_id,facility_name,inventory_owned,distribution_type,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,Avg_Items_per_Order,total_gmv_local,lost_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,is_churned,churn_date,closed_won_date,activated_date,billable_date,total_cu_ft,MRR_local,processing_fee_pct,pct_cancelled,pct_issues,pct_odr,pct_scanned,succesful_orders,succesful_gmv_local,processing_income_local,store_gmv_mrr,gmv_mrr_cat,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,lost_gmv_usd,succesful_gmv_usd,processing_income_usd
0,2023-08-07,USA,Chicago,017b46ad-3a10-4d42-80f2-b3b84abb1019,​Tubby's Taste LLC,017b46ad-3a10-4d42-80f2-b3b84abb1019,​Tubby's Taste LLC,75187d2a-01d8-467f-9993-90e200a3a586,NA-US-IL-CHI-River West (Correct),False,3PCR,1,0,0,0,1,3.000000,27.00,0.0,0.00,1,1,1,False,,2023-07-20,2023-08-10,,,NaN,NaN,0.000000,0.0,0.000000,1.0,1,27.00,NaN,NaN,0X,1,NaN,27.00,0.00,0.0,27.00,NaN
1,2023-08-07,USA,Dallas,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,4411e09a-c8b7-4a2e-ae6e-5d028e303791,NA-US-TX-DAL-Commerce,False,3PCR,6,5,0,5,0,2.333333,127.66,0.0,106.68,1,1,1,False,,NaT,,,,NaN,NaN,0.833333,0.0,0.833333,0.0,1,20.98,NaN,NaN,0X,1,NaN,127.66,106.68,0.0,20.98,NaN


In [133]:
gmv_mrr_latam_2.head(2)

,created_week,country,city,prep_organization_id,prep_organization_name,seller_organization_id,seller_organization_name,facility_id,facility_name,inventory_owned,distribution_type,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,Avg_Items_per_Order,total_gmv_local,lost_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,is_churned,churn_date,closed_won_date,Start_Date__c,Move_In_Date__c,CR_Opp_OB_Status__c,total_cu_ft,MRR_local,processing_fee_pct,pct_cancelled,pct_issues,pct_odr,pct_scanned,succesful_orders,succesful_gmv_local,processing_income_local,store_gmv_mrr,gmv_mrr_cat,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,lost_gmv_usd,succesful_gmv_usd,processing_income_usd
0,2023-08-07,Brazil,Rio de Janeiro,893bb087-45ee-4436-9463-d8d3667b0d43,Patrícia Croce Doces - Pik n' Pak,893bb087-45ee-4436-9463-d8d3667b0d43,Patrícia Croce Doces - Pik n' Pak,3759ad21-4537-48ea-b1f1-48809e8a03ac,RIO - Botafogo,False,3PCR,85,5,0,5,79,2.152941,4576.19,0,242.9,2,1,1,False,NaT,2022-11-30,2023-01-05,2023-01-06,g - Launched,8.0,740.0,0.1,0.058824,0.0,0.058824,0.929412,80,4333.29,433.329,25.096274,15x or more,4.9,151.020408,933.916327,49.571429,0,884.344898,88.43449
1,2023-07-17,Brazil,Sao Paulo,67f3637f-fff2-4ac7-9ad7-78d976319994,Açaí RUDE | Rude Food,67f3637f-fff2-4ac7-9ad7-78d976319994,Açaí RUDE | Rude Food,a63559d1-93c6-4f60-b683-1e207ccda10a,SAO - Batista,False,3PCR,32,0,0,0,30,1.062500,698.40,0,0.0,2,1,1,False,NaT,2022-12-20,2023-02-16,2023-02-17,g - Launched,4.0,400.0,0.1,0.000000,0.0,0.000000,0.937500,32,698.40,69.840,7.482857,5x to 9.9x,4.8,83.333333,145.500000,0.000000,0,145.500000,14.55000


### Append both tables

In [134]:
# Check gmvs and orders
print("US sGMV is", gmv_mrr_us_3.succesful_gmv_usd.sum())
print("US sOrders are", gmv_mrr_us_3.succesful_orders.sum())
print("Latam rows are", len(gmv_mrr_us_3))

US sGMV is 2986749.54
US sOrders are 157903
Latam rows are 7792


In [135]:
# Check gmvs and orders
print("Latam sGMV is", gmv_mrr_latam_2.succesful_gmv_usd.sum())
print("Latam sOrders are", gmv_mrr_latam_2.succesful_orders.sum())
print("Latam rows are", len(gmv_mrr_latam_2))

Latam sGMV is 2757961.3387339474
Latam sOrders are 226640
Latam rows are 8183


In [136]:
# Append and ignore index
doors_americas = pd.concat([gmv_mrr_us_3, gmv_mrr_latam_2], ignore_index = True)

In [137]:
doors_americas.head(2)

,created_week,country,city,prep_organization_id,prep_organization_name,seller_organization_id,seller_organization_name,facility_id,facility_name,inventory_owned,distribution_type,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,Avg_Items_per_Order,total_gmv_local,lost_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,is_churned,churn_date,closed_won_date,activated_date,billable_date,total_cu_ft,MRR_local,processing_fee_pct,pct_cancelled,pct_issues,pct_odr,pct_scanned,succesful_orders,succesful_gmv_local,processing_income_local,store_gmv_mrr,gmv_mrr_cat,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,lost_gmv_usd,succesful_gmv_usd,processing_income_usd,Start_Date__c,Move_In_Date__c,CR_Opp_OB_Status__c
0,2023-08-07,USA,Chicago,017b46ad-3a10-4d42-80f2-b3b84abb1019,​Tubby's Taste LLC,017b46ad-3a10-4d42-80f2-b3b84abb1019,​Tubby's Taste LLC,75187d2a-01d8-467f-9993-90e200a3a586,NA-US-IL-CHI-River West (Correct),False,3PCR,1,0,0,0,1,3.000000,27.00,0.0,0.00,1,1,1,False,,2023-07-20,2023-08-10,,,NaN,NaN,0.000000,0.0,0.000000,1.0,1,27.00,NaN,NaN,0X,1.0,NaN,27.00,0.00,0.0,27.00,NaN,NaN,NaN,NaN
1,2023-08-07,USA,Dallas,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,4411e09a-c8b7-4a2e-ae6e-5d028e303791,NA-US-TX-DAL-Commerce,False,3PCR,6,5,0,5,0,2.333333,127.66,0.0,106.68,1,1,1,False,,NaT,,,,NaN,NaN,0.833333,0.0,0.833333,0.0,1,20.98,NaN,NaN,0X,1.0,NaN,127.66,106.68,0.0,20.98,NaN,NaN,NaN,NaN


In [138]:
# Add last updated timestamps
doors_americas["data_pulled_ts_MEX"] = datetime.datetime.now(pytz.timezone('Mexico/General'))
doors_americas["data_pulled_ts_LAX"] = datetime.datetime.now(pytz.timezone('America/Los_Angeles'))

In [139]:
# Check gmvs and orders
print("Americas sGMV is", doors_americas.succesful_gmv_usd.sum())
print("Americas sOrders are", doors_americas.succesful_orders.sum())
print("Americas rows are", len(doors_americas))

Americas sGMV is 5744710.878733947
Americas sOrders are 384543
Americas rows are 15975


**Add additional fields from SF**

In [140]:
# Import credentials
path = "/home/shared/joseph.jensen/Alan Data Pipelines/Credentials/sf_auth_JJ.json"

loginInfo = json.load(open(path))
session_id, instance = SalesforceLogin(username= loginInfo["username"],
                                                 password= loginInfo["password"],
                                                 security_token= loginInfo["security_token"],
                                                 domain= "login")

# Set SF connection
sf = Salesforce(instance= instance, session_id= session_id)

In [141]:
add_fields = """
SELECT
Org_Id_Otter__c,
CR_Account_Categories__c,
CR_Account_Sub_category__c,
CR_Acc_Social_Media_Presence_Numeric__c,
Yelp_of_Reviews__c


FROM
Account
"""


print("Querying data...")

# Query users to dataframe
add_fields = pd.DataFrame(sf.query_all(add_fields)['records'])
print("Data pulled!")

# Inspect 
acct_query.head()
print("Rows:", len(add_fields))

Querying data...
Data pulled!
Rows: 65570


In [142]:
print("add fields len is {}".format(len(add_fields)))
add_fields.sample(3)

add fields len is 65570


,attributes,Org_Id_Otter__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Acc_Social_Media_Presence_Numeric__c,Yelp_of_Reviews__c
46185,"{'type': 'Account', 'url': '/services/data/v42...",None,None,None,NaN,NaN
1889,"{'type': 'Account', 'url': '/services/data/v42...",None,None,None,NaN,NaN
54645,"{'type': 'Account', 'url': '/services/data/v42...",None,None,None,NaN,NaN


In [143]:
print("doors americas len is {}".format(len(doors_americas)))
doors_americas.head(2)

doors americas len is 15975


,created_week,country,city,prep_organization_id,prep_organization_name,seller_organization_id,seller_organization_name,facility_id,facility_name,inventory_owned,distribution_type,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,Avg_Items_per_Order,total_gmv_local,lost_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,is_churned,churn_date,closed_won_date,activated_date,billable_date,total_cu_ft,MRR_local,processing_fee_pct,pct_cancelled,pct_issues,pct_odr,pct_scanned,succesful_orders,succesful_gmv_local,processing_income_local,store_gmv_mrr,gmv_mrr_cat,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,lost_gmv_usd,succesful_gmv_usd,processing_income_usd,Start_Date__c,Move_In_Date__c,CR_Opp_OB_Status__c,data_pulled_ts_MEX,data_pulled_ts_LAX
0,2023-08-07,USA,Chicago,017b46ad-3a10-4d42-80f2-b3b84abb1019,​Tubby's Taste LLC,017b46ad-3a10-4d42-80f2-b3b84abb1019,​Tubby's Taste LLC,75187d2a-01d8-467f-9993-90e200a3a586,NA-US-IL-CHI-River West (Correct),False,3PCR,1,0,0,0,1,3.000000,27.00,0.0,0.00,1,1,1,False,,2023-07-20,2023-08-10,,,NaN,NaN,0.000000,0.0,0.000000,1.0,1,27.00,NaN,NaN,0X,1.0,NaN,27.00,0.00,0.0,27.00,NaN,NaN,NaN,NaN,2023-09-20 17:57:04.219296-05:00,2023-09-20 15:57:04.220804-07:00
1,2023-08-07,USA,Dallas,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,4411e09a-c8b7-4a2e-ae6e-5d028e303791,NA-US-TX-DAL-Commerce,False,3PCR,6,5,0,5,0,2.333333,127.66,0.0,106.68,1,1,1,False,,NaT,,,,NaN,NaN,0.833333,0.0,0.833333,0.0,1,20.98,NaN,NaN,0X,1.0,NaN,127.66,106.68,0.0,20.98,NaN,NaN,NaN,NaN,2023-09-20 17:57:04.219296-05:00,2023-09-20 15:57:04.220804-07:00


In [144]:
# Merge
doors_americas_2 = doors_americas.merge(add_fields, how = "left", left_on = ["prep_organization_id"], right_on = ["Org_Id_Otter__c"])
print("doors americas 2 len is {}".format(len(doors_americas_2)))

doors americas 2 len is 15975


In [145]:
doors_americas_2.head(2)

,created_week,country,city,prep_organization_id,prep_organization_name,seller_organization_id,seller_organization_name,facility_id,facility_name,inventory_owned,distribution_type,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,Avg_Items_per_Order,total_gmv_local,lost_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,is_churned,churn_date,closed_won_date,activated_date,billable_date,total_cu_ft,MRR_local,processing_fee_pct,pct_cancelled,pct_issues,pct_odr,pct_scanned,succesful_orders,succesful_gmv_local,processing_income_local,store_gmv_mrr,gmv_mrr_cat,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,lost_gmv_usd,succesful_gmv_usd,processing_income_usd,Start_Date__c,Move_In_Date__c,CR_Opp_OB_Status__c,data_pulled_ts_MEX,data_pulled_ts_LAX,attributes,Org_Id_Otter__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Acc_Social_Media_Presence_Numeric__c,Yelp_of_Reviews__c
0,2023-08-07,USA,Chicago,017b46ad-3a10-4d42-80f2-b3b84abb1019,​Tubby's Taste LLC,017b46ad-3a10-4d42-80f2-b3b84abb1019,​Tubby's Taste LLC,75187d2a-01d8-467f-9993-90e200a3a586,NA-US-IL-CHI-River West (Correct),False,3PCR,1,0,0,0,1,3.000000,27.00,0.0,0.00,1,1,1,False,,2023-07-20,2023-08-10,,,NaN,NaN,0.000000,0.0,0.000000,1.0,1,27.00,NaN,NaN,0X,1.0,NaN,27.00,0.00,0.0,27.00,NaN,NaN,NaN,NaN,2023-09-20 17:57:04.219296-05:00,2023-09-20 15:57:04.220804-07:00,"{'type': 'Account', 'url': '/services/data/v42...",017b46ad-3a10-4d42-80f2-b3b84abb1019,CR - Baked Goods,CR - Cookies,5851.0,5.0
1,2023-08-07,USA,Dallas,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,4411e09a-c8b7-4a2e-ae6e-5d028e303791,NA-US-TX-DAL-Commerce,False,3PCR,6,5,0,5,0,2.333333,127.66,0.0,106.68,1,1,1,False,,NaT,,,,NaN,NaN,0.833333,0.0,0.833333,0.0,1,20.98,NaN,NaN,0X,1.0,NaN,127.66,106.68,0.0,20.98,NaN,NaN,NaN,NaN,2023-09-20 17:57:04.219296-05:00,2023-09-20 15:57:04.220804-07:00,"{'type': 'Account', 'url': '/services/data/v42...",658618ea-2e32-47ba-8a6b-fe89342ad1c5,CR - Baked Goods,CR - Bakery,350.0,4.5


In [146]:
# Create bucket for social media presence with same SF field name
social_media = []

for s in doors_americas_2["CR_Acc_Social_Media_Presence_Numeric__c"]:
    if s >= 100000:
        s = '>100k'
    elif s >= 50000:
        s = '50k-100k'
    elif s >= 50000:
        s = '20k-50k'
    elif s >= 10000:
        s = '10k-20k'
    elif s >= 7000:
        s = '7k-10k'
    elif s >= 5000:
        s = '5k-7k'
    elif s >= 3000:
        s = '3k-5k'
    elif s >= 1000:
        s = '1k-3k'
    else:
        s = '<1k'
    social_media.append(s)

# Create new col with ex SF name
doors_americas_2["CR_Acc_Social_Media_Presence_Group"] = social_media

In [147]:
test_us = doors_americas_2[doors_americas_2["country"] == 'USA']
test_us.sample(3)

,created_week,country,city,prep_organization_id,prep_organization_name,seller_organization_id,seller_organization_name,facility_id,facility_name,inventory_owned,distribution_type,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,Avg_Items_per_Order,total_gmv_local,lost_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,is_churned,churn_date,closed_won_date,activated_date,billable_date,total_cu_ft,MRR_local,processing_fee_pct,pct_cancelled,pct_issues,pct_odr,pct_scanned,succesful_orders,succesful_gmv_local,processing_income_local,store_gmv_mrr,gmv_mrr_cat,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,lost_gmv_usd,succesful_gmv_usd,processing_income_usd,Start_Date__c,Move_In_Date__c,CR_Opp_OB_Status__c,data_pulled_ts_MEX,data_pulled_ts_LAX,attributes,Org_Id_Otter__c,CR_Account_Categories__c,CR_Account_Sub_category__c,CR_Acc_Social_Media_Presence_Numeric__c,Yelp_of_Reviews__c,CR_Acc_Social_Media_Presence_Group
824,2023-08-14,USA,Austin,a7951b53-7910-4896-893b-9eec847da80c,The Dabs Bakery LLC,a7951b53-7910-4896-893b-9eec847da80c,The Dabs Bakery LLC,ce1e2ca1-6944-48d4-a93c-67bf011560f6,NA-US-TX-AUS-North Loop-CORRECT,False,3PCR,31,0,0,0,30,2.83871,704.00,0.0,0.0,3,1,1,False,,2022-12-06,2022-12-15,,,NaN,0.15,0.0,0.0,0.0,0.967742,31,704.00,105.600,NaN,0X,1.0,NaN,704.00,0.0,0.0,704.00,105.600,NaN,NaN,NaN,2023-09-20 17:57:04.219296-05:00,2023-09-20 15:57:04.220804-07:00,"{'type': 'Account', 'url': '/services/data/v42...",a7951b53-7910-4896-893b-9eec847da80c,CR - Baked Goods,CR - Cookies,285.0,NaN,<1k
7719,2023-09-11,USA,Oakland,3f61d74b-e067-4ab1-a69e-79c588df1dc3,,d9fd27bd-5f7c-4ba0-8417-ee88489ff660,,5d574e15-e1d4-42a3-837a-71eebe29728a,NA-US-CA(N)-OAK-Adeline,False,2PCR,1,0,0,0,1,1.00000,10.00,0.0,0.0,1,1,1,False,,2021-08-16,2021-10-13,2021-10-13,,NaN,0.15,0.0,0.0,0.0,1.000000,1,10.00,1.500,NaN,0X,1.0,NaN,10.00,0.0,0.0,10.00,1.500,NaN,NaN,NaN,2023-09-20 17:57:04.219296-05:00,2023-09-20 15:57:04.220804-07:00,"{'type': 'Account', 'url': '/services/data/v42...",3f61d74b-e067-4ab1-a69e-79c588df1dc3,CR - Frozen Desserts,CR - Ice Cream,466000.0,NaN,>100k
2673,2023-06-12,USA,San Jose,df5b8d1d-2745-457e-9300-d8810a863efa,ReTHINK Ice Cream,df5b8d1d-2745-457e-9300-d8810a863efa,ReTHINK Ice Cream,905cbe43-327f-4d3e-a897-48ee81a64ade,NA-US-CA(N)-SJ-Ruff (Kitchens),False,3PCR,3,0,0,0,3,1.00000,35.48,0.0,0.0,2,1,1,True,2023-06-07,2021-06-02,2021-06-02,2021-06-02,,NaN,0.15,0.0,0.0,0.0,1.000000,3,35.48,5.322,NaN,0X,1.0,NaN,35.48,0.0,0.0,35.48,5.322,NaN,NaN,NaN,2023-09-20 17:57:04.219296-05:00,2023-09-20 15:57:04.220804-07:00,"{'type': 'Account', 'url': '/services/data/v42...",df5b8d1d-2745-457e-9300-d8810a863efa,CR - Frozen Desserts,CR - Ice Cream,15400.0,NaN,10k-20k


## Upload to GBQ

In [148]:
# Import libraries for GBQ
from pandas.io import gbq
import pandas_gbq
from google.oauth2 import service_account
from google.cloud import bigquery

In [149]:
doors_americas_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15975 entries, 0 to 15974
Data columns (total 59 columns):
 #   Column                                   Non-Null Count  Dtype                              
---  ------                                   --------------  -----                              
 0   created_week                             15975 non-null  object                             
 1   country                                  15975 non-null  object                             
 2   city                                     15975 non-null  object                             
 3   prep_organization_id                     15975 non-null  object                             
 4   prep_organization_name                   15975 non-null  object                             
 5   seller_organization_id                   15975 non-null  object                             
 6   seller_organization_name                 15975 non-null  object                             
 7   faci

In [150]:
s = secrets.Secrets()

In [151]:
doors_americas

,created_week,country,city,prep_organization_id,prep_organization_name,seller_organization_id,seller_organization_name,facility_id,facility_name,inventory_owned,distribution_type,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,Avg_Items_per_Order,total_gmv_local,lost_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,is_churned,churn_date,closed_won_date,activated_date,billable_date,total_cu_ft,MRR_local,processing_fee_pct,pct_cancelled,pct_issues,pct_odr,pct_scanned,succesful_orders,succesful_gmv_local,processing_income_local,store_gmv_mrr,gmv_mrr_cat,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,lost_gmv_usd,succesful_gmv_usd,processing_income_usd,Start_Date__c,Move_In_Date__c,CR_Opp_OB_Status__c,data_pulled_ts_MEX,data_pulled_ts_LAX
0,2023-08-07,USA,Chicago,017b46ad-3a10-4d42-80f2-b3b84abb1019,​Tubby's Taste LLC,017b46ad-3a10-4d42-80f2-b3b84abb1019,​Tubby's Taste LLC,75187d2a-01d8-467f-9993-90e200a3a586,NA-US-IL-CHI-River West (Correct),False,3PCR,1,0,0,0,1,3.000000,27.00,0.0,0.00,1,1,1,False,,2023-07-20,2023-08-10,,,NaN,NaN,0.000000,0.0,0.000000,1.0,1,27.00,NaN,NaN,0X,1.00,NaN,27.000000,0.00,0.0,27.000000,NaN,NaN,NaN,NaN,2023-09-20 17:57:04.219296-05:00,2023-09-20 15:57:04.220804-07:00
1,2023-08-07,USA,Dallas,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,4411e09a-c8b7-4a2e-ae6e-5d028e303791,NA-US-TX-DAL-Commerce,False,3PCR,6,5,0,5,0,2.333333,127.66,0.0,106.68,1,1,1,False,,NaT,,,,NaN,NaN,0.833333,0.0,0.833333,0.0,1,20.98,NaN,NaN,0X,1.00,NaN,127.660000,106.68,0.0,20.980000,NaN,NaN,NaN,NaN,2023-09-20 17:57:04.219296-05:00,2023-09-20 15:57:04.220804-07:00
2,2023-07-31,USA,Dallas,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,4411e09a-c8b7-4a2e-ae6e-5d028e303791,NA-US-TX-DAL-Commerce,False,3PCR,2,2,0,2,0,2.000000,37.72,0.0,37.72,1,1,1,False,,NaT,,,,NaN,NaN,1.000000,0.0,1.000000,0.0,0,0.00,NaN,NaN,0X,1.00,NaN,37.720000,37.72,0.0,0.000000,NaN,NaN,NaN,NaN,2023-09-20 17:57:04.219296-05:00,2023-09-20 15:57:04.220804-07:00
3,2023-08-14,USA,Dallas,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,4411e09a-c8b7-4a2e-ae6e-5d028e303791,NA-US-TX-DAL-Commerce,False,3PCR,1,1,0,1,0,1.000000,3.99,0.0,3.99,1,1,1,False,,NaT,,,,NaN,NaN,1.000000,0.0,1.000000,0.0,0,0.00,NaN,NaN,0X,1.00,NaN,3.990000,3.99,0.0,0.000000,NaN,NaN,NaN,NaN,2023-09-20 17:57:04.219296-05:00,2023-09-20 15:57:04.220804-07:00
4,2023-07-31,USA,Los Angeles,6440b884-ccd2-465d-b220-b93b8c7cda62,p.o.p. candy co.,6440b884-ccd2-465d-b220-b93b8c7cda62,p.o.p. candy co.,99573ac0-a3f2-4c70-946c-e9c06b425d5c,NA-US-CA(S)-LA-Koreatown,False,3PCR,2,2,0,2,1,2.000000,64.00,0.0,64.00,1,1,1,True,2023-05-16,2023-01-25,2023-02-10,,,NaN,0.15,1.000000,0.0,1.000000,0.5,0,0.00,0.000,NaN,0X,1.00,NaN,64.000000,64.00,0.0,0.000000,0.000000,NaN,NaN,NaN,2023-09-20 17:57:04.219296-05:00,2023-09-20 15:57:04.220804-07:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15970,2023-05-29 00:00:00,Brazil,Sao Paulo,9a5914a1-2b47-487b-ad56-0e4f492515d9,Nah's Bakery - Pik n' Pak,9a5914a1-2b47-487b-ad56-0e4f492515d9,Nah's Bakery - Pik n' Pak,dcc461ae-c80c-46cc-a5dc-4a5a72761510,SAO - Lapa Clelia,False,3PCR,2,0,0,0,2,1.000000,67.80,0.0,0.00,1,1,1,True,2023-05-26 00:00:00,2022-09-21,NaN,NaN,4.0,500.00,0.07,0.000000,0.0,0.000000,1.0,2,67.80,4.746,0.581143,0X,5.01,99.800399,13.532934,0.00,0.0,13.532934,0.947305,2022-11-01,2022-10-28,i - Churned,2023-09-20 17:57:04.219296-05:00,2023-09-20 15:57:04.220804-07:00
15971,2023-06-26 00:00:00,Brazil,Rio de Janeiro,048caf43-8454-4693-8bca-f78a474a8673,Viva o Doce,048caf43-8454-4693-8bca-f78a474a8673,Viva o Doce,466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb,RIO - Barra,False,3PCR,2,0,0,0,2,1.500000

In [152]:
# Upload data to GBQ (original DB without the new names and columns nov 22)

print("Uploading data to GBQ...")

#table_id = 'latam_cr_bizops.cr_doors_americas'
#project_id = 'css-operations'
#credentials_bq = service_account.Credentials.from_service_account_info(s.get_secret("cloud_retail_sp_service_account"))
#pandas_gbq.to_gbq(doors_americas_2,table_id, project_id = project_id, if_exists = 'replace', credentials = credentials_bq)


print("Data uploaded!")


Uploading data to GBQ...
Data uploaded!


In [153]:
doors_americas.columns

Index(['created_week', 'country', 'city', 'prep_organization_id',
       'prep_organization_name', 'seller_organization_id',
       'seller_organization_name', 'facility_id', 'facility_name',
       'inventory_owned', 'distribution_type', 'total_orders',
       'cancelled_orders', 'issue_orders', 'ODR_orders', 'scanned_orders',
       'Avg_Items_per_Order', 'total_gmv_local', 'lost_gmv_local',
       'cancelled_gmv_local', 'OFOs', 'cities', 'facilities', 'is_churned',
       'churn_date', 'closed_won_date', 'activated_date', 'billable_date',
       'total_cu_ft', 'MRR_local', 'processing_fee_pct', 'pct_cancelled',
       'pct_issues', 'pct_odr', 'pct_scanned', 'succesful_orders',
       'succesful_gmv_local', 'processing_income_local', 'store_gmv_mrr',
       'gmv_mrr_cat', 'exchange_rate', 'mrr_usd', 'total_gmv_usd',
       'cancelled_gmv_usd', 'lost_gmv_usd', 'succesful_gmv_usd',
       'processing_income_usd', 'Start_Date__c', 'Move_In_Date__c',
       'CR_Opp_OB_Status__c', 'data_p

In [154]:
doors_americas['created_week']=pd.to_datetime(doors_americas['created_week'])

In [155]:
#doors_americas['created_month'].dt.date

In [156]:
doors_americas['created_week'].value_counts()

2023-09-11    1031
2023-08-14    1000
2023-06-26     994
2023-08-07     993
2023-08-28     986
2023-07-24     980
2023-07-31     977
2023-09-04     974
2023-07-10     956
2023-07-17     954
2023-08-21     952
2023-07-03     922
2023-06-19     903
2023-06-12     869
2023-06-05     867
2023-05-29     836
2023-09-18     781
Name: created_week, dtype: int64

In [160]:
doors_americas['churn_date']=pd.to_datetime(doors_americas['churn_date'])

In [161]:
doors_americas['churn_date'].value_counts()

2023-02-28    153
2023-07-31    128
2023-07-01     79
2023-08-29     69
2023-09-19     68
2023-06-30     58
2023-07-05     49
2023-06-07     42
2023-08-31     40
2023-08-22     39
2023-08-27     39
2023-08-28     38
2023-09-01     34
2023-10-01     33
2023-03-22     31
2023-08-09     25
2023-09-30     22
2023-06-23     20
2023-09-14     19
2023-05-11     19
2023-07-07     19
2023-02-23     17
2023-04-10     17
2023-09-22     17
2023-06-03     17
2021-08-31     16
2023-08-14     16
2023-08-08     14
2023-07-24     13
2023-07-12     12
2023-10-04     11
2023-08-03     10
2023-09-08     10
2023-09-09     10
2023-07-26     10
2023-07-20      9
2023-09-21      9
2023-09-18      9
2023-02-01      8
2023-08-13      8
2023-08-06      7
2023-01-03      7
2022-10-28      7
2023-08-20      7
2022-12-31      7
2023-07-15      7
2023-07-10      6
2023-08-15      6
2023-04-24      5
2021-12-31      5
2022-10-01      5
2023-05-16      5
2023-09-13      5
2023-06-02      5
2023-06-01      5
2023-04-06

In [162]:
doors_americas

,created_week,country,city,prep_organization_id,prep_organization_name,seller_organization_id,seller_organization_name,facility_id,facility_name,inventory_owned,distribution_type,total_orders,cancelled_orders,issue_orders,ODR_orders,scanned_orders,Avg_Items_per_Order,total_gmv_local,lost_gmv_local,cancelled_gmv_local,OFOs,cities,facilities,is_churned,churn_date,closed_won_date,activated_date,billable_date,total_cu_ft,MRR_local,processing_fee_pct,pct_cancelled,pct_issues,pct_odr,pct_scanned,succesful_orders,succesful_gmv_local,processing_income_local,store_gmv_mrr,gmv_mrr_cat,exchange_rate,mrr_usd,total_gmv_usd,cancelled_gmv_usd,lost_gmv_usd,succesful_gmv_usd,processing_income_usd,Start_Date__c,Move_In_Date__c,CR_Opp_OB_Status__c,data_pulled_ts_MEX,data_pulled_ts_LAX
0,2023-08-07,USA,Chicago,017b46ad-3a10-4d42-80f2-b3b84abb1019,​Tubby's Taste LLC,017b46ad-3a10-4d42-80f2-b3b84abb1019,​Tubby's Taste LLC,75187d2a-01d8-467f-9993-90e200a3a586,NA-US-IL-CHI-River West (Correct),False,3PCR,1,0,0,0,1,3.000000,27.00,0.0,0.00,1,1,1,False,NaT,2023-07-20,2023-08-10,,,NaN,NaN,0.000000,0.0,0.000000,1.0,1,27.00,NaN,NaN,0X,1.00,NaN,27.000000,0.00,0.0,27.000000,NaN,NaN,NaN,NaN,2023-09-20 17:57:04.219296-05:00,2023-09-20 15:57:04.220804-07:00
1,2023-08-07,USA,Dallas,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,4411e09a-c8b7-4a2e-ae6e-5d028e303791,NA-US-TX-DAL-Commerce,False,3PCR,6,5,0,5,0,2.333333,127.66,0.0,106.68,1,1,1,False,NaT,NaT,,,,NaN,NaN,0.833333,0.0,0.833333,0.0,1,20.98,NaN,NaN,0X,1.00,NaN,127.660000,106.68,0.0,20.980000,NaN,NaN,NaN,NaN,2023-09-20 17:57:04.219296-05:00,2023-09-20 15:57:04.220804-07:00
2,2023-07-31,USA,Dallas,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,4411e09a-c8b7-4a2e-ae6e-5d028e303791,NA-US-TX-DAL-Commerce,False,3PCR,2,2,0,2,0,2.000000,37.72,0.0,37.72,1,1,1,False,NaT,NaT,,,,NaN,NaN,1.000000,0.0,1.000000,0.0,0,0.00,NaN,NaN,0X,1.00,NaN,37.720000,37.72,0.0,0.000000,NaN,NaN,NaN,NaN,2023-09-20 17:57:04.219296-05:00,2023-09-20 15:57:04.220804-07:00
3,2023-08-14,USA,Dallas,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,658618ea-2e32-47ba-8a6b-fe89342ad1c5,​Pickle Mee Munchies,4411e09a-c8b7-4a2e-ae6e-5d028e303791,NA-US-TX-DAL-Commerce,False,3PCR,1,1,0,1,0,1.000000,3.99,0.0,3.99,1,1,1,False,NaT,NaT,,,,NaN,NaN,1.000000,0.0,1.000000,0.0,0,0.00,NaN,NaN,0X,1.00,NaN,3.990000,3.99,0.0,0.000000,NaN,NaN,NaN,NaN,2023-09-20 17:57:04.219296-05:00,2023-09-20 15:57:04.220804-07:00
4,2023-07-31,USA,Los Angeles,6440b884-ccd2-465d-b220-b93b8c7cda62,p.o.p. candy co.,6440b884-ccd2-465d-b220-b93b8c7cda62,p.o.p. candy co.,99573ac0-a3f2-4c70-946c-e9c06b425d5c,NA-US-CA(S)-LA-Koreatown,False,3PCR,2,2,0,2,1,2.000000,64.00,0.0,64.00,1,1,1,True,2023-05-16,2023-01-25,2023-02-10,,,NaN,0.15,1.000000,0.0,1.000000,0.5,0,0.00,0.000,NaN,0X,1.00,NaN,64.000000,64.00,0.0,0.000000,0.000000,NaN,NaN,NaN,2023-09-20 17:57:04.219296-05:00,2023-09-20 15:57:04.220804-07:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15970,2023-05-29,Brazil,Sao Paulo,9a5914a1-2b47-487b-ad56-0e4f492515d9,Nah's Bakery - Pik n' Pak,9a5914a1-2b47-487b-ad56-0e4f492515d9,Nah's Bakery - Pik n' Pak,dcc461ae-c80c-46cc-a5dc-4a5a72761510,SAO - Lapa Clelia,False,3PCR,2,0,0,0,2,1.000000,67.80,0.0,0.00,1,1,1,True,2023-05-26,2022-09-21,NaN,NaN,4.0,500.00,0.07,0.000000,0.0,0.000000,1.0,2,67.80,4.746,0.581143,0X,5.01,99.800399,13.532934,0.00,0.0,13.532934,0.947305,2022-11-01,2022-10-28,i - Churned,2023-09-20 17:57:04.219296-05:00,2023-09-20 15:57:04.220804-07:00
15971,2023-06-26,Brazil,Rio de Janeiro,048caf43-8454-4693-8bca-f78a474a8673,Viva o Doce,048caf43-8454-4693-8bca-f78a474a8673,Viva o Doce,466abb82-57b9-4e0d-9c5b-d1ef6c7f92bb,RIO - Barra,False,3PCR,2,0,0,0,2,1.500000,89.99,0.0,0.00

In [163]:
#group_monthly=pd.pivot_table(data=doors_americas, index=['country', 'city', 'prep_organization_id',
      #'facility_id'], columns=(doors_americas['created_month'].dt.date),
             # values=['total_orders','cancelled_orders', 'issue_orders', 'ODR_orders',
                    # 'cancelled_gmv_usd', 'succesful_gmv_usd'], aggfunc='sum').fillna(0).reset_index()

In [164]:
#group_monthly.columns[1]

In [165]:
#nuevas_nombres_mensual=[]
#for i in range(len(group_monthly.columns)):
    #try: 
       # nuevo=group_monthly.columns[i][0]+str(group_monthly.columns[i][1])
    #except:
        #nuevo=str(group_monthly.columns[i][0])
    #nuevas_nombres_mensual.append(nuevo)

In [166]:
#nuevas_nombres_mensual

In [167]:
#new_names=dict(zip(group_monthly.columns,nuevas_nombres_mensual))

In [168]:
#new_names

In [169]:
#group_monthly.columns=nuevas_nombres_mensual

In [170]:
from google.oauth2 import service_account
from ds.utilities.secrets import secrets
import pygsheets
SCOPES = ('https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive')
s = secrets.Secrets()
credentials_sheets = service_account.Credentials.from_service_account_info(s.get_secret("cloud_retail_latam_service_account"),scopes=SCOPES)
#gc = pygsheets.authorize(custom_credentials=credentials)

In [173]:
# open the spreedsheet
print('Opening Spreadsheet')
gc = pygsheets.authorize(custom_credentials=credentials_sheets)

ss = gc.open_by_key('1-4ASzH3Yg9ixQlCup8k15LFc6JSrl9JPpCcj7aA27Po')

# select sheet

ws = ss.worksheet('title','D_Orders')

print('Clearing Range ...')
ws.clear(start='A', end='AI')
print('Range CLEARED')

print('Pasting Info ...')
ws.set_dataframe(doors_americas, start="A1",copy_head=True, extend=True, nan= '' )
print('UPDATED!!')

Opening Spreadsheet
Clearing Range ...
Range CLEARED
Pasting Info ...
UPDATED!!
